In [ ]:
%pip install numpy matplotlib scipy

In [2]:
### Cell: Imports

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union, Dict
import time

# For inline plots in Jupyter
%matplotlib inline

EPS = 1e-12

In [ ]:
%pip install librosa

In [ ]:
import librosa
import numpy as np
from scipy.signal import gammatone
from typing import List, Tuple

class AudioInput:
    def __init__(self, sample_rate: int = 44100):
        self.sample_rate = sample_rate
        
    def load_audio(self, file_path: str) -> np.ndarray:
        """Load and normalize audio file"""
        audio, sr = librosa.load(file_path, sr=self.sample_rate)
        return audio / np.max(np.abs(audio))

class CochlearFilter:
    def __init__(self, num_bands: int = 64, min_freq: float = 20, max_freq: float = 20000):
        self.num_bands = num_bands
        self.frequencies = np.logspace(np.log10(min_freq), np.log10(max_freq), num_bands)
        
    def create_filterbank(self, sample_rate: int) -> List[np.ndarray]:
        """Create gammatone filterbank"""
        filters = []
        for freq in self.frequencies:
            b = gammatone(freq, sample_rate)
            filters.append(b)
        return filters
    
    def process_signal(self, audio: np.ndarray, sample_rate: int) -> np.ndarray:
        """Split audio into frequency bands"""
        filterbank = self.create_filterbank(sample_rate)
        filtered_signals = np.zeros((self.num_bands, len(audio)))
        
        for i, filt in enumerate(filterbank):
            filtered_signals[i] = np.abs(np.convolve(audio, filt, mode='same'))
            # Normalize each band
            filtered_signals[i] /= np.max(np.abs(filtered_signals[i])) + 1e-6
            
        return filtered_signals

class NeuralPreprocessor:
    def __init__(self, time_window: int = 1000):
        self.time_window = time_window
        
    def prepare_neural_input(self, filtered_signals: np.ndarray) -> List[np.ndarray]:
        """Convert filtered signals to neural input format"""
        num_bands, signal_length = filtered_signals.shape
        neural_inputs = []
        
        for band in range(num_bands):
            # Reshape into time windows
            windows = np.array([filtered_signals[band, i:i+self.time_window] 
                              for i in range(0, signal_length-self.time_window, self.time_window)])
            neural_inputs.append(windows)
            
        return neural_inputs

# Usage example
if __name__ == "__main__":
    audio_processor = AudioInput()
    cochlear = CochlearFilter()
    neural_prep = NeuralPreprocessor()
    
    # Process audio file
    audio = audio_processor.load_audio("input.wav")
    filtered = cochlear.process_signal(audio, audio_processor.sample_rate)
    neural_input = neural_prep.prepare_neural_input(filtered)

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Optional
from fractions import Fraction
import math

EPS = 1e-12

def generate_sine_lookup(resolution):
    """Generates a lookup table for a quarter sine wave, with rational values."""
    table = []
    for i in range(resolution + 1):
        angle = Fraction(i, resolution) * Fraction(1, 2)
        # Use a Taylor series approximation of sin(x) for small angles (x<1)
        sin_approx = angle - (angle**3) / Fraction(6) + (angle**5) / Fraction(120)
        table.append(sin_approx)
    return table

def rational_sine(t, freq, resolution, sine_lookup):
    """Generates a sine wave using the lookup table and symmetry"""
    phase = (t * freq) % 1
    phase *= 4
    index = int(abs(phase) * resolution)
    if index > resolution:
        index = resolution
    if index < 0:
        index = 0
    value = sine_lookup[index]
    if phase > 1 and phase < 2:
        value = 1 - value
    elif phase > 2 and phase < 3:
        value = -value
    elif phase > 3:
        value = value - 1
    return value

def rational_window_lookup(resolution, window_type = "hann"):
  """Generates a lookup table for a window function, with rational values."""
  if window_type == "hann":
    table = []
    for i in range(resolution+1):
        angle = Fraction(i,resolution) * 2 * math.pi # use 2pi here.
        window_value = 0.5 - (0.5 * math.cos(angle))
        table.append(Fraction(window_value).limit_denominator(100))
  return table

def rational_window(n, window_size, window_lookup):
  """Generates a window using a lookup table"""
  index = int(n * len(window_lookup) / window_size)
  if index > len(window_lookup) - 1:
      index = len(window_lookup) - 1
  if index < 0:
      index = 0
  return window_lookup[index]

def generate_acos_lookup(resolution):
  table = []
  pi_fraction = Fraction(22, 7) # You can use other approximations of pi
  for i in range(resolution + 1):
        x = Fraction(i, resolution) * 2 -1 # map to -1 to 1
        # Using a rational approximation for arccos using Taylor series expansion
        acos_approx =  pi_fraction / 2 - (x - (x**3) / 6 - (3 * x**5) / 40 - (5 * x**7) / 112)
        table.append(acos_approx)
  return table


def rational_acos(x, resolution, acos_lookup):
    """Generates acos using a lookup table"""
    x = min(max(x, -1), 1) # clip the value of x
    index = int((x+1) * resolution / 2)
    if index > resolution:
        index = resolution
    if index < 0:
        index = 0
    return acos_lookup[index]

def partial_interference_sphere(base: np.ndarray, new: np.ndarray, alpha: float = 1.0, resolution = 100, acos_lookup = None) -> np.ndarray:
    """
    Original partial interference on the 4D hypersphere without resonance.
    """
    def normalize_4d(vec):
        norm = np.linalg.norm(vec)
        if norm < EPS:
            return np.array([1.0, 0.0, 0.0, 0.0], dtype = float)
        return vec / norm
    
    p_n = normalize_4d(base)
    q_n = normalize_4d(new)
    dot_ = np.clip(np.dot(p_n, q_n), -1.0, 1.0)
    angle = rational_acos(dot_, resolution, acos_lookup) # now using rational acos

    if angle < EPS:
        return p_n
    
    perp = q_n - dot_ * p_n
    perp_norm = np.linalg.norm(perp)
    if perp_norm < EPS:
        return p_n
    
    dir_ = perp / perp_norm
    two_pi = Fraction(2 * 22, 7) # change it here!
    v_scaled = (alpha * Fraction(angle, two_pi) * two_pi) * dir_ #  alpha * angle in terms of 2*pi * 2*pi
    norm_v = np.linalg.norm(v_scaled)
    if norm_v < EPS:
        return p_n
    
    new_point = np.cos(norm_v) * p_n + np.sin(norm_v) * (v_scaled / norm_v)
    return normalize_4d(new_point)

############################################################
# 1. Resonant Partial Interference (Optional)
############################################################
def partial_interference_sphere_resonant(
    base: np.ndarray,
    new: np.ndarray,
    alpha: float = 1.0,
    resonance_gain: float = 0.3,
    resolution = 100,
    acos_lookup = None
) -> np.ndarray:
    """
    A resonant version of partial_interference_sphere that includes 
    a phase-sensitive gain factor. In-phase waves get amplified, 
    out-of-phase waves get damped.
    
    Steps:
      1) Standard partial interference on the 4D sphere
      2) Multiply the result by a gain_factor = (1 + resonance_gain * dot),
         clamped [0.5..2.0], to emphasize in-phase signals
      3) Re-normalize
    """
    def normalize_4d(vec):
        norm = np.linalg.norm(vec)
        if norm < EPS:
            return np.array([1.0, 0.0, 0.0, 0.0], dtype = float)
        return vec / norm
    
    p_n = normalize_4d(base)
    q_n = normalize_4d(new)
    
    dot_ = np.clip(np.dot(p_n, q_n), -1.0, 1.0)
    angle = rational_acos(dot_, resolution, acos_lookup) # now using rational acos
    
    if angle < EPS:
        return p_n

    # Standard rotation on the sphere
    perp = q_n - dot_ * p_n
    perp_norm = np.linalg.norm(perp)
    if perp_norm < EPS:
        return p_n
    
    dir_ = perp / perp_norm
    v_scaled = (alpha * Fraction(angle, 2 * math.pi) * 2*math.pi) * dir_ #  alpha * angle in terms of 2*pi * 2*pi
    norm_v = np.linalg.norm(v_scaled)
    if norm_v < EPS:
        return p_n
    
    new_point = (np.cos(norm_v) * p_n 
                + np.sin(norm_v) * (v_scaled / norm_v))

    # Apply resonance-based gain
    gain_factor = 1.0 + resonance_gain * dot_
    gain_factor = np.clip(gain_factor, 0.5, 2.0)
    new_point *= gain_factor
    return normalize_4d(new_point)

############################################################
# 2. Standard Partial Interference (Non-Resonant)
############################################################
def partial_interference_sphere_resonant(
    base: np.ndarray,
    new: np.ndarray,
    alpha: float = 1.0,
    resonance_gain: float = 0.3,
    resolution = 100,
    acos_lookup = None
) -> np.ndarray:
    """
    A resonant version of partial_interference_sphere that includes 
    a phase-sensitive gain factor. In-phase waves get amplified, 
    out-of-phase waves get damped.
    
    Steps:
      1) Standard partial interference on the 4D sphere
      2) Multiply the result by a gain_factor = (1 + resonance_gain * dot),
         clamped [0.5..2.0], to emphasize in-phase signals
      3) Re-normalize
    """
    def normalize_4d(vec):
        norm = np.linalg.norm(vec)
        if norm < EPS:
            return np.array([1.0, 0.0, 0.0, 0.0], dtype = float)
        return vec / norm
    
    p_n = normalize_4d(base)
    q_n = normalize_4d(new)
    
    dot_ = np.clip(np.dot(p_n, q_n), -1.0, 1.0)
    angle = rational_acos(dot_, resolution, acos_lookup) # now using rational acos
    
    if angle < EPS:
        return p_n

    # Standard rotation on the sphere
    perp = q_n - dot_ * p_n
    perp_norm = np.linalg.norm(perp)
    if perp_norm < EPS:
        return p_n
    
    dir_ = perp / perp_norm
    two_pi = Fraction(2 * 22, 7) # change it here!
    v_scaled = (alpha * Fraction(angle, two_pi) * two_pi) * dir_ #  alpha * angle in terms of 2*pi * 2*pi
    norm_v = np.linalg.norm(v_scaled)
    if norm_v < EPS:
        return p_n
    
    new_point = (np.cos(norm_v) * p_n 
                + np.sin(norm_v) * (v_scaled / norm_v))

    # Apply resonance-based gain
    gain_factor = 1.0 + resonance_gain * dot_
    gain_factor = np.clip(gain_factor, 0.5, 2.0)
    new_point *= gain_factor
    return normalize_4d(new_point)

import numpy as np
import matplotlib.pyplot as plt

class DendriticModulation:
    def __init__(self, threshold=0.5, coupling_strength=0.8):
        self.threshold = threshold
        self.coupling = coupling_strength
    
    def nonlinear_response(self, input_signal):
        # Sigmoid-like nonlinear response similar to Purkinje dendrites
        return 1 / (1 + np.exp(-(input_signal - self.threshold)))
    
    def modulate(self, carrier_wave, input_signal):
        # Combine signals with dendritic-like nonlinearity
        interference = self.coupling * carrier_wave * input_signal
        modulated = self.nonlinear_response(interference)
        return modulated

# Generate test signals
t = np.linspace(0, 1, 1000)
carrier = np.sin(2 * np.pi * 10 * t)  # 10Hz carrier
input_wave = np.sin(2 * np.pi * 2 * t)  # 2Hz input

# Create modulator and process signals
modulator = DendriticModulation()
output = modulator.modulate(carrier, input_wave)

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(t, output, label='Modulated Output')
plt.plot(t, carrier, 'r--', alpha=0.5, label='Carrier')
plt.plot(t, input_wave, 'g--', alpha=0.5, label='Input')
plt.legend()
plt.title('Dendritic-like Amplitude Modulation')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

############################################################
# 3. WaveSphere Class (fast/slow states + optional resonance)
############################################################
class WaveSphere:
    """
    A single 'neuron-like' wave sphere with:
      - fast_state: quick timescale
      - slow_state: slower amplitude memory
      - optional resonance_gain for in-phase amplification on inputs
    """

    def __init__(self, 
                 fast_alpha=0.8, 
                 slow_alpha=0.2, 
                 noise_std=0.0, 
                 resonance_gain=0.0,
                 rotation_type="simple",
                 sine_resolution = 100,
                 window_resolution = 100,
                 acos_resolution = 100):
        self.fast_alpha = fast_alpha
        self.slow_alpha = slow_alpha
        self.noise_std = noise_std
        self.resonance_gain = resonance_gain
        self.rotation_type = rotation_type
        self.sine_lookup = generate_sine_lookup(sine_resolution)
        self.window_lookup = rational_window_lookup(window_resolution)
        self.acos_lookup = generate_acos_lookup(acos_resolution)
        self.sine_resolution = sine_resolution
        self.window_resolution = window_resolution
        self.acos_resolution = acos_resolution
        self.reset()
    
    def reset(self) -> None:
        self.fast_state = np.array([1.0, 0.0, 0.0, 0.0], dtype=float)
        self.slow_state = np.array([1.0, 0.0, 0.0, 0.0], dtype=float)

    @staticmethod
    def normalize_sphere_4d(vec: np.ndarray) -> np.ndarray:
        norm = np.linalg.norm(vec)
        if norm < EPS:
            return np.array([1.0, 0.0, 0.0, 0.0], dtype = float)
        return vec / norm
    
    def _two_step_rotation(self, base, new, alpha, resonance_gain):
      """Performs a two-step rotation with an intermediate state"""
      # 1) Rotate towards intermediate vector based on the dot product
      dot_product = np.clip(np.dot(base, new), -1.0, 1.0)
      intermediate = base + alpha * dot_product * (new - base)
      intermediate = WaveSphere.normalize_sphere_4d(intermediate)

      # 2) Rotate towards the new input using the standard partial interference
      if resonance_gain > 0:
          new_state = partial_interference_sphere_resonant(
              intermediate, new, alpha, resonance_gain, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
          )
      else:
          new_state = partial_interference_sphere(
              intermediate, new, alpha, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
          )
      return new_state
    
    def _modulated_rotation(self, base, new, alpha, resonance_gain):
      """Performs a single-step rotation with alpha modulation"""
      # 1) Modulate alpha based on dot_product with a sigmoid like function
      dot_product = np.clip(np.dot(base, new), -1.0, 1.0)
      alpha_mod = alpha * (1 / (1 + np.exp(-(dot_product - 0.5))))
      
      # 2) Rotate using partial interference using modulated alpha
      if resonance_gain > 0:
          new_state = partial_interference_sphere_resonant(
              base, new, alpha_mod, resonance_gain, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
          )
      else:
          new_state = partial_interference_sphere(
              base, new, alpha_mod, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
          )
      return new_state


    def inject_random_spike(self, amplitude=1.0) -> None:
        """
        Inject random noise into fast_state if noise_std > 0
        """
        if self.noise_std >= EPS:
            rnd = np.random.randn(4) * self.noise_std * amplitude
            spike_target = WaveSphere.normalize_sphere_4d(rnd)
            if self.rotation_type == "two_step":
              self.fast_state = self._two_step_rotation(
                self.fast_state, spike_target, alpha=0.5, resonance_gain = 0
            )
            elif self.rotation_type == "modulated":
              self.fast_state = self._modulated_rotation(
                  self.fast_state, spike_target, alpha=0.5, resonance_gain = 0
              )
            else:
              self.fast_state = partial_interference_sphere(
                self.fast_state, spike_target, alpha=0.5, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
              )

    def update_with_input(self, input_wave: np.ndarray, gating=True) -> None:
        """
        Merge input_wave with fast and slow states. If resonance_gain > 0,
        use partial_interference_sphere_resonant for an in-phase boost.
        """
        if not gating:
            return
        
        if self.rotation_type == "two_step":
          # Two Step approach
          self.fast_state = self._two_step_rotation(
              self.fast_state, 
              input_wave, 
              alpha=self.fast_alpha,
              resonance_gain=self.resonance_gain
          )
          self.slow_state = self._two_step_rotation(
                self.slow_state, 
                input_wave, 
                alpha=self.slow_alpha,
                resonance_gain=self.resonance_gain
            )
        elif self.rotation_type == "modulated":
          # Modulated Approach
          self.fast_state = self._modulated_rotation(
                self.fast_state, 
                input_wave, 
                alpha=self.fast_alpha,
                resonance_gain=self.resonance_gain
            )
          self.slow_state = self._modulated_rotation(
                self.slow_state, 
                input_wave, 
                alpha=self.slow_alpha,
                resonance_gain=self.resonance_gain
            )
        elif self.resonance_gain > 0.0:
            # Resonant approach
            self.fast_state = partial_interference_sphere_resonant(
                self.fast_state, 
                input_wave, 
                alpha=self.fast_alpha,
                resonance_gain=self.resonance_gain, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
            )
            self.slow_state = partial_interference_sphere_resonant(
                self.slow_state, 
                input_wave, 
                alpha=self.slow_alpha,
                resonance_gain=self.resonance_gain, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
            )
        else:
            # Standard partial interference
            self.fast_state = partial_interference_sphere(
                self.fast_state, 
                input_wave,
                alpha=self.fast_alpha, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
            )
            self.slow_state = partial_interference_sphere(
                self.slow_state, 
                input_wave,
                alpha=self.slow_alpha, resolution = self.acos_resolution, acos_lookup = self.acos_lookup
            )

    def combine_states(self) -> np.ndarray:
        """
        Merge fast & slow states for final output. Typically non-resonant here,
        but you could swap in resonant if you want.
        """
        return partial_interference_sphere(self.fast_state, self.slow_state, alpha=0.5, resolution = self.acos_resolution, acos_lookup = self.acos_lookup)

############################################################
# 4. Data class for neighbor overlaps
############################################################
@dataclass
class SphereOverlap:
    idx_A: int
    idx_B: int
    reflection_coeff: float
    transmission_coeff: float

############################################################
# 5. MultiSphereWaveModel
############################################################
class MultiSphereWaveModel:
    """
    Attributes:
    -----------
    M : int
        Number of spheres in the model.
    noisy_indices : list
        Indices of spheres where noise is injected.
    spheres : list
        List of WaveSphere objects representing each sphere.
    overlaps : list
        List of SphereOverlap objects representing the overlaps between neighboring spheres.
    Methods:
    --------
    __init__(M=3, reflection=0.2, transmission=0.7, noise_std=0.0, resonance_gain=0.0, noisy_indices=None):
        Initializes the MultiSphereWaveModel with given parameters.
    set_initial_state(idx, fast_vec, slow_vec=None):
    _process_boundary_interactions(new_fast, new_slow):
    step(input_waves=None, gating_flags=None):
        One step of wave propagation.
    run(steps=10, input_waves_seq=None, gating_seq=None):
    
    A multi-sphere wave memory model. 
    We can:
     - Only inject noise in certain spheres (noisy_indices)
     - Use resonance_gain if we want wave queries to be amplified
     - Perform reflection/transmission among neighbors
    """

    def __init__(self, 
                 M=3, 
                 reflection=0.2, 
                 transmission=0.7, 
                 noise_std=0.0,
                 resonance_gain=0.0,
                 noisy_indices=None,
                 rotation_type="simple",
                 sine_resolution = 100,
                 window_resolution = 100,
                 acos_resolution = 100):
        self.M = M
        self.noisy_indices = noisy_indices if noisy_indices else []
        
        # Create M spheres
        self.spheres = []
        for i in range(M):
            ns = noise_std if i in self.noisy_indices else 0.0
            sphere = WaveSphere(fast_alpha=0.8,
                                slow_alpha=0.2,
                                noise_std=ns,
                                resonance_gain=resonance_gain,
                                rotation_type = rotation_type,
                                sine_resolution = sine_resolution,
                                window_resolution = window_resolution,
                                acos_resolution = acos_resolution)
            self.spheres.append(sphere)

        # Chain-based overlaps
        self.overlaps = [
            SphereOverlap(i, i+1, reflection, transmission)
            for i in range(M-1)
        ]

    def set_initial_state(self, idx, fast_vec, slow_vec=None):
        """
        Initialize a sphere's fast/slow states manually.
        """
        self.spheres[idx].fast_state = WaveSphere.normalize_sphere_4d(np.array(fast_vec, dtype=float))
        if slow_vec is not None:
            self.spheres[idx].slow_state = WaveSphere.normalize_sphere_4d(np.array(slow_vec, dtype=float))

    def _process_boundary_interactions(self, new_fast, new_slow):
        """
        Reflect & transmit neighbor states using standard partial_interference_sphere.
        You could also do resonant interference if you want in-phase neighbor boosting.
        """
        for overlap in self.overlaps:
            iA, iB = overlap.idx_A, overlap.idx_B
            r_coef, t_coef = overlap.reflection_coeff, overlap.transmission_coeff

            dotF = np.clip(np.dot(new_fast[iA], new_fast[iB]), -1.0, 1.0)
            dotS = np.clip(np.dot(new_slow[iA], new_slow[iB]), -1.0, 1.0)

            scaleF = (1.0 - 0.5*abs(dotF)) if dotF < 0 else (1.0 + 0.2*dotF)
            scaleS = (1.0 - 0.5*abs(dotS)) if dotS < 0 else (1.0 + 0.2*dotS)

            alphas = {
                'fr': np.clip(r_coef * scaleF, 0, 1),
                'ft': np.clip(t_coef * scaleF, 0, 1),
                'sr': np.clip(r_coef * scaleS, 0, 1),
                'st': np.clip(t_coef * scaleS, 0, 1)
            }

            # partial_interference for neighbor merges
            fA_merged = partial_interference_sphere(new_fast[iA], new_fast[iB], alpha=alphas['fr'], resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)
            sA_merged = partial_interference_sphere(new_slow[iA], new_slow[iB], alpha=alphas['sr'], resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)
            fB_merged = partial_interference_sphere(new_fast[iB], new_fast[iA], alpha=alphas['ft'], resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)
            sB_merged = partial_interference_sphere(new_slow[iB], new_slow[iA], alpha=alphas['st'], resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)

            # Then partially merge them back
            new_fast[iA] = partial_interference_sphere(new_fast[iA], fA_merged, alpha=0.5, resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)
            new_slow[iA] = partial_interference_sphere(new_slow[iA], sA_merged, alpha=0.5, resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)
            new_fast[iB] = partial_interference_sphere(new_fast[iB], fB_merged, alpha=0.5, resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)
            new_slow[iB] = partial_interference_sphere(new_slow[iB], sB_merged, alpha=0.5, resolution = self.spheres[iA].acos_resolution, acos_lookup = self.spheres[iA].acos_lookup)

        return new_fast, new_slow

    def step(self, input_waves=None, gating_flags=None):
        """
        One step of wave propagation:
         - Only spheres in self.noisy_indices get random spikes
         - Merge with external input if any (resonant if resonance_gain>0)
         - Reflect/transmit among neighbors
         - Return combined states
        """
        input_waves = input_waves or [None]*self.M
        gating_flags = gating_flags or [True]*self.M

        # 1) Noise injection & external input merges
        for i, sphere in enumerate(self.spheres):
            sphere.inject_random_spike(amplitude=1.0)
            if input_waves[i] is not None:
                wave_in = WaveSphere.normalize_sphere_4d(input_waves[i])
                sphere.update_with_input(wave_in, gating=gating_flags[i])

        # 2) Gather fresh states
        new_fast = [s.fast_state.copy() for s in self.spheres]
        new_slow = [s.slow_state.copy() for s in self.spheres]

        # 3) Reflect/transmit among neighbors
        new_fast, new_slow = self._process_boundary_interactions(new_fast, new_slow)

        # 4) Update sphere states
        for i, sphere in enumerate(self.spheres):
            sphere.fast_state = new_fast[i]
            sphere.slow_state = new_slow[i]

        # 5) Return combined states
        return [sphere.combine_states() for sphere in self.spheres]

    def run(self, steps=10, input_waves_seq=None, gating_seq=None):
        """
        Perform multiple time steps, storing final combined states at each step.
        """
        input_waves_seq = input_waves_seq or [[None]*self.M for _ in range(steps)]
        gating_seq = gating_seq or [[True]*self.M for _ in range(steps)]

        history = []
        for t in range(steps):
            outs = self.step(input_waves_seq[t], gating_seq[t])
            history.append(outs)
        return np.array(history)

############################################################
# 6. Demo: Frequency-Based Amplitude Modulation
############################################################

def generate_frequency_wave(t: int, freq: float, amplitude: float = 1.0, phase: float = 0.0) -> np.ndarray:
    """
    Example function to produce a time-varying wave in 4D, 
    based on a single frequency for demonstration.
    
    We make a 4D vector using sin and cos in a simple pattern:
      [ amplitude*sin(freq*t + phase), amplitude*cos(freq*t + phase),
        amplitude*sin(2*freq*t + phase), amplitude*cos(2*freq*t + phase) ]
    Then normalize it to simulate a wave's direction with amplitude mod.
    """
    # In a real system, you might do something more elaborate
    raw_vec = np.array([
        amplitude * np.sin(freq*t + phase),
        amplitude * np.cos(freq*t + phase),
        amplitude * np.sin(2*(freq*t + phase)),
        amplitude * np.cos(2*(freq*t + phase))
    ])
    norm = np.linalg.norm(raw_vec)
    if norm < EPS:
        return np.array([1,0,0,0], dtype=float)
    return raw_vec / norm

############################################################
# 7. Bit Processor Class (Example Bitwise Layer)
############################################################
class BitProcessor:
    def __init__(self, theta_freq=0.3, bit_window=4, sine_resolution = 100):
        self.theta_freq = theta_freq
        self.bit_window = bit_window
        self.bit_history = []
        self.sine_lookup = generate_sine_lookup(sine_resolution)
        self.sine_resolution = sine_resolution

    def process_bit(self, bit: int) -> np.ndarray:
        """
        Process a single bit, updates the window of bits,
        and generates an output wave.
        """
        self.bit_history.append(bit)
        if len(self.bit_history) > self.bit_window:
            self.bit_history.pop(0)

        # Apply XOR pattern to select input frequency
        xor_pattern = 0
        for i, b in enumerate(reversed(self.bit_history)):
          xor_pattern += b * (2**i)

        input_freq = self.theta_freq * (xor_pattern + 1) # change frequency based on pattern
        wave = generate_frequency_wave(0, input_freq) # phase modulation here.
        
        return wave # bit level + XOR

############################################################
# SINGLE-CELL DEMO
############################################################
if __name__ == "__main__":
    # We'll build a small wave model with 3 spheres.
    # - Sphere0 gets noise
    # - We'll feed it time-varying wave inputs derived from a single frequency = 0.3 rad/step
    # - We'll see amplitude/phase evolve over 10 steps.

    model = MultiSphereWaveModel(
        M=3, 
        reflection=0.2, 
        transmission=0.7, 
        noise_std=0.05,   # base noise for spheres in noisy_indices
        resonance_gain=0.3,  # allow in-phase wave queries to be amplified
        noisy_indices=[0],  # only sphere #0 gets random spikes
        rotation_type = "two_step", # "modulated" , or "simple"
        sine_resolution = 100,
        window_resolution = 100,
        acos_resolution = 100
    )

    # Optionally set initial states
    model.set_initial_state(0, [1.0, 0.0, 0.0, 0.0])
    model.set_initial_state(1, [0.7, 0.7, 0.0, 0.0])
    model.set_initial_state(2, [0.0, 1.0, 0.0, 0.0])
    
    steps = 10
    #freq = 0.3  # rad/step
    amplitude = 0.8
    bit_processor = BitProcessor()

    # Example input bit sequence
    bit_sequence = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]

    # Build the input sequence with a frequency-based wave for sphere0
    input_waves_seq = []
    gating_seq = []
    for bit in bit_sequence:
        wave0 = bit_processor.process_bit(bit)
        # We'll feed wave0 to sphere0, none to others
        input_waves_seq.append([wave0, None, None])
        gating_seq.append([True, False, False])

    history = model.run(steps=steps, 
                        input_waves_seq=input_waves_seq, 
                        gating_seq=gating_seq)

    print("\nFinal states after {} steps:".format(steps))
    print(history[-1])

In [4]:
import numpy as np
from typing import List, Optional, Union
from dataclasses import dataclass

@dataclass 
class WaveConfig:
    """Configuration for wave frequencies and duty cycles"""
    # Core neural frequencies
    theta_freq: float = 5.0      # 4-8 Hz theta rhythm
    gamma_freq: float = 40.0     # 30-90 Hz gamma
    
    # Ion channel frequencies
    calcium_freq: float = 12000.0  
    sodium_freq: float = 15000.0
    potassium_freq: float = 17000.0
    
    # Duty cycles
    theta_duty: float = 0.5
    gamma_duty: float = 0.3  
    ion_duty: float = 0.3
    
    # Timing windows
    stdp_window: int = 10
    sample_rate: int = 44100

class WaveMemory:
    """Core wave memory system with ion channels and STDP"""
    
    def __init__(self, config: WaveConfig):
        self.config = config
        
        # Ion channel states
        self.calcium_state = 0
        self.sodium_state = 0
        self.potassium_state = 0
        
        # Timing windows
        self.pre_history = [0] * config.stdp_window
        self.post_history = [0] * config.stdp_window
        
        # State history
        self.history = []

    def generate_wave(self, t: float, freq: float, duty: float) -> int:
        """Generate binary wave with duty cycle"""
        phase = (t * freq) % 1.0
        return 1 if phase < duty else 0

    def self_interfere(self, state: int, delay: int = 1) -> int:
        """Natural decay through self-interference"""
        return state ^ (state >> delay)
    
    def update_ion_channels(self, input_val: int, t: float):
        """Update ion channel states through interference"""
        # Generate carrier waves
        ca_wave = self.generate_wave(t, self.config.calcium_freq, self.config.ion_duty)
        na_wave = self.generate_wave(t, self.config.sodium_freq, self.config.ion_duty)
        k_wave = self.generate_wave(t, self.config.potassium_freq, self.config.ion_duty)
        
        # Update each channel through interference and self-interference
        self.calcium_state ^= (ca_wave & input_val)
        self.calcium_state = self.self_interfere(self.calcium_state)
        
        self.sodium_state ^= (na_wave & input_val)
        self.sodium_state = self.self_interfere(self.sodium_state)
        
        self.potassium_state ^= (k_wave & input_val)
        self.potassium_state = self.self_interfere(self.potassium_state)

    def process_stdp(self, pre_spike: int, post_spike: int, t: float) -> int:
        """Process STDP using theta-gamma coupling"""
        # Update timing histories
        self.pre_history = [pre_spike] + self.pre_history[:-1]
        self.post_history = [post_spike] + self.post_history[:-1]
        
        # Generate timing waves
        theta = self.generate_wave(t, self.config.theta_freq, self.config.theta_duty)
        gamma = self.generate_wave(t, self.config.gamma_freq, self.config.gamma_duty)
        
        # Theta gates gamma
        gated_gamma = theta & gamma
        
        # Calculate timing-based effects
        stdp_effect = 0
        
        # Pre-before-post: strengthen through constructive interference
        for i in range(self.config.stdp_window - 1):
            if self.pre_history[i] and self.post_history[i+1]:
                stdp_effect ^= gated_gamma
                
        # Post-before-pre: weaken through destructive interference  
        for i in range(self.config.stdp_window - 1):
            if self.post_history[i] and self.pre_history[i+1]:
                stdp_effect &= ~gated_gamma
                
        return stdp_effect

    def update(self, input_val: int, t: float) -> int:
        """Full update incorporating all mechanisms"""
        # Get current combined state
        current = self.calcium_state ^ self.sodium_state ^ self.potassium_state
        
        # Update ion channels
        self.update_ion_channels(input_val, t)
        
        # Process STDP
        stdp_effect = self.process_stdp(input_val, current, t)
        
        # Combine new ion states
        new_state = self.calcium_state ^ self.sodium_state ^ self.potassium_state
        
        # Apply STDP effect
        final_state = new_state ^ stdp_effect
        
        self.history.append((final_state, t))
        return final_state

class WaveNetwork:
    """Network of wave memories with traveling wave propagation"""
    
    def __init__(self, num_neurons: int, config: WaveConfig):
        self.config = config
        self.neurons = [WaveMemory(config) for _ in range(num_neurons)]
        
    def process(self, input_pattern: List[int], t: float) -> List[int]:
        """Process input through the network"""
        outputs = []
        prev_out = None
        
        for i, neuron in enumerate(self.neurons):
            if i < len(input_pattern):
                # Combine input with previous output through interference
                if prev_out is None:
                    neuron_in = input_pattern[i]
                else:
                    neuron_in = input_pattern[i] ^ prev_out
                    
                out = neuron.update(neuron_in, t)
                outputs.append(out)
                prev_out = out
                
        return outputs

def run_simulation(config: WaveConfig, num_steps: int = 1000):
    """Run example simulation"""
    network = WaveNetwork(num_neurons=3, config=config)
    
    states = []
    t = np.linspace(0, 1, num_steps)
    
    for ti in t:
        # Generate example input pattern
        input_pattern = [1 if np.random.random() < 0.3 else 0 for _ in range(3)]
        outputs = network.process(input_pattern, ti)
        states.append(outputs)
        
    return np.array(states)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple

class WaveVisualizer:
    """Visualizes digital duty cycle waves and their interference patterns"""
    
    def __init__(self, time_steps: int = 100):
        self.time_steps = time_steps
        
    def plot_duty_cycle_wave(self, freq: float, duty: float, 
                            time_range: np.ndarray, ax: plt.Axes, 
                            label: str = None, color: str = 'b'):
        """Plot single duty cycle wave"""
        wave = np.zeros(len(time_range))
        for i, t in enumerate(time_range):
            phase = (t * freq) % 1.0
            wave[i] = 1 if phase < duty else 0
            
        ax.step(time_range, wave, where='post', label=label, color=color)
        ax.set_ylim(-0.2, 1.2)
        ax.grid(True)
        if label:
            ax.legend()

    def plot_interference(self, waves: List[np.ndarray], time_range: np.ndarray, 
                         ax: plt.Axes, mode: str = 'xor'):
        """Plot interference between multiple waves"""
        result = waves[0].copy()
        for wave in waves[1:]:
            if mode == 'xor':
                result = np.logical_xor(result, wave)
            else:  # AND mode
                result = np.logical_and(result, wave)
                
        ax.step(time_range, result, 'r', where='post', 
                label=f'{mode.upper()} Interference')
        ax.set_ylim(-0.2, 1.2)
        ax.grid(True)
        ax.legend()

    def plot_state_evolution(self, states: Dict[str, List[int]], 
                           time_range: np.ndarray, ax: plt.Axes):
        """Plot evolution of multiple states over time"""
        for name, state in states.items():
            ax.step(time_range[:len(state)], state, where='post', 
                   label=name, alpha=0.7)
        ax.set_ylim(-0.2, 1.2)
        ax.grid(True)
        ax.legend()

    def create_wave_subplot(self, waves: Dict[str, Tuple[float, float]], 
                          time_range: np.ndarray, title: str) -> plt.Figure:
        """Create subplot showing multiple waves and their interference"""
        fig, axes = plt.subplots(len(waves) + 1, 1, figsize=(12, 2*len(waves)), 
                                sharex=True)
        wave_arrays = []
        
        # Plot individual waves
        for i, (name, (freq, duty)) in enumerate(waves.items()):
            wave = np.zeros(len(time_range))
            for j, t in enumerate(time_range):
                phase = (t * freq) % 1.0
                wave[j] = 1 if phase < duty else 0
            wave_arrays.append(wave)
            
            self.plot_duty_cycle_wave(freq, duty, time_range, axes[i], 
                                    label=name)
            axes[i].set_title(f'{name} (f={freq:.1f}, duty={duty:.2f})')
            
        # Plot interference
        self.plot_interference(wave_arrays, time_range, axes[-1])
        axes[-1].set_title('Wave Interference Pattern')
        
        plt.suptitle(title)
        plt.tight_layout()
        return fig

    def visualize_memory_system(self, wave_memory: 'EnhancedWaveMemory',  # type: ignore
                              duration: float = 1.0):
        """Create comprehensive visualization of wave memory system"""
        time_range = np.linspace(0, duration, self.time_steps)
        
        # Create figure with subplots
        fig = plt.figure(figsize=(15, 10))
        gs = fig.add_gridspec(3, 2)
        
        # 1. Ion Channel Waves
        ax1 = fig.add_subplot(gs[0, 0])
        waves = {
            'Calcium': (wave_memory.config.calcium_freq, wave_memory.config.ion_duty),
            'Sodium': (wave_memory.config.sodium_freq, wave_memory.config.ion_duty),
            'Potassium': (wave_memory.config.potassium_freq, wave_memory.config.ion_duty)
        }
        for name, (freq, duty) in waves.items():
            self.plot_duty_cycle_wave(freq, duty, time_range, ax1, name)
        ax1.set_title('Ion Channel Carrier Waves')
        
        # 2. Timing Waves
        ax2 = fig.add_subplot(gs[0, 1])
        waves = {
            'Theta': (wave_memory.config.theta_freq, wave_memory.config.theta_duty),
            'Gamma': (wave_memory.config.gamma_freq, wave_memory.config.gamma_duty)
        }
        for name, (freq, duty) in waves.items():
            self.plot_duty_cycle_wave(freq, duty, time_range, ax2, name)
        ax2.set_title('Timing Waves')
        
        # 3. State Evolution
        ax3 = fig.add_subplot(gs[1, :])
        history = np.array(wave_memory.history)
        if len(history) > 0:
            states = {
                'Calcium': [state & 1 for state, _ in history],
                'Sodium': [state & 2 for state, _ in history],
                'Potassium': [state & 4 for state, _ in history]
            }
            self.plot_state_evolution(states, time_range, ax3)
        ax3.set_title('Ion Channel States')
        
        # 4. Combined Output
        ax4 = fig.add_subplot(gs[2, :])
        if len(history) > 0:
            final_states = [state for state, _ in history]
            ax4.step(time_range[:len(final_states)], final_states, 'k', 
                    where='post', label='Combined Output')
            ax4.set_title('Final Output')
            ax4.grid(True)
            ax4.legend()
        
        plt.tight_layout()
        return fig

def plot_example():
    """Show example visualization"""
    time_range = np.linspace(0, 1, 1000)
    waves = {
        'Wave A': (10.0, 0.3),
        'Wave B': (15.0, 0.4),
        'Wave C': (20.0, 0.5)
    }
    
    viz = WaveVisualizer()
    fig = viz.create_wave_subplot(waves, time_range, 
                                'Digital Duty Cycle Wave Interference')
    plt.show()

if __name__ == "__main__":
    plot_example()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple, Union

@dataclass
class WaveConfig:
    """Complete configuration for all wave frequencies and duty cycles"""
    # Sample rate matching common audio standard
    sample_rate: int = 44100
    
    # Neural rhythms
    theta_freq: float = 5.0      # 4-8 Hz
    gamma_freq: float = 40.0     # 30-90 Hz
    beta_freq: float = 20.0      # 13-30 Hz
    alpha_freq: float = 10.0     # 8-13 Hz
    
    # Ion channels
    calcium_freq: float = 12000.0
    sodium_freq: float = 15000.0
    potassium_freq: float = 17000.0
    
    # Duty cycles
    theta_duty: float = 0.5
    gamma_duty: float = 0.3
    beta_duty: float = 0.4
    alpha_duty: float = 0.45
    ion_duty: float = 0.3
    
    # STDP parameters
    stdp_window: int = 10
    stdp_duty: float = 0.2

class WaveMemory:
    """Complete wave memory implementation with all frequencies"""
    
    def __init__(self, config: WaveConfig):
        self.config = config
        
        # Ion channel states
        self.calcium_state = 0
        self.sodium_state = 0
        self.potassium_state = 0
        
        # Neural rhythm states
        self.theta_state = 0
        self.gamma_state = 0
        self.beta_state = 0
        self.alpha_state = 0
        
        # STDP windows
        self.pre_history = [0] * config.stdp_window
        self.post_history = [0] * config.stdp_window
        
        # History tracking for visualization
        self.history = {
            'ion_states': [],
            'rhythm_states': [],
            'stdp_states': [],
            'combined_output': []
        }

    def generate_wave(self, t: float, freq: float, duty: float) -> int:
        """Generate digital duty cycle wave"""
        phase = (t * freq) % 1.0
        return 1 if phase < duty else 0

    def process_ion_channels(self, input_val: int, t: float) -> int:
        """Process input through all ion channels"""
        # Generate carrier waves
        ca_wave = self.generate_wave(t, self.config.calcium_freq, self.config.ion_duty)
        na_wave = self.generate_wave(t, self.config.sodium_freq, self.config.ion_duty)
        k_wave = self.generate_wave(t, self.config.potassium_freq, self.config.ion_duty)
        
        # Update states through interference
        self.calcium_state ^= (ca_wave & input_val)
        self.sodium_state ^= (na_wave & input_val)
        self.potassium_state ^= (k_wave & input_val)
        
        # Store history
        self.history['ion_states'].append({
            'calcium': self.calcium_state,
            'sodium': self.sodium_state,
            'potassium': self.potassium_state,
            't': t
        })
        
        return self.calcium_state ^ self.sodium_state ^ self.potassium_state

    def process_neural_rhythms(self, input_val: int, t: float) -> int:
        """Process through neural rhythm waves"""
        # Generate rhythm waves
        theta = self.generate_wave(t, self.config.theta_freq, self.config.theta_duty)
        gamma = self.generate_wave(t, self.config.gamma_freq, self.config.gamma_duty)
        beta = self.generate_wave(t, self.config.beta_freq, self.config.beta_duty)
        alpha = self.generate_wave(t, self.config.alpha_freq, self.config.alpha_duty)
        
        # Update states
        self.theta_state ^= (theta & input_val)
        self.gamma_state ^= (gamma & input_val)
        self.beta_state ^= (beta & input_val)
        self.alpha_state ^= (alpha & input_val)
        
        # Store history
        self.history['rhythm_states'].append({
            'theta': self.theta_state,
            'gamma': self.gamma_state,
            'beta': self.beta_state,
            'alpha': self.alpha_state,
            't': t
        })
        
        return self.theta_state ^ self.gamma_state ^ self.beta_state ^ self.alpha_state

    def process_stdp(self, pre_spike: int, post_spike: int, t: float) -> int:
        """Full STDP processing"""
        # Update timing windows
        self.pre_history = [pre_spike] + self.pre_history[:-1]
        self.post_history = [post_spike] + self.post_history[:-1]
        
        stdp_effect = 0
        
        # Pre-before-post: strengthen through constructive interference
        for i in range(self.config.stdp_window - 1):
            if self.pre_history[i] and self.post_history[i+1]:
                stdp_effect ^= 1
                
        # Post-before-pre: weaken through destructive interference
        for i in range(self.config.stdp_window - 1):
            if self.post_history[i] and self.pre_history[i+1]:
                stdp_effect &= 0
                
        self.history['stdp_states'].append({
            'pre_spike': pre_spike,
            'post_spike': post_spike,
            'effect': stdp_effect,
            't': t
        })
        
        return stdp_effect

    def update(self, input_val: int, t: float) -> int:
        """Complete update incorporating all mechanisms"""
        # Process through different wave systems
        ion_out = self.process_ion_channels(input_val, t)
        rhythm_out = self.process_neural_rhythms(input_val, t)
        
        # Current state for STDP
        current = ion_out ^ rhythm_out
        
        # Process STDP
        stdp_out = self.process_stdp(input_val, current, t)
        
        # Combine all outputs
        final_state = ion_out ^ rhythm_out ^ stdp_out
        
        self.history['combined_output'].append({
            'state': final_state,
            't': t
        })
        
        return final_state

class WaveVisualizer:
    """Visualization for wave memory system using sinusoidal representations"""
    
    def __init__(self, wave_memory: WaveMemory):
        self.wave_memory = wave_memory
        
    def digital_to_analog(self, times: List[float], states: List[int], 
                         freq: float, phase: float = 0.0) -> np.ndarray:
        """Convert digital states to sinusoidal representation"""
        # Generate base sine wave
        sine_wave = np.sin(2 * np.pi * freq * np.array(times) + phase)
        
        # Modulate amplitude with digital state
        modulated = sine_wave * np.array(states)
        return modulated
        
    def plot_full_state(self, duration: float = 1.0) -> plt.Figure:
        """Create comprehensive visualization with sinusoidal representations"""
        fig = plt.figure(figsize=(15, 12))
        gs = fig.add_gridspec(4, 1, height_ratios=[3, 3, 2, 2])
        
        # 1. Ion Channel States
        ax1 = fig.add_subplot(gs[0])
        self.plot_ion_states(ax1)
        
        # 2. Neural Rhythms
        ax2 = fig.add_subplot(gs[1])
        self.plot_rhythm_states(ax2)
        
        # 3. STDP Effects
        ax3 = fig.add_subplot(gs[2])
        self.plot_stdp_states(ax3)
        
        # 4. Combined Output with Phase Analysis
        ax4 = fig.add_subplot(gs[3])
        self.plot_combined_output(ax4)
        
        plt.tight_layout()
        return fig
    
    def plot_ion_states(self, ax: plt.Axes):
        """Plot ion channel states as modulated sine waves"""
        if not self.wave_memory.history['ion_states']:
            return
            
        data = self.wave_memory.history['ion_states']
        times = [d['t'] for d in data]
        
        channels = {
            'calcium': (self.wave_memory.config.calcium_freq, 0),
            'sodium': (self.wave_memory.config.sodium_freq, np.pi/3),
            'potassium': (self.wave_memory.config.potassium_freq, 2*np.pi/3)
        }
        
        for channel, (freq, phase) in channels.items():
            values = [d[channel] for d in data]
            analog = self.digital_to_analog(times, values, freq, phase)
            ax.plot(times, analog, label=f'{channel.title()} Channel', alpha=0.7)
            
        ax.set_title('Ion Channel Activity (Sinusoidal Representation)')
        ax.grid(True)
        ax.legend()
        ax.set_ylim(-1.2, 1.2)
        
    def plot_rhythm_states(self, ax: plt.Axes):
        """Plot neural rhythms as modulated sine waves"""
        if not self.wave_memory.history['rhythm_states']:
            return
            
        data = self.wave_memory.history['rhythm_states']
        times = [d['t'] for d in data]
        
        rhythms = {
            'theta': (self.wave_memory.config.theta_freq, 0),
            'gamma': (self.wave_memory.config.gamma_freq, np.pi/4),
            'beta': (self.wave_memory.config.beta_freq, np.pi/3),
            'alpha': (self.wave_memory.config.alpha_freq, np.pi/2)
        }
        
        for rhythm, (freq, phase) in rhythms.items():
            values = [d[rhythm] for d in data]
            analog = self.digital_to_analog(times, values, freq, phase)
            ax.plot(times, analog, label=f'{rhythm.title()} Rhythm', alpha=0.7)
            
        ax.set_title('Neural Rhythm Activity (Sinusoidal Representation)')
        ax.grid(True)
        ax.legend()
        ax.set_ylim(-1.2, 1.2)
        
    def plot_stdp_states(self, ax: plt.Axes):
        """Plot STDP with phase relationships"""
        if not self.wave_memory.history['stdp_states']:
            return
            
        data = self.wave_memory.history['stdp_states']
        times = [d['t'] for d in data]
        
        # Show pre/post spikes as modulated waves
        pre_spikes = [d['pre_spike'] for d in data]
        post_spikes = [d['post_spike'] for d in data]
        effects = [d['effect'] for d in data]
        
        # Use different frequencies to show timing relationships
        pre_analog = self.digital_to_analog(times, pre_spikes, 20.0, 0)
        post_analog = self.digital_to_analog(times, post_spikes, 20.0, np.pi/2)
        effect_analog = self.digital_to_analog(times, effects, 10.0, 0)
        
        ax.plot(times, pre_analog, label='Pre-spike', alpha=0.5)
        ax.plot(times, post_analog, label='Post-spike', alpha=0.5)
        ax.plot(times, effect_analog, label='STDP Effect', color='r', alpha=0.7)
        
        ax.set_title('STDP Activity (Phase Relationships)')
        ax.grid(True)
        ax.legend()
        ax.set_ylim(-1.2, 1.2)
        
    def plot_combined_output(self, ax: plt.Axes):
        """Plot combined output with phase analysis"""
        if not self.wave_memory.history['combined_output']:
            return
            
        data = self.wave_memory.history['combined_output']
        times = [d['t'] for d in data]
        states = [d['state'] for d in data]
        
        # Show combined output as modulated wave
        analog = self.digital_to_analog(times, states, 15.0, 0)
        ax.plot(times, analog, label='Combined State', color='k', alpha=0.7)
        
        ax.set_title('Combined System Output (Phase Analysis)')
        ax.grid(True)
        ax.legend()
        ax.set_ylim(-1.2, 1.2)

    def plot_phase_relationships(self) -> plt.Figure:
        """Additional plot focusing on phase relationships between waves"""
        fig = plt.figure(figsize=(15, 8))
        ax = plt.gca()
        
        data = self.wave_memory.history['combined_output']
        times = [d['t'] for d in data]
        
        # Combine all waves with their natural frequencies
        waves = []
        for rhythm, freq in [
            ('theta', self.wave_memory.config.theta_freq),
            ('gamma', self.wave_memory.config.gamma_freq),
            ('calcium', self.wave_memory.config.calcium_freq)
        ]:
            if rhythm in ['theta', 'gamma']:
                states = [d[rhythm] for d in self.wave_memory.history['rhythm_states']]
            else:
                states = [d[rhythm.lower()] for d in self.wave_memory.history['ion_states']]
            
            analog = self.digital_to_analog(times, states, freq)
            waves.append(analog)
            ax.plot(times, analog + len(waves)*2, label=f'{rhythm} Wave', alpha=0.7)
        
        # Show combined interference
        combined = np.sum(waves, axis=0) / len(waves)
        ax.plot(times, combined, label='Wave Interference', color='r', linewidth=2)
        
        ax.set_title('Phase Relationships and Wave Interference')
        ax.grid(True)
        ax.legend()
        
        return fig

def run_example():
    """Run example simulation with visualization"""
    config = WaveConfig()
    memory = WaveMemory(config)
    viz = WaveVisualizer(memory)
    
    # Run simulation
    duration = 0.1  # seconds
    dt = 1.0 / config.sample_rate
    steps = int(duration * config.sample_rate)
    
    for step in range(steps):
        t = step * dt
        input_val = 1 if np.random.random() < 0.3 else 0
        memory.update(input_val, t)
    
    # Visualize results
    viz.plot_full_state()
    plt.show()

if __name__ == "__main__":
    run_example()

In [ ]:
%pip uninstall bizarromath -y

In [ ]:
%pip install mpmath mpmathify

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Dict
from mpmath import mp, mpf

# Set decimal precision (e.g., 50 decimal places)
mp.dps = 50

def to_float(m) -> float:
    """Convert mpmath number to Python float."""
    return float(m)

def to_int(m) -> int:
    """Convert mpmath number to Python int."""
    return int(m)

@dataclass
class WaveConfig:
    """Complete configuration for all wave frequencies and duty cycles"""
    # Sample rate matching common audio standard (now BizarroWorld)
    sample_rate = mpf("44100")
    
    # Neural rhythms (now BizarroWorld)
    theta_freq = mpf("5.0")
    gamma_freq = mpf("40.0")
    beta_freq = mpf("20.0")
    alpha_freq = mpf("10.0")
    
    # Ion channels (now BizarroWorld)
    calcium_freq = mpf("12000.0")
    sodium_freq = mpf("15000.0")
    potassium_freq = mpf("17000.0")
    
    # Duty cycles (remain as float)
    theta_duty = mpf("0.5")
    gamma_duty = mpf("0.3")
    beta_duty = mpf("0.4")
    alpha_duty = mpf("0.45")
    ion_duty = mpf("0.3")
    
    # STDP parameters (remain as int)
    stdp_window = mpf("10")
    stdp_duty = mpf("0.2")
    
class WaveMemory:
    """Complete wave memory implementation with all frequencies"""
    def __init__(self, config: WaveConfig):
            self.config = config
            
            # Ion channel states
            self.calcium_state = mpf("0")
            self.sodium_state = mpf("0")
            self.potassium_state = mpf("0")
            
            # Neural rhythm states
            self.theta_state = mpf("0")
            self.gamma_state = mpf("0")
            self.beta_state = mpf("0")
            self.alpha_state = mpf("0")
            
            # STDP windows
            self.pre_history = [mpf("0")] * to_int(config.stdp_window)
            self.post_history = [mpf("0")] * to_int(config.stdp_window)
            
            # History tracking for visualization
            self.history = {
                'ion_states': [],
                'rhythm_states': [],
                'stdp_states': [],
                'combined_output': []
            }

    def generate_wave(self, t, freq, duty) -> int:
        """Generate digital duty cycle wave using mpmath"""
        one = mpf("1")
        phase = (t * freq) % one
        return 1 if phase < duty else 0

    def process_ion_channels(self, input_val: int, t) -> int:
        """Process input through all ion channels using mpmath"""
        ca_wave = self.generate_wave(t, self.config.calcium_freq, self.config.ion_duty)
        na_wave = self.generate_wave(t, self.config.sodium_freq, self.config.ion_duty)
        k_wave = self.generate_wave(t, self.config.potassium_freq, self.config.ion_duty)
        
        # Perform XOR operations and update states
        self.calcium_state = mpf(str(int(to_int(self.calcium_state) ^ (ca_wave & input_val))))
        self.sodium_state = mpf(str(int(to_int(self.sodium_state) ^ (na_wave & input_val))))
        self.potassium_state = mpf(str(int(to_int(self.potassium_state) ^ (k_wave & input_val))))
        
        # Record history
        self.history['ion_states'].append({
            'calcium': self.calcium_state,
            'sodium': self.sodium_state,
            'potassium': self.potassium_state,
            't': t
        })
        
        return to_int(self.calcium_state) ^ to_int(self.sodium_state) ^ to_int(self.potassium_state)

    def process_neural_rhythms(self, input_val: int, t) -> int:
        """Process through neural rhythm waves using mpmath"""
        theta = self.generate_wave(t, self.config.theta_freq, self.config.theta_duty)
        gamma = self.generate_wave(t, self.config.gamma_freq, self.config.gamma_duty)
        beta = self.generate_wave(t, self.config.beta_freq, self.config.beta_duty)
        alpha = self.generate_wave(t, self.config.alpha_freq, self.config.alpha_duty)
        
        # Perform XOR operations and update states
        self.theta_state = mpf(str(int(to_int(self.theta_state) ^ (theta & input_val))))
        self.gamma_state = mpf(str(int(to_int(self.gamma_state) ^ (gamma & input_val))))
        self.beta_state = mpf(str(int(to_int(self.beta_state) ^ (beta & input_val))))
        self.alpha_state = mpf(str(int(to_int(self.alpha_state) ^ (alpha & input_val))))
        
        # Record history
        self.history['rhythm_states'].append({
            'theta': self.theta_state,
            'gamma': self.gamma_state,
            'beta': self.beta_state,
            'alpha': self.alpha_state,
            't': t
        })
        
        return (to_int(self.theta_state) ^ to_int(self.gamma_state) ^ 
                to_int(self.beta_state) ^ to_int(self.alpha_state))

    def process_stdp(self, pre_spike: int, post_spike: int, t) -> int:
        """Full STDP processing using mpmath"""
        # Update histories
        self.pre_history = [mpf(str(pre_spike))] + self.pre_history[:-1]
        self.post_history = [mpf(str(post_spike))] + self.post_history[:-1]
        
        stdp_effect = 0
        
        # STDP Window Processing
        window_size = to_int(self.config.stdp_window) - 1
        for i in range(window_size):
            if to_int(self.pre_history[i]) and to_int(self.post_history[i+1]):
                stdp_effect ^= 1
                
        for i in range(window_size):
            if to_int(self.post_history[i]) and to_int(self.pre_history[i+1]):
                stdp_effect &= 0
                
        # Record history
        self.history['stdp_states'].append({
            'pre_spike': pre_spike,
            'post_spike': post_spike,
            'effect': stdp_effect,
            't': t
        })
        
        return stdp_effect

    def update(self, input_val: int, t) -> int:
        """Complete update incorporating all mechanisms using mpmath"""
        ion_out = self.process_ion_channels(input_val, t)
        rhythm_out = self.process_neural_rhythms(input_val, t)
        
        current = ion_out ^ rhythm_out
        
        stdp_out = self.process_stdp(input_val, current, t)
        
        final_state = ion_out ^ rhythm_out ^ stdp_out
        
        self.history['combined_output'].append({
            'state': final_state,
            't': t
        })
        
        return final_state
    
    def run_example():
        """Run example simulation with visualization"""
        config = WaveConfig()
        memory = WaveMemory(config)
        viz = WaveVisualizer(memory)
        
        duration = mpf("0.1")  # seconds, now BizarroWorld
        dt = mpf("1") / config.sample_rate  # High-precision time step
        steps = to_int(config.sample_rate * duration)  # Total steps
        
        for step in range(steps):
            t = mpf(step) * dt  # Use mpmath arithmetic
            input_val = 1 if np.random.random() < 0.3 else 0
            memory.update(input_val, t)
        
        viz.plot_full_state()
        viz.plot_phase_relationships()
        
def run_example():
    """Run example simulation with visualization"""
    config = WaveConfig()
    memory = WaveMemory(config)
    viz = WaveVisualizer(memory)
    
    duration = mpf("0.1")  # seconds, now BizarroWorld
    dt = mpf("1") / config.sample_rate  # High-precision time step
    steps = to_int(config.sample_rate * duration)  # Total steps
    
    for step in range(steps):
        t = mpf(step) * dt  # Use mpmath arithmetic
        input_val = 1 if np.random.random() < 0.3 else 0
        memory.update(input_val, t)
    
    viz.plot_full_state()
    viz.plot_phase_relationships()
if __name__ == "__main__":
    run_example()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List

@dataclass
class WaveConfig:
    """ Configuration for wave frequencies and duty cycles at 44kHz sample rate. """
    sample_rate: int = 44100
    
    # Neural rhythms
    theta_freq: float = 5.0      # 4-8 Hz
    gamma_freq: float = 40.0     # 30-90 Hz
    beta_freq: float = 20.0      # 13-30 Hz
    alpha_freq: float = 10.0     # 8-13 Hz
    
    # Ion channels
    calcium_freq: float = 12000.0
    sodium_freq: float = 15000.0
    potassium_freq: float = 17000.0
    
    # Duty cycles
    theta_duty: float = 0.5
    gamma_duty: float = 0.3
    beta_duty: float = 0.4
    alpha_duty: float = 0.45
    ion_duty: float = 0.3
    
    # STDP parameters
    stdp_window: int = 10
    stdp_duty: float = 0.2

class WaveMemory:
    """ Wave-based memory system, combining ion channels, neural rhythms, and STDP in a duty-cycle format. """
    
    def __init__(self, config: WaveConfig):
        self.config = config
        
        # Ion channel states
        self.calcium_state = 0
        self.sodium_state = 0
        self.potassium_state = 0
        
        # Neural rhythm states
        self.theta_state = 0
        self.gamma_state = 0
        self.beta_state = 0
        self.alpha_state = 0
        
        # Simple STDP windows
        self.pre_history = [0] * config.stdp_window
        self.post_history = [0] * config.stdp_window
        
        # Record history for plotting
        self.history = {
            'ion_states': [],
            'rhythm_states': [],
            'stdp_states': [],
            'combined_output': []
        }

    def generate_wave(self, t: float, freq: float, duty: float) -> int:
        """Create a digital duty-cycle wave at time t (0 or 1)."""
        phase = (t * freq) % 1.0
        return 1 if phase < duty else 0

    def process_ion_channels(self, input_val: int, t: float) -> int:
        """Ion channels: calcium, sodium, potassium wave interference."""
        ca_wave = self.generate_wave(t, self.config.calcium_freq, self.config.ion_duty)
        na_wave = self.generate_wave(t, self.config.sodium_freq, self.config.ion_duty)
        k_wave  = self.generate_wave(t, self.config.potassium_freq, self.config.ion_duty)
        
        # XOR-based wave updates
        self.calcium_state   ^= (ca_wave & input_val)
        self.sodium_state    ^= (na_wave & input_val)
        self.potassium_state ^= (k_wave  & input_val)
        
        self.history['ion_states'].append({
            'calcium': self.calcium_state,
            'sodium':  self.sodium_state,
            'potassium': self.potassium_state,
            't': t
        })
        
        return self.calcium_state ^ self.sodium_state ^ self.potassium_state

    def process_neural_rhythms(self, input_val: int, t: float) -> int:
        """Rhythm waves: theta, gamma, beta, alpha."""
        theta = self.generate_wave(t, self.config.theta_freq, self.config.theta_duty)
        gamma = self.generate_wave(t, self.config.gamma_freq, self.config.gamma_duty)
        beta  = self.generate_wave(t, self.config.beta_freq,  self.config.beta_duty)
        alpha = self.generate_wave(t, self.config.alpha_freq, self.config.alpha_duty)
        
        # XOR again for demonstration
        self.theta_state ^= (theta & input_val)
        self.gamma_state ^= (gamma & input_val)
        self.beta_state  ^= (beta  & input_val)
        self.alpha_state ^= (alpha & input_val)
        
        self.history['rhythm_states'].append({
            'theta': self.theta_state,
            'gamma': self.gamma_state,
            'beta':  self.beta_state,
            'alpha': self.alpha_state,
            't': t
        })
        
        return self.theta_state ^ self.gamma_state ^ self.beta_state ^ self.alpha_state

    def process_stdp(self, pre_spike: int, post_spike: int, t: float) -> int:
        """Simple STDP: pre-before-post => constructive, post-before-pre => destructive."""
        self.pre_history  = [pre_spike]  + self.pre_history[:-1]
        self.post_history = [post_spike] + self.post_history[:-1]
        
        stdp_effect = 0
        # Constructive
        for i in range(self.config.stdp_window - 1):
            if self.pre_history[i] and self.post_history[i+1]:
                stdp_effect ^= 1
        # Destructive
        for i in range(self.config.stdp_window - 1):
            if self.post_history[i] and self.pre_history[i+1]:
                stdp_effect &= 0
        
        self.history['stdp_states'].append({
            'pre_spike':  pre_spike,
            'post_spike': post_spike,
            'effect':     stdp_effect,
            't': t
        })
        return stdp_effect

    def update(self, input_val: int, t: float) -> int:
        """
        Full wave-based update:
         1) Ion channels
         2) Rhythms
         3) STDP
         4) Combine all into final wave output
        """
        ion_out    = self.process_ion_channels(input_val, t)
        rhythm_out = self.process_neural_rhythms(input_val, t)
        
        # Combine so STDP sees "pre" (the input) and "post" (the net's current wave)
        current = ion_out ^ rhythm_out
        stdp_out = self.process_stdp(input_val, current, t)
        
        final_state = ion_out ^ rhythm_out ^ stdp_out
        
        self.history['combined_output'].append({
            'state': final_state,
            't': t
        })
        return final_state

class WaveVisualizer:
    """Convert the recorded integer states into sinusoidal plots for each subsystem."""
    def __init__(self, wave_memory: WaveMemory):
        self.m = wave_memory

    def digital_to_analog(self, t_arr, s_arr, freq, phase=0.0):
        """
        Convert duty cycle digital signal to proper sinusoidal waveform.
        t_arr: time points
        s_arr: digital signal (0s and 1s)
        freq: carrier frequency
        phase: initial phase offset
        """
        # Create high frequency carrier
        t = np.array(t_arr)
        carrier = np.sin(2 * np.pi * freq * t + phase)
        
        # Convert digital signal to analog envelope
        window = int(self.m.config.sample_rate / freq / 10)  # Smoothing window
        if window > 1:
            envelope = np.convolve(s_arr, np.ones(window)/window, mode='same')
        else:
            envelope = s_arr
        
        # Modulate carrier with envelope
        return carrier * envelope

    def plot_all(self):
        fig = plt.figure(figsize=(12,10))
        gs = fig.add_gridspec(4,1)
        
        # Ion
        ax1 = fig.add_subplot(gs[0])
        self.plot_ion_states(ax1)
        
        # Rhythms
        ax2 = fig.add_subplot(gs[1])
        self.plot_rhythm_states(ax2)
        
        # STDP
        ax3 = fig.add_subplot(gs[2])
        self.plot_stdp_states(ax3)
        
        # Combined
        ax4 = fig.add_subplot(gs[3])
        self.plot_combined(ax4)
        
        plt.tight_layout()
        plt.show()

    def plot_ion_states(self, ax):
        data = self.m.history['ion_states']
        if not data:
            return
        t = [d['t'] for d in data]

        channels = {
            'calcium':    (self.m.config.calcium_freq,    0),
            'sodium':     (self.m.config.sodium_freq,     np.pi/4),
            'potassium':  (self.m.config.potassium_freq,  np.pi/2)
        }
        for name, (freq, phase) in channels.items():
            s = [d[name] for d in data]
            wave = self.digital_to_analog(t, s, freq, phase)
            ax.plot(t, wave, label=name)
        ax.set_ylim(-1.2, 1.2)
        ax.legend()
        ax.set_title("Ion Channels (Duty-Cycle → Sine)")

    def plot_rhythm_states(self, ax):
        data = self.m.history['rhythm_states']
        if not data:
            return
        t = [d['t'] for d in data]

        rhythms = {
            'theta': (self.m.config.theta_freq, 0),
            'gamma': (self.m.config.gamma_freq, np.pi/4),
            'beta':  (self.m.config.beta_freq,  np.pi/3),
            'alpha': (self.m.config.alpha_freq, np.pi/2)
        }
        for name, (freq, phase) in rhythms.items():
            s = [d[name] for d in data]
            wave = self.digital_to_analog(t, s, freq, phase)
            ax.plot(t, wave, label=name)
        ax.set_ylim(-1.2, 1.2)
        ax.legend()
        ax.set_title("Neural Rhythms (Duty-Cycle → Sine)")

    def plot_stdp_states(self, ax):
        data = self.m.history['stdp_states']
        if not data:
            return
        t = [d['t'] for d in data]

        pre_s  = [d['pre_spike']  for d in data]
        post_s = [d['post_spike'] for d in data]
        eff_s  = [d['effect']     for d in data]

        pre_w  = self.digital_to_analog(t, pre_s,  15.0, 0)
        post_w = self.digital_to_analog(t, post_s, 15.0, np.pi/2)
        eff_w  = self.digital_to_analog(t, eff_s,  10.0, 0)

        ax.plot(t, pre_w,  label="Pre-spike",  alpha=0.6)
        ax.plot(t, post_w, label="Post-spike", alpha=0.6)
        ax.plot(t, eff_w,  label="STDP effect", color='r', alpha=0.7)
        ax.set_ylim(-1.2,1.2)
        ax.legend()
        ax.set_title("STDP Phase Relation")

    def plot_combined(self, ax):
        data = self.m.history['combined_output']
        if not data:
            return
        t = [d['t'] for d in data]
        s = [d['state'] for d in data]

        wave = self.digital_to_analog(t, s, 12.0, 0)
        ax.plot(t, wave, 'k-', label="Combined Output")
        ax.set_title("Overall Output (Duty-Cycle → Sine)")
        ax.set_ylim(-1.2,1.2)
        ax.legend()

def run_chained_demo():
    config = WaveConfig()
    memory = WaveMemory(config)
    viz    = WaveVisualizer(memory)
    
    duration = 0.02  # 20 ms
    dt = 1.0 / config.sample_rate
    steps = int(duration * config.sample_rate)
    
    for step in range(steps):
        t = step * dt
        # Random input 30% chance = 1
        input_val = 1 if np.random.random() < 0.3 else 0
        memory.update(input_val, t)

    fig = viz.plot_all()
    plt.show()

if __name__ == "__main__":
    run_chained_demo()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class WaveLTCConfig:
    """
    Configuration for wave-based LTC approximation with a carrier frequency
    plus a scale factor to ensure toggles_total isn't always zero.
    """
    def __init__(self,
                 alpha=2.0,          # LTC gain
                 sample_rate=1000,   # macro steps per second (the LTC integration rate)
                 duration=0.05,      # total simulation time (seconds)
                 max_state=100,      # max integer for h(t)
                 init_state=50,      # starting integer for h(t)
                 
                 # Sinusoidal input
                 input_freq=20.0,
                 input_amp=1.0,
                 input_phase=0.0,

                 # Carrier approach
                 carrier_freq=20000,  

                 # Additional scale factor for toggles
                 wave_ltc_scale=200.0  
                 ):
        
        self.alpha         = alpha
        self.sample_rate   = sample_rate
        self.duration      = duration
        self.max_state     = max_state
        self.init_state    = init_state
        self.input_freq    = input_freq
        self.input_amp     = input_amp
        self.input_phase   = input_phase
        self.carrier_freq  = carrier_freq
        self.wave_ltc_scale= wave_ltc_scale

class WaveLTCNeuron:
    """
    Approximate LTC neuron using a high-frequency carrier + scale factor
    so toggles_total won't be zero as easily.
    """
    def __init__(self, config: WaveLTCConfig):
        self.config = config
        # integer state for h(t)
        self.h_state = config.init_state
        
        # For reference, a float-based LTC solution:
        self.h_float = float(config.init_state) / config.max_state

        self.history_wave = []
        self.history_true = []
        self.times = []
        
    def run_simulation(self):
        """
        Each macro-step, we run 'carrier_cycles' sub-steps,
        distributing toggles according to the difference x_int - h_state.
        """
        dt_macro = 1.0 / self.config.sample_rate
        steps = int(self.config.sample_rate * self.config.duration)
        
        # how many fine toggles do we do per macro-step
        carrier_cycles = int(round(self.config.carrier_freq / self.config.sample_rate))
        dt_sub = dt_macro / carrier_cycles
        
        for step_i in range(steps):
            t = step_i * dt_macro
            
            # 1) Input wave in [0..max_state]
            x_t = self.config.input_amp * np.sin(
                2*np.pi*self.config.input_freq*t + self.config.input_phase
            )
            x_scaled = 0.5*(x_t + 1.0)  # now in [0,1]
            x_int    = int(round(x_scaled * self.config.max_state))
            
            # 2) Wave LTC step
            diff = x_int - self.h_state
            magnitude = abs(diff)
            sign = np.sign(diff)
            
            # We'll compute toggles_total, but multiply by wave_ltc_scale
            # so that even moderate differences produce toggles.
            toggles_total = self.config.alpha * magnitude * dt_macro
            toggles_total *= self.config.wave_ltc_scale
            toggles_total = int(round(toggles_total))
            
            # saturate toggles at carrier_cycles
            if toggles_total > carrier_cycles:
                toggles_total = carrier_cycles
            
            # Distribute toggles_total across sub-steps
            for sub_i in range(carrier_cycles):
                if sub_i < toggles_total:
                    if sign > 0:
                        self.h_state = min(self.h_state + 1, self.config.max_state)
                    elif sign < 0:
                        self.h_state = max(self.h_state - 1, 0)
            
            self.history_wave.append(self.h_state)
            
            # 3) "True" LTC step
            dh = self.config.alpha*(x_scaled - self.h_float)
            self.h_float += dh*dt_macro
            self.h_float = np.clip(self.h_float, 0.0, 1.0)
            self.history_true.append(self.h_float*self.config.max_state)
            
            self.times.append(t)
            
    def plot_results(self):
        """Plot input wave, wave-based LTC state, and float LTC reference."""
        fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(10,8), sharex=True)
        
        t_arr = np.array(self.times)
        wave_arr = np.array(self.history_wave)
        true_arr = np.array(self.history_true)
        
        # reconstruct input wave in [0..max_state]
        x_wave = self.config.input_amp * np.sin(
            2*np.pi*self.config.input_freq*t_arr + self.config.input_phase
        )
        x_scaled = (x_wave + 1.0)*0.5*self.config.max_state
        
        ax1.plot(t_arr, x_scaled, label='Input scaled')
        ax1.set_ylabel('Scaled Input')
        ax1.legend()
        
        ax2.plot(t_arr, wave_arr, color='tab:orange', label='Wave LTC (Carrier-based)')
        ax2.set_ylabel('Wave LTC State')
        ax2.legend()
        
        ax3.plot(t_arr, true_arr, color='tab:green', label='Float LTC State')
        ax3.set_ylabel('Float LTC State')
        ax3.set_xlabel('Time (s)')
        ax3.legend()
        
        plt.suptitle(
            "Wave-based LTC with Carrier + Scale vs. Float LTC\n(Sinusoidal Input)"
        )
        plt.tight_layout()
        plt.show()

def demo_wave_ltc_carrier():
    # Try a short run with alpha=2, wave_ltc_scale=200,
    # sample_rate=1000, carrier_freq=20k
    config = WaveLTCConfig(
        alpha=2.0,
        sample_rate=1000,
        duration=0.05,
        max_state=100,
        init_state=50,
        input_freq=20.0,
        input_amp=1.0,
        input_phase=0.0,
        carrier_freq=20000,
        wave_ltc_scale=200.0  # This helps ensure toggles_total >= 1 in typical conditions
    )
    neuron = WaveLTCNeuron(config)
    neuron.run_simulation()
    neuron.plot_results()

if __name__ == "__main__":
    demo_wave_ltc_carrier()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class SelfInterferenceConfig:
    """
    Config for a single wave-based LTC-like neuron with XOR-based self-interference for decay.
    """
    def __init__(self,
                 sample_rate=1000,   # macro steps per second
                 duration=0.05,      # total sim time (seconds)
                 max_state=64,       # integer range
                 init_state=32,      # start
                 
                 # input wave
                 input_freq=10.0,
                 input_amp=1.0,
                 input_phase=0.0,
                 
                 # wave LTC parameters
                 alpha=2.0,          # LTC gain
                 wave_scale=50.0,    # scale factor for toggles
                 
                 # self interference
                 self_delay=0.0005,  # shift in time for XOR
                 carrier_freq=10000
                 ):

        self.sample_rate  = sample_rate
        self.duration     = duration
        self.max_state    = max_state
        self.init_state   = init_state
        
        self.input_freq   = input_freq
        self.input_amp    = input_amp
        self.input_phase  = input_phase
        
        self.alpha        = alpha
        self.wave_scale   = wave_scale
        self.self_delay   = self_delay
        self.carrier_freq = carrier_freq
        
class SelfInterferenceNeuron:
    """
    A wave-based LTC-like neuron that uses XOR-based self-interference for decay.
    """
    def __init__(self, config: SelfInterferenceConfig):
        self.config = config
        self.h_state = config.init_state  # integer
        # For reference, let's keep a float LTC as well:
        self.h_float = float(config.init_state)/config.max_state
        
        # For storing time, wave-based state, float-based state
        self.times = []
        self.wave_hist = []
        self.float_hist = []
        
    def run_simulation(self):
        dt_macro = 1.0/self.config.sample_rate
        steps = int(self.config.sample_rate * self.config.duration)
        
        # We'll define how many sub toggles we do each macro step:
        carrier_cycles = int(round(self.config.carrier_freq / self.config.sample_rate))
        
        # Precompute the delay in # of sub steps
        sub_dt = dt_macro / carrier_cycles
        self_delay_steps = int(round(self.config.self_delay / sub_dt))
        
        # We'll keep a small ring buffer for the neuron's wave bits
        # e.g. store on/off states over the last X sub steps
        ring_size = 2*max(1, self_delay_steps+1)
        wave_ring = np.zeros(ring_size, dtype=int)
        ring_index = 0
        
        # For the float LTC
        h_f = self.h_float
        
        for step_i in range(steps):
            t = step_i*dt_macro
            
            # 1) compute input wave
            x_t = self.config.input_amp * np.sin(2*np.pi*self.config.input_freq*t + self.config.input_phase)
            # scale to [0..max_state]
            x_scaled = 0.5*(x_t + 1.0)
            x_int = int(round(x_scaled*self.config.max_state))
            
            # difference
            diff = x_int - self.h_state
            mag = abs(diff)
            sign = np.sign(diff)
            
            # toggles total
            toggles_total = int(round(self.config.alpha * mag * dt_macro * self.config.wave_scale))
            if toggles_total>carrier_cycles:
                toggles_total = carrier_cycles
            
            # For each sub-step in the carrier:
            for sub_i in range(carrier_cycles):
                # 1) external wave toggles
                do_toggle = 0
                if sub_i < toggles_total:
                    do_toggle = 1
                
                # 2) self interference wave
                # We'll see what was the neuron wave bit "self_delay_steps" ago
                # ring buffer approach
                old_index = (ring_index - self_delay_steps) % ring_size
                old_bit = wave_ring[old_index]
                
                # XOR with the old bit to cause partial destructive interference if the neuron is "too high"
                # We'll define "self_interf = old_bit" if h_state > x_int, otherwise 0
                # or simpler approach: if sign<0 => we do a self-interference toggle
                self_interf = 0
                if sign<0:
                    self_interf = old_bit  # meaning if the wave was 1 in the past, we XOR it now
                
                # final wave bit for this sub-step
                wave_bit = do_toggle ^ self_interf  # XOR
                
                # apply to h_state
                if wave_bit==1:
                    if sign>0:
                        self.h_state = min(self.h_state+1, self.config.max_state)
                    else:
                        self.h_state = max(self.h_state-1, 0)
                        
                # store wave_bit in ring
                wave_ring[ring_index] = wave_bit
                ring_index = (ring_index+1) % ring_size
            
            self.wave_hist.append(self.h_state)
            
            # float LTC ref
            dh = self.config.alpha*(x_scaled - h_f)
            h_f += dh*dt_macro
            h_f = np.clip(h_f, 0.0, 1.0)
            self.float_hist.append(h_f*self.config.max_state)
            
            self.times.append(t)
    
    def plot_results(self):
        fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(10,8), sharex=True)
        
        t_arr = np.array(self.times)
        wave_arr = np.array(self.wave_hist)
        float_arr = np.array(self.float_hist)
        
        # reconstruct input wave in [0..max_state]
        x_wave = self.config.input_amp*np.sin(2*np.pi*self.config.input_freq*t_arr + self.config.input_phase)
        x_scaled = (x_wave+1.0)*0.5*self.config.max_state
        
        ax1.plot(t_arr, x_scaled, label='Input [0..max_state]')
        ax1.set_ylabel('Input scaled')
        ax1.legend()
        
        ax2.plot(t_arr, wave_arr, color='tab:orange', label='Wave LTC + XOR Decay')
        ax2.set_ylabel('Wave LTC State')
        ax2.legend()
        
        ax3.plot(t_arr, float_arr, color='tab:green', label='Float LTC Ref')
        ax3.set_ylabel('Float LTC State')
        ax3.set_xlabel('Time (s)')
        ax3.legend()
        
        plt.suptitle("Wave LTC w/ Self-Interference via XOR for Decay\n(Comparison to Float LTC)")
        plt.tight_layout()
        plt.show()

def demo_self_interference():
    cfg = SelfInterferenceConfig(
        sample_rate=1000,
        duration=0.07,
        max_state=64,
        init_state=32,
        input_freq=8.0,
        input_amp=1.0,
        input_phase=0.0,
        
        alpha=2.0,
        wave_scale=60.0,     # scale toggles
        self_delay=0.0008,   # ~ 0.8ms delay
        carrier_freq=8000
    )
    neuron = SelfInterferenceNeuron(cfg)
    neuron.run_simulation()
    neuron.plot_results()

if __name__=="__main__":
    demo_self_interference()

In [ ]:
%pip install seaborn

In [ ]:
########################################
# BIZARRO C. ELEGANS WAVE NOTEBOOK
########################################

#########################
# 1) Imports
#########################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#########################
# 2) Binary Wave Model
#########################

class BinaryWaveSegment:
    """
    A single segment of the Bizarro wave chain, storing:
      - wave_state as 8-bit integer
      - thresholds for Ca and K
      - toggle_history for conduction delay
    """
    def __init__(self):
        # 8-bit approximations for typical C. elegans ranges
        self.wave_state    = 0b00101100   # ~300 => resting ~ -40mV
        self.ca_threshold  = 0b01001100   # ~600 => UNC-2 activation
        self.k_threshold   = 0b11001000   # ~800 => EXP-2 activation
        self.toggle_history= []           # backlog for conduction delay
        self.max_state     = 0xFF         # 8-bit saturation

    def __repr__(self):
        return f"BinaryWaveSegment(wave_state={bin(self.wave_state)})"

def binary_propagate(segments, conduction_delay=2):
    """
    Bidirectional wave conduction in base-2 integer form.
    - Each segment’s wave_state is used to derive a conduction amplitude (field_bits).
    - We pass toggles left/right, store them, and apply after conduction_delay steps.
    """
    n = len(segments)
    toggles = [0]*n  # new toggles to be queued for each segment

    for i, seg in enumerate(segments):
        # Example field strength ~ wave_state >> 3 => 5 bits
        field_bits = (seg.wave_state >> 3) & 0x1F  # 0..31 range
        
        # Bidirectional conduction
        if i>0:
            toggles[i-1] = min(field_bits, 0x1F)
        if i<n-1:
            toggles[i+1] = min(field_bits, 0x1F)

    # Apply conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            # apply the oldest toggles
            oldest_toggles = seg.toggle_history.pop(0)
            new_val = seg.wave_state + oldest_toggles
            # saturate at 8 bits
            if new_val > seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Ion channel logic in base-2:
    - If wave_state>ca_threshold => Ca push
    - If wave_state>k_threshold => K pull
    """
    # UNC-2 (Ca) push
    if segment.wave_state > segment.ca_threshold:
        # partial increments
        ca_current = (segment.wave_state - segment.ca_threshold) >> 2
        # optional fine-grain
        ca_current += ((segment.wave_state & 0x0F) >> 1)

        new_val = segment.wave_state + ca_current
        if new_val>segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val

    # EXP-2 (K) pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 1
        # extra decay
        k_current += (segment.wave_state >> 4)

        new_val = segment.wave_state - k_current
        if new_val<0:
            new_val=0
        segment.wave_state = new_val

def update_binary_wave(segments):
    """
    Full update step:
      1) conduction
      2) ion channel logic
      3) reflection at boundaries
    """
    # conduction
    binary_propagate(segments)

    # channels
    for seg in segments:
        binary_ion_channels(seg)

    # reflection at edges (~30%)
    if segments:
        # left
        ref_left = segments[0].wave_state >> 2  # ~1/4 bits
        segments[0].wave_state ^= ref_left
        # right
        ref_right = segments[-1].wave_state >> 2
        segments[-1].wave_state ^= ref_right

#########################
# 3) Demo + Plotting
#########################

def run_bizarro_c_elegans(num_segments=10, steps=25):
    """
    Initialize a chain of wave segments, run for some steps,
    store wave_state history, then return it for plotting.
    """
    segs = [BinaryWaveSegment() for _ in range(num_segments)]
    
    # artificially excite the first segment
    segs[0].wave_state = 0b01100000  # ~0x60 => partial amplitude
    history = []
    
    for step in range(steps):
        # record states
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        # update
        update_binary_wave(segs)
    
    # store final
    wave_snapshot = [seg.wave_state for seg in segs]
    history.append(wave_snapshot)

    return np.array(history)

def plot_bizarro_results(history):
    """
    Plot wave states in a Kaggle-competition–style multi-plot:
      1) Heatmap of wave_state vs. (time, segment)
      2) Time-series lines for each segment
      3) Final distribution
    """
    # history: shape (steps+1, num_segments)
    steps_plus = history.shape[0]
    num_segments = history.shape[1]

    # create time axis
    time_axis = np.arange(steps_plus)

    fig = plt.figure(figsize=(15, 10))
    gs = fig.add_gridspec(3, 1, height_ratios=[3, 3, 2])
    
    # 1) Heatmap
    ax1 = fig.add_subplot(gs[0])
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")
    
    # 2) Time-series lines for each segment
    ax2 = fig.add_subplot(gs[1])
    for seg_idx in range(num_segments):
        ax2.plot(time_axis, history[:, seg_idx], label=f"Seg {seg_idx}")
    ax2.set_title("Wave State Over Time (Line Plot)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0-255)")
    ax2.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    ax2.grid(True)
    
    # 3) Final distribution
    ax3 = fig.add_subplot(gs[2])
    final_states = history[-1, :]
    ax3.hist(final_states, bins=range(0,256,8), alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State (0-255)")
    ax3.set_ylabel("Count")
    ax3.grid(True)
    
    plt.tight_layout()
    return fig

#########################
# 4) Main Execution
#########################

if __name__ == "__main__":
    # run the simulation
    history = run_bizarro_c_elegans(num_segments=10, steps=25)
    # create the multi-plot
    fig = plot_bizarro_results(history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary Wave Model with Higher Bit Depth
########################################

class BinaryWaveSegment:
    """
    A single segment of the wave chain, storing:
      - wave_state as an integer with higher bit depth (e.g., 16-bit).
      - thresholds for Ca and K in the same range
      - toggle_history for conduction delay
      - max_state defines saturation based on bit_depth
    """
    def __init__(self, bit_depth=16):
        self.bit_depth    = bit_depth
        self.max_state    = (1 << bit_depth) - 1  # e.g., 65535 for 16-bit

        # Choose sensible default wave states:
        self.wave_state  = self.max_state // 4    # ~25% => resting
        self.ca_threshold= self.max_state // 2    # ~50% => Calcium activation
        self.k_threshold = (3 * self.max_state) // 4  # ~75% => Potassium activation

        self.toggle_history = []
    
    def __repr__(self):
        return (f"BinaryWaveSegment("
                f"bit_depth={self.bit_depth}, "
                f"wave_state={self.wave_state})")


def binary_propagate(segments, conduction_delay=2):
    """
    Bidirectional wave conduction using integer toggles,
    scaled for higher bit-depth.  We'll do a field_bits approach
    that takes a fraction of wave_state as conduction amplitude.
    """
    n = len(segments)
    toggles = [0]*n
    
    for i, seg in enumerate(segments):
        # Example approach: 
        # scale wave_state down by (bit_depth - 5) bits
        # so we get ~ 5 bits of conduction amplitude
        shift_amount = max(0, seg.bit_depth - 5)  # keep about 5 bits
        field_bits = (seg.wave_state >> shift_amount)
        
        # Cap the conduction amplitude
        if field_bits > 0x1F:  # 31 in decimal
            field_bits = 0x1F
        
        # Pass toggles left & right
        if i>0:
            toggles[i-1] = field_bits
        if i<n-1:
            toggles[i+1] = field_bits

    # apply conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val > seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])


def binary_ion_channels(segment):
    """
    Ion channel logic for 16-bit or more:
    - If wave_state>ca_threshold => partial Ca push
    - If wave_state>k_threshold  => partial K pull
    """
    # Calcium up
    if segment.wave_state > segment.ca_threshold:
        # partial difference
        ca_current = (segment.wave_state - segment.ca_threshold) >> 2
        # optional finer control: read lower bits
        lower_mask = (1 << 4) - 1  # e.g. 4 bits
        ca_current += (segment.wave_state & lower_mask) >> 1
        
        new_val = segment.wave_state + ca_current
        if new_val > segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val

    # Potassium down
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 1
        # extra decay
        k_current += (segment.wave_state >> (segment.bit_depth // 4))
        # e.g. wave_state >> 4 if bit_depth=16

        new_val = segment.wave_state - k_current
        if new_val < 0:
            new_val = 0
        segment.wave_state = new_val


def update_binary_wave(segments):
    """
    Full update step:
      1) conduction
      2) ion channel logic
      3) reflection at boundaries (some fraction)
    """
    # conduction
    binary_propagate(segments)

    # channel updates
    for seg in segments:
        binary_ion_channels(seg)

    # boundary reflection (~30%)
    if segments:
        left_seg  = segments[0]
        right_seg = segments[-1]
        # e.g. shift by bit_depth // 3 => ~ ~33% reflection
        shift_for_reflection = max(0, left_seg.bit_depth - 2)
        
        ref_left = left_seg.wave_state >> shift_for_reflection
        left_seg.wave_state ^= ref_left
        
        ref_right = right_seg.wave_state >> shift_for_reflection
        right_seg.wave_state ^= ref_right


########################################
# 3) Demo + Plotting
########################################

def run_bizarro_c_elegans(bit_depth=16, num_segments=10, steps=25):
    """
    Initialize a chain with a chosen bit_depth (16 by default),
    run for 'steps', collect wave_state history, and return it
    for visualization.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    
    # artificially excite the first segment:
    # e.g. wave_state ~ 50% of max for a decent wave start
    segs[0].wave_state = (segs[0].max_state // 2)
    
    history = []
    for step in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    history.append([seg.wave_state for seg in segs])  # final state
    return np.array(history)

def plot_bizarro_results(history):
    """
    Plot wave states with the same approach:
      1) Heatmap
      2) Time-series lines
      3) Final distribution
    """
    steps_plus = history.shape[0]
    num_segments = history.shape[1]
    
    time_axis = np.arange(steps_plus)
    
    fig = plt.figure(figsize=(15, 10))
    gs = fig.add_gridspec(3,1, height_ratios=[3,3,2])
    
    # 1) Heatmap
    ax1 = fig.add_subplot(gs[0])
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")
    
    # 2) Time-series lines
    ax2 = fig.add_subplot(gs[1])
    for seg_idx in range(num_segments):
        ax2.plot(time_axis, history[:, seg_idx], label=f"Seg {seg_idx}")
    ax2.set_title("Wave State Over Time (Line Plot)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0..max_val)")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) Final distribution
    ax3 = fig.add_subplot(gs[2])
    final_states = history[-1, :]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)
    
    plt.tight_layout()
    return fig

########################################
# 4) Main Execution for Kaggle Notebook
########################################

if __name__ == "__main__":
    # e.g., 16-bit, 10 segments, 25 steps
    history = run_bizarro_c_elegans(bit_depth=16, num_segments=10, steps=25)
    fig = plot_bizarro_results(history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Enhanced Binary Wave Model
########################################

class BinaryWaveSegment:
    """
    16-bit wave segment for the Bizarro Universe wave model,
    with improved conduction strength and threshold logic.
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1  # e.g. 65535 for 16-bit

        # Lower resting => more room to build conduction
        self.wave_state  = self.max_state // 8   # ~12.5% => 'resting'
        
        # Ca threshold ~25% => earlier Ca activation
        self.ca_threshold= self.max_state // 4
        # K threshold ~50% => earlier K repolarization
        self.k_threshold = self.max_state // 2
        
        self.toggle_history = []
    
    def __repr__(self):
        return (f"BinaryWaveSegment("
                f"bit_depth={self.bit_depth}, "
                f"wave_state={self.wave_state})")

def binary_propagate(segments, conduction_delay=1):
    """
    Improved conduction:
     - shift_amount = bit_depth - 8 => keep more bits => stronger conduction amplitude
     - double field_bits => 2x conduction
     - conduction_delay=1 => faster wave progression
    """
    n = len(segments)
    toggles = [0]*n
    
    for i, seg in enumerate(segments):
        # Fewer bits shifted => stronger conduction
        shift_amount = max(0, seg.bit_depth - 8)  # e.g. 16-8=8 => wave_state>>8
        field_bits   = (seg.wave_state >> shift_amount)
        
        # Double conduction amplitude, cap at 255 (arbitrary clamp)
        conduction_strength = min(field_bits * 2, 0xFF)
        
        if i>0:
            toggles[i-1] = conduction_strength
        if i<n-1:
            toggles[i+1] = conduction_strength

    # apply conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Enhanced Ca & K dynamics:
     - Ca: partial difference + bits from wave_state
     - K: partial difference + shift-based decay
    """
    # Ca push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 2
        # additional fine control from lower 8 bits
        ca_current += (segment.wave_state & 0xFF) >> 2
        
        new_val = segment.wave_state + ca_current
        if new_val>segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # K pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 1
        # additional decay factor
        k_current += (segment.wave_state >> (segment.bit_depth // 8)) # e.g. wave_state>>2 if bit_depth=16

        new_val = segment.wave_state - k_current
        if new_val<0:
            new_val = 0
        segment.wave_state = new_val

def update_binary_wave(segments):
    """
    1) conduction
    2) ion channels
    3) reflection
    """
    # conduction
    binary_propagate(segments, conduction_delay=1)
    
    # ion channels
    for seg in segments:
        binary_ion_channels(seg)
    
    # reflection (~30%) at boundaries
    if len(segments)>0:
        left_seg = segments[0]
        right_seg= segments[-1]
        
        # partial reflection using shift of bit_depth//3 => ~1/8 or 1/6
        shift_for_reflect = max(1, left_seg.bit_depth//3)
        
        ref_left = left_seg.wave_state >> shift_for_reflect
        left_seg.wave_state ^= ref_left
        
        ref_right= right_seg.wave_state >> shift_for_reflect
        right_seg.wave_state ^= ref_right

########################################
# 3) Demo + Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    """
    Initialize a chain with higher bit depth,
    run for 'steps', collect wave_state history, then return for plotting.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    
    # artificially excite segment[0] a bit more
    segs[0].wave_state = (segs[0].max_state // 2) # ~50% to ensure wave
    history = []
    
    for step in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    # final
    wave_snapshot = [seg.wave_state for seg in segs]
    history.append(wave_snapshot)
    return np.array(history)

def plot_bizarro_results(history):
    """
    Plot wave states:
      1) heatmap
      2) line plot
      3) final distribution
    """
    steps_plus   = history.shape[0]
    num_segments = history.shape[1]
    
    time_axis = np.arange(steps_plus)
    
    fig = plt.figure(figsize=(15,10))
    gs = fig.add_gridspec(3,1, height_ratios=[3,3,2])
    
    # 1) Heatmap
    ax1 = fig.add_subplot(gs[0])
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")
    
    # 2) Time-series lines
    ax2 = fig.add_subplot(gs[1])
    for seg_idx in range(num_segments):
        ax2.plot(time_axis, history[:, seg_idx], label=f"Seg {seg_idx}")
    ax2.set_title("Wave State Over Time (Line Plot)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0..max_val)")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)
    
    # 3) Final distribution
    ax3 = fig.add_subplot(gs[2])
    final_states = history[-1, :]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)
    
    plt.tight_layout()
    return fig

########################################
# 4) Main Execution
########################################

if __name__ == "__main__":
    history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary Wave Model with Further Adjustments
########################################

class BinaryWaveSegment:
    """
    Each segment stores a 16-bit (or user-specified) wave_state,
    plus threshold settings for Ca and K channels.
    These parameters have been lowered so we don't saturate as easily.
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1
        
        # Lower initial state => about 6.25% of max
        self.wave_state  = self.max_state // 16

        # Ca threshold ~12.5% => earlier activation
        self.ca_threshold= self.max_state // 8

        # K threshold ~25% => faster repolarization
        self.k_threshold = self.max_state // 4
        
        self.toggle_history = []
    
    def __repr__(self):
        return (f"BinaryWaveSegment("
                f"bit_depth={self.bit_depth}, "
                f"wave_state={self.wave_state})")


def binary_propagate(segments, conduction_delay=1):
    """
    Enhanced conduction:
      - shift_amount = bit_depth - 10 => keep more bits for conduction amplitude
      - multiply by 3 => stronger conduction
      - conduction_delay=1 => wave toggles apply each step
    """
    n = len(segments)
    toggles = [0]*n

    for i, seg in enumerate(segments):
        # Keep more bits => wave_state >> (bit_depth - 10)
        # ensures we still get up to (2^10=1024) amplitude
        shift_amount = max(0, seg.bit_depth - 10)
        field_bits   = (seg.wave_state >> shift_amount)

        # triple conduction amplitude, cap at 0x3FF (~1023)
        conduction_strength = min(field_bits * 3, 0x3FF)

        # Pass toggles left/right
        if i > 0:
            toggles[i - 1] = conduction_strength
        if i < n-1:
            toggles[i + 1] = conduction_strength

    # apply conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            # saturate at max_state
            if new_val > seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val

        seg.toggle_history.append(toggles[i])


def binary_ion_channels(segment):
    """
    Ion channel logic with more emphasis on repolarization:
      - Ca threshold triggers partial 'ca_current'
      - K threshold triggers partial 'k_current' to pull wave_state down
    """
    # Calcium push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 2
        # Additional fine control from lower 8 bits
        ca_current += (segment.wave_state & 0xFF) >> 2
        
        new_val = segment.wave_state + ca_current
        if new_val > segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val

    # Potassium pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 1
        # Additional decay factor
        k_current += (segment.wave_state >> (segment.bit_depth // 8))
        
        new_val = segment.wave_state - k_current
        if new_val < 0:
            new_val = 0
        segment.wave_state = new_val


def update_binary_wave(segments):
    """
    Full step:
      1) conduction
      2) ion channel updates
      3) reflection at boundaries (~30%)
    """
    # conduction
    binary_propagate(segments, conduction_delay=1)

    # channels
    for seg in segments:
        binary_ion_channels(seg)

    # boundary reflection (~30%)
    if segments:
        left_seg  = segments[0]
        right_seg = segments[-1]
        
        # reflection with shift_of ~ bit_depth//3 => ~1/8 to 1/6 wave_state
        shift_for_reflect = max(1, left_seg.bit_depth // 3)
        
        # left boundary
        ref_left = left_seg.wave_state >> shift_for_reflect
        left_seg.wave_state ^= ref_left
        
        # right boundary
        ref_right= right_seg.wave_state >> shift_for_reflect
        right_seg.wave_state ^= ref_right

########################################
# 3) Demo & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    """
    Create wave segments with the new parameters,
    run for 'steps' updates, record wave_state each step,
    then return for plotting.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    
    # artificially excite segment 0 for wave initiation
    # ~40% => helps ensure it crosses threshold
    segs[0].wave_state = (segs[0].max_state * 2)//5

    history = []
    for step in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    # final
    wave_snapshot = [seg.wave_state for seg in segs]
    history.append(wave_snapshot)
    return np.array(history)


def plot_bizarro_results(history):
    """
    Plot:
     1) Heatmap (time step vs segment)
     2) Time-series lines
     3) Distribution of final wave_state
    """
    steps_plus   = history.shape[0]
    num_segments = history.shape[1]
    time_axis    = np.arange(steps_plus)

    fig = plt.figure(figsize=(15,10))
    gs = fig.add_gridspec(3,1, height_ratios=[3,3,2])

    # 1) Heatmap
    ax1 = fig.add_subplot(gs[0])
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")

    # 2) Time-series
    ax2 = fig.add_subplot(gs[1])
    for seg_idx in range(num_segments):
        ax2.plot(time_axis, history[:, seg_idx], label=f"Seg {seg_idx}")
    ax2.set_title("Wave State Over Time (Line Plot)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0..max_val)")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) Final Distribution
    ax3 = fig.add_subplot(gs[2])
    final_states = history[-1, :]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    plt.tight_layout()
    return fig

########################################
# 4) Main
########################################

if __name__ == "__main__":
    # e.g., 16 bits, 10 segments, 25 steps
    history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary Wave Model with Further Refined Parameters
########################################

class BinaryWaveSegment:
    """
    A single segment storing a 16-bit (or user-specified) wave_state,
    plus conduction and threshold logic refined to reduce saturation
    and produce smoother wave propagation.
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1  # e.g. 65535

        # Further reduced initial wave_state => ~3%
        self.wave_state  = self.max_state // 32
        
        # Ca threshold ~6%
        self.ca_threshold= self.max_state // 16
        
        # K threshold ~12%
        self.k_threshold = self.max_state // 8
        
        self.toggle_history = []
    
    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"

def binary_propagate(segments, conduction_delay=1):
    """
    Enhanced conduction approach:
      - shift_amount = bit_depth - 12 => keep more bits => conduction amplitude
      - multiply by 2, cap at 0x7FF => up to 2047 toggles
    """
    n = len(segments)
    toggles = [0]*n
    
    for i, seg in enumerate(segments):
        # keep about 12 bits => wave_state >> (bit_depth - 12)
        shift_amount = max(0, seg.bit_depth - 12)
        field_bits   = (seg.wave_state >> shift_amount)
        
        # double conduction amplitude, cap at 0x7FF (~2047)
        conduction_strength = min(field_bits * 2, 0x7FF)

        # pass toggles left & right
        if i > 0:
            toggles[i-1] = conduction_strength
        if i < n-1:
            toggles[i+1] = conduction_strength

    # apply conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val > seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Ion channel logic:
      - Ca threshold => smaller increments
      - K threshold => stronger repolarization
    """
    # UNC-2 (Ca2+) push
    if segment.wave_state > segment.ca_threshold:
        # partial difference
        ca_current = (segment.wave_state - segment.ca_threshold) >> 3
        # extra fine control from lower 10 bits
        ca_current += (segment.wave_state & 0x3FF) >> 3

        new_val = segment.wave_state + ca_current
        if new_val > segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # EXP-2 (K) pull
    if segment.wave_state > segment.k_threshold:
        # partial difference
        k_current = (segment.wave_state - segment.k_threshold) >> 2
        # additional decay factor
        k_current += segment.wave_state >> (segment.bit_depth // 6)
        # e.g. wave_state>>2 if bit_depth=16 => more K effect

        new_val = segment.wave_state - k_current
        if new_val < 0:
            new_val = 0
        segment.wave_state = new_val

def update_binary_wave(segments):
    """
    Full step:
     1) conduction
     2) ion channels
     3) reflection (~30%)
    """
    # conduction
    binary_propagate(segments, conduction_delay=1)

    # channel logic
    for seg in segments:
        binary_ion_channels(seg)

    # boundary reflection
    if segments:
        left_seg  = segments[0]
        right_seg = segments[-1]
        
        shift_for_reflect = max(1, left_seg.bit_depth // 3)
        
        # partial reflection left
        ref_left = left_seg.wave_state >> shift_for_reflect
        left_seg.wave_state ^= ref_left
        
        # partial reflection right
        ref_right = right_seg.wave_state >> shift_for_reflect
        right_seg.wave_state ^= ref_right

########################################
# 3) Demo & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    """
    Create wave segments with the new parameters,
    run the system for 'steps', record wave_state each step,
    then return for plotting.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    
    # artificially excite segment[0] => ~10% => ensures crossing threshold
    segs[0].wave_state = segs[0].max_state // 10

    history = []
    for step in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    # final state
    wave_snapshot = [seg.wave_state for seg in segs]
    history.append(wave_snapshot)
    return np.array(history)

def plot_bizarro_results(history):
    """
    Standard triple-plot: heatmap, line plot, final distribution
    """
    steps_plus   = history.shape[0]
    num_segments = history.shape[1]
    time_axis    = np.arange(steps_plus)

    fig = plt.figure(figsize=(15,10))
    gs = fig.add_gridspec(3,1, height_ratios=[3,3,2])

    # 1) Heatmap
    ax1 = fig.add_subplot(gs[0])
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")

    # 2) Time-series
    ax2 = fig.add_subplot(gs[1])
    for seg_idx in range(num_segments):
        ax2.plot(time_axis, history[:, seg_idx], label=f"Seg {seg_idx}")
    ax2.set_title("Wave State Over Time (Line Plot)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0..max_val)")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) Final distribution
    ax3 = fig.add_subplot(gs[2])
    final_states = history[-1, :]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    plt.tight_layout()
    return fig

########################################
# 4) Main
########################################

if __name__ == "__main__":
    # e.g., run 10 segments, 25 steps, 16-bit
    history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary Wave Model with Even Lower Initial States & Finer Control
########################################

class BinaryWaveSegment:
    """
    A single segment storing a wave_state in 16-bit (by default) integer form,
    with extremely low initial states, lower thresholds, and
    toggles for conduction, Ca, and K in pure integer ops.
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1  # e.g. 65535 for 16-bit

        # Very low initial wave_state => ~1.5% of max
        self.wave_state = self.max_state // 64

        # Ca threshold ~3% => triggers earlier
        self.ca_threshold = self.max_state // 32

        # K threshold ~6% => repolarizes sooner
        self.k_threshold  = self.max_state // 16

        self.toggle_history = []
    
    def __repr__(self):
        return (f"BinaryWaveSegment("
                f"bit_depth={self.bit_depth}, "
                f"wave_state={self.wave_state})")


def binary_propagate(segments, conduction_delay=1):
    """
    Conduction logic with:
      - shift_amount = bit_depth - 14 => keep up to 14 bits
      - multiply by 3, then shift right by 1 => ~1.5x conduction
      - capped at 0x1FFF (~8191) for conduction amplitude
    """
    n = len(segments)
    toggles = [0]*n
    
    for i, seg in enumerate(segments):
        shift_amount = max(0, seg.bit_depth - 14)  # e.g. 16-14=2 => wave_state>>2
        field_bits   = (seg.wave_state >> shift_amount)
        
        # conduction_strength = min((field_bits*3)>>1, 0x1FFF)
        conduction_strength  = (field_bits * 3) >> 1
        if conduction_strength > 0x1FFF:
            conduction_strength = 0x1FFF
        
        # pass toggles left & right
        if i>0:
            toggles[i-1] = conduction_strength
        if i<n-1:
            toggles[i+1] = conduction_strength

    # conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val > seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])


def binary_ion_channels(segment):
    """
    Ion channel updates with finer shifts:
      - Ca partial increments
      - K partial decrements
    """
    # Ca push
    if segment.wave_state > segment.ca_threshold:
        # partial difference
        ca_current = (segment.wave_state - segment.ca_threshold) >> 4
        # finer control from lower ~13 bits
        ca_current += (segment.wave_state & 0x1FFF) >> 4

        new_val = segment.wave_state + ca_current
        if new_val > segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # K pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 3
        # extra decay factor
        k_current += (segment.wave_state >> (segment.bit_depth // 4))
        # e.g. bit_depth//4=4 => wave_state>>4

        new_val = segment.wave_state - k_current
        if new_val < 0:
            new_val = 0
        segment.wave_state = new_val

def update_binary_wave(segments):
    """
    1) conduction
    2) ion channels
    3) reflection (~30%)
    """
    binary_propagate(segments, conduction_delay=1)

    for seg in segments:
        binary_ion_channels(seg)

    # boundary reflection
    if segments:
        left_seg  = segments[0]
        right_seg = segments[-1]
        
        # partial reflection: bit_depth//3 => ~1/8 or 1/6
        shift_for_reflect = max(1, left_seg.bit_depth // 3)
        
        # left boundary
        ref_left = left_seg.wave_state >> shift_for_reflect
        left_seg.wave_state ^= ref_left
        
        # right boundary
        ref_right = right_seg.wave_state >> shift_for_reflect
        right_seg.wave_state ^= ref_right

########################################
# 3) Demo & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    """
    Creates wave segments with recommended parameters,
    runs for 'steps', records wave_state, returns for plotting.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    
    # artificially excite segment 0 => ~5% => ensures wave crosses threshold
    segs[0].wave_state = (segs[0].max_state // 20)

    history = []
    for step in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    # final
    wave_snapshot = [seg.wave_state for seg in segs]
    history.append(wave_snapshot)
    return np.array(history)

def plot_bizarro_results(history):
    """
    Triple-plot (heatmap, line plot, final distribution).
    """
    steps_plus   = history.shape[0]
    num_segments = history.shape[1]
    time_axis    = np.arange(steps_plus)

    fig = plt.figure(figsize=(15,10))
    gs = fig.add_gridspec(3,1, height_ratios=[3,3,2])

    # 1) Heatmap
    ax1 = fig.add_subplot(gs[0])
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")

    # 2) Time-series
    ax2 = fig.add_subplot(gs[1])
    for seg_idx in range(num_segments):
        ax2.plot(time_axis, history[:, seg_idx], label=f"Seg {seg_idx}")
    ax2.set_title("Wave State Over Time (Line Plot)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0..max_val)")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) Final distribution
    ax3 = fig.add_subplot(gs[2])
    final_states = history[-1, :]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    plt.tight_layout()
    return fig

########################################
# 4) Main
########################################

if __name__ == "__main__":
    # e.g., run 10 segments, 25 steps, 16-bit
    history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Helper: Convert wave_state -> mV
########################################

def binary_to_mV(wave_state, bit_depth=16, range_mV=100.0, rest_mV=-70.0):
    """
    Convert [0..max_state] => [rest_mV..(rest_mV+range_mV)]
    e.g. [-70..+30] mV if range_mV=100.
    """
    max_state = (1 << bit_depth) - 1
    fraction = wave_state / max_state
    return rest_mV + range_mV * fraction

########################################
# 3) Bizarro Universe Wave Segment with Refractory & Self-Interference
########################################

class BinaryWaveSegment:
    """
    Each segment holds:
      - wave_state (16-bit)
      - ca_threshold, k_threshold
      - a short ring buffer for self-interference
      - a refractory_counter to block conduction after spiking
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1

        # Very low initial wave_state => ~1.5% to keep states near rest
        self.wave_state  = self.max_state // 64

        # Ca threshold ~4%
        self.ca_threshold= self.max_state // 24
        # K threshold ~8%
        self.k_threshold = self.max_state // 12
        
        # conduction toggles backlog
        self.toggle_history = []

        # conduction amplitude for visualization
        self.conduction_amplitude_history = []

        # self-interference ring buffer
        self.self_buffer = [0]*5  # store last 5 wave_state values

        # refractory (once wave_state surpasses k_threshold, block conduction for a few steps)
        self.refractory_counter = 0
        
        self.max_refractory = 4   # e.g. 4 steps of no conduction after a big spike

    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"

########################################
# 4) Conduction + Self-Interference + Reflection
########################################

def binary_propagate(segments, conduction_delay=1):
    """
    Conduction with gentler scaling:
      conduction_strength = (field_bits >> 1) + (field_bits >> 2)
    Then partial reflection is done later in update.
    If segment is in refractory, pass toggles=0.
    """
    n = len(segments)
    toggles = [0]*n
    conduction_strengths = [0]*n

    for i, seg in enumerate(segments):
        # self-interference ring
        seg.self_buffer = [seg.wave_state] + seg.self_buffer[:-1]

        if seg.refractory_counter>0:
            # in refractory => no conduction
            conduction_strength = 0
        else:
            # Keep e.g. 12 bits => wave_state >> (bit_depth-12)
            shift_amount = max(0, seg.bit_depth - 12)
            field_bits   = (seg.wave_state >> shift_amount)
            # gentler conduction:
            conduction_strength = (field_bits >> 1) + (field_bits >> 2)
            # e.g. capping at ~4095 if you want
            conduction_strength = min(conduction_strength, 0xFFF)

        conduction_strengths[i] = conduction_strength
        
        # pass toggles if not refractory
        if conduction_strength>0:
            if i>0:
                toggles[i-1] = conduction_strength
            if i<n-1:
                toggles[i+1] = conduction_strength

    # store conduction amplitude for plotting
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_strengths[i])

    # conduction delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Ion channel logic:
      - Ca push => if wave_state>ca_threshold
      - K pull => if wave_state>k_threshold
      - self-interference => ring of 5. If all 5 are high, forcibly subtract bits
      - refractory => if wave_state>k_threshold, start the counter
    """
    # self-interference
    # if last 5 states are all above ~ (max_state//5?), forcibly subtract?
    if all(s > segment.max_state//5 for s in segment.self_buffer):
        # forcibly subtract e.g. 2%:
        subtract_bits = segment.wave_state >> 6
        new_val = segment.wave_state - subtract_bits
        if new_val<0: new_val=0
        segment.wave_state = new_val

    # Ca push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 3
        # extra fine from lower 12 bits
        ca_current += (segment.wave_state & 0xFFF) >> 3
        new_val = segment.wave_state + ca_current
        if new_val>segment.max_state:
            new_val = segment.max_state
        segment.wave_state=new_val

    # K pull
    if segment.wave_state > segment.k_threshold:
        # strong partial
        k_current = (segment.wave_state - segment.k_threshold) >> 2
        # extra factor
        k_current += (segment.wave_state >> (segment.bit_depth // 4))
        new_val = segment.wave_state - k_current
        if new_val<0: new_val=0
        segment.wave_state=new_val
        # start or reset refractory
        segment.refractory_counter = segment.max_refractory
    
    else:
        # if not above k_threshold, reduce refractory if >0
        if segment.refractory_counter>0:
            segment.refractory_counter-=1

def partial_reflection(segments):
    """
    Instead of XOR, do a partial subtract for reflection
    ~30% reflection => wave_state >> ~2
    """
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]
    
    shift_for_reflect= max(2, left_seg.bit_depth//3)
    
    # left boundary
    ref_left= left_seg.wave_state >> shift_for_reflect
    new_val = left_seg.wave_state - ref_left
    if new_val<0: new_val=0
    left_seg.wave_state= new_val

    # right boundary
    ref_right= right_seg.wave_state >> shift_for_reflect
    new_val2 = right_seg.wave_state - ref_right
    if new_val2<0: new_val2=0
    right_seg.wave_state= new_val2

def update_binary_wave(segments):
    # conduction
    binary_propagate(segments, conduction_delay=1)
    # channels
    for seg in segments:
        binary_ion_channels(seg)
    # partial reflection subtract
    partial_reflection(segments)

########################################
# 5) Demo & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    # artificially excite segment[0] => ~5%:
    segs[0].wave_state = segs[0].max_state // 20

    history = []
    for _ in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    history.append([seg.wave_state for seg in segs])
    return segs, np.array(history)

def plot_bizarro_results(segs, history):
    steps_plus = history.shape[0]
    num_segments = history.shape[1]
    time_axis    = np.arange(steps_plus)

    # conduction array
    conduction_history = np.zeros((steps_plus, num_segments))
    max_len = max(len(seg.conduction_amplitude_history) for seg in segs)
    for i, seg in enumerate(segs):
        for t in range(len(seg.conduction_amplitude_history)):
            conduction_history[t,i] = seg.conduction_amplitude_history[t]

    # wave_mV
    wave_mV = np.zeros_like(history, dtype=float)
    for t in range(steps_plus):
        for i in range(num_segments):
            wave_mV[t,i] = binary_to_mV(history[t,i], segs[i].bit_depth, range_mV=100.0, rest_mV=-70.0)

    fig, axes = plt.subplots(3,2, figsize=(18,15))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # 1) wave_state Heatmap
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")

    # 2) wave_state lines
    for i in range(num_segments):
        ax2.plot(time_axis, history[:, i], label=f"Seg {i}")
    ax2.set_title("Wave State Over Time")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State")
    ax2.grid(True)

    # 3) final wave_state distribution
    final_states= history[-1,:]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    # 4) conduction amplitude Heatmap
    sns.heatmap(conduction_history[:steps_plus-1].T, ax=ax4, cmap='magma')
    ax4.set_title("Conduction Amplitude Heatmap")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Segment Index")

    # 5) conduction amplitude lines
    for i in range(num_segments):
        ax5.plot(np.arange(len(segs[i].conduction_amplitude_history)),
                 segs[i].conduction_amplitude_history, label=f"Seg {i}")
    ax5.set_title("Conduction Strength Over Time")
    ax5.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (toggles)")
    ax5.grid(True)

    # 6) wave_mV lines
    for i in range(num_segments):
        ax6.plot(time_axis, wave_mV[:, i], label=f"Seg {i}")
    ax6.set_title("Membrane Potential (mV approx.)")
    ax6.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax6.set_xlabel("Time Step")
    ax6.set_ylabel("Voltage (mV)")
    ax6.grid(True)

    plt.tight_layout()
    return fig

########################################
# 6) Main
########################################

if __name__ == "__main__":
    segs, history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(segs, history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary -> mV Helper
########################################

def binary_to_mV(wave_state, bit_depth=16, range_mV=100.0, rest_mV=-70.0):
    """
    Convert wave_state in [0..(2^bit_depth -1)] to an approximate membrane potential.
    By default, [0..65535] -> [-70..+30] mV.
    """
    max_state = (1 << bit_depth) - 1
    fraction = wave_state / max_state
    return rest_mV + range_mV * fraction

########################################
# 3) BinaryWaveSegment Class
########################################

class BinaryWaveSegment:
    """
    Each segment has:
      - wave_state (16-bit integer by default)
      - thresholds for Ca (~3%) and K (~6%) to encourage earlier spiking
      - conduction_amplitude_history for advanced plotting
      - wave_state starts extremely low (~1.5%) to avoid early saturation
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1  # e.g. 65535

        # Very low initial wave_state => ~1.5%
        self.wave_state = self.max_state // 64

        # Ca threshold ~3%
        self.ca_threshold = self.max_state // 32
        # K threshold ~6%
        self.k_threshold  = self.max_state // 16

        # conduction toggles backlog (for conduction delay)
        self.toggle_history = []

        # store conduction amplitude each step
        self.conduction_amplitude_history = []

    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"

########################################
# 4) Propagation & Ion Channel
########################################

def binary_propagate(segments, conduction_delay=1):
    """
    Gentle conduction approach:
      shift_amount = bit_depth - 14 => keep 14 bits
      conduction_strength = min(field_bits * 3 >>1, 0x1FFF)
    We then store conduction_strength in each segment's conduction_amplitude_history.
    """
    n = len(segments)
    toggles = [0]*n
    conduction_strengths = [0]*n

    for i, seg in enumerate(segments):
        # conduction amplitude
        shift_amount = max(0, seg.bit_depth - 14)   # keep up to 14 bits
        field_bits   = (seg.wave_state >> shift_amount)
        # multiply by ~1.5 => (field_bits * 3)>>1
        conduction_strength = (field_bits * 3) >> 1
        # cap at 0x1FFF => 8191
        if conduction_strength > 0x1FFF:
            conduction_strength = 0x1FFF

        conduction_strengths[i] = conduction_strength

        if conduction_strength>0:
            # pass toggles left/right
            if i>0:
                toggles[i-1] = conduction_strength
            if i<n-1:
                toggles[i+1] = conduction_strength
    
    # store conduction amplitude & apply conduction delay
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_strengths[i])

        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Enhanced Ion Channel logic:
      Ca push => wave_state>ca_threshold => partial difference >>4
      K pull => wave_state>k_threshold => partial difference >>3
    """
    # Calcium push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 4
        # plus some from lower ~13 bits
        ca_current += (segment.wave_state & 0x1FFF) >> 4

        new_val = segment.wave_state + ca_current
        if new_val>segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # K pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 3
        k_current += (segment.wave_state >> (segment.bit_depth // 4))
        new_val = segment.wave_state - k_current
        if new_val<0:
            new_val = 0
        segment.wave_state = new_val

def partial_reflection(segments):
    """
    partial reflection at boundaries => subtract wave_state>>shift_for_reflect
    ~30% reflection => shift_for_reflect ~ bit_depth//3
    """
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]

    shift_for_reflect = max(2, left_seg.bit_depth//3)

    # left boundary
    ref_left = left_seg.wave_state >> shift_for_reflect
    new_val  = left_seg.wave_state - ref_left
    if new_val<0:
        new_val=0
    left_seg.wave_state= new_val

    # right boundary
    ref_right= right_seg.wave_state >> shift_for_reflect
    new_val2  = right_seg.wave_state - ref_right
    if new_val2<0:
        new_val2=0
    right_seg.wave_state= new_val2

def update_binary_wave(segments):
    # conduction
    binary_propagate(segments, conduction_delay=1)
    # ion channels
    for seg in segments:
        binary_ion_channels(seg)
    # partial reflection
    partial_reflection(segments)

########################################
# 5) Demo & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    # artificially excite segment[0] => ~5%
    segs[0].wave_state = segs[0].max_state // 20

    history = []
    for _ in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    history.append([seg.wave_state for seg in segs])
    return segs, np.array(history)

def plot_bizarro_results(segs, history):
    steps_plus = history.shape[0]
    num_segments = history.shape[1]
    time_axis = np.arange(steps_plus)

    # conduction amplitude
    conduction_history = np.zeros((steps_plus, num_segments))
    for i, seg in enumerate(segs):
        for t, val in enumerate(seg.conduction_amplitude_history):
            if t<steps_plus:
                conduction_history[t,i] = val

    # wave_mV
    wave_mV = np.zeros_like(history, dtype=float)
    for t in range(steps_plus):
        for i in range(num_segments):
            wave_mV[t,i] = binary_to_mV(history[t,i], segs[i].bit_depth, range_mV=100.0, rest_mV=-70.0)

    fig, axes = plt.subplots(3,2, figsize=(18,15))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # 1) wave_state Heatmap
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment")

    # 2) wave_state line
    for i in range(num_segments):
        ax2.plot(time_axis, history[:, i], label=f"Seg {i}")
    ax2.set_title("Wave State Over Time")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State")
    ax2.grid(True)

    # 3) final wave_state distribution
    final_states = history[-1,:]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    # 4) conduction amplitude Heatmap
    sns.heatmap(conduction_history[:steps_plus-1].T, ax=ax4, cmap='magma')
    ax4.set_title("Conduction Amplitude Heatmap")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Segment")

    # 5) conduction amplitude line
    for i in range(num_segments):
        ax5.plot(np.arange(len(segs[i].conduction_amplitude_history)),
                 segs[i].conduction_amplitude_history, label=f"Seg {i}")
    ax5.set_title("Conduction Strength Over Time")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (toggles)")
    ax5.grid(True)
    ax5.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 6) wave_mV line
    for i in range(num_segments):
        ax6.plot(time_axis, wave_mV[:, i], label=f"Seg {i}")
    ax6.set_title("Membrane Potential (mV approx.)")
    ax6.set_xlabel("Time Step")
    ax6.set_ylabel("Voltage (mV)")
    ax6.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax6.grid(True)

    plt.tight_layout()
    return fig

########################################
# 6) Main
########################################

if __name__ == "__main__":
    segs, history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(segs, history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Helper: Convert binary wave_state -> mV
########################################

def binary_to_mV(wave_state, bit_depth=16, range_mV=100.0, rest_mV=-70.0):
    """
    Convert wave_state in [0..(2^bit_depth -1)] to an approximate membrane potential.
    By default: [0..65535] -> [-70..+30] mV.
    """
    max_state = (1 << bit_depth) - 1
    fraction = wave_state / max_state
    return rest_mV + range_mV * fraction

########################################
# 3) Binary Wave Segment Class
########################################

class BinaryWaveSegment:
    """
    Represents one segment in the binary wave chain.
    - wave_state: 16-bit integer
    - thresholds: Ca ~3%, K ~6% => earlier spike & repolarization
    - conduction_amplitude_history: logs conduction strength each step
    - wave_state starts extremely low (~1.5%) to avoid early saturation
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1  # e.g. 65535

        # ~1.5% of max => near rest
        self.wave_state = self.max_state // 64

        # Ca threshold ~3%
        self.ca_threshold = self.max_state // 32
        # K threshold ~6%
        self.k_threshold  = self.max_state // 16

        # Conduction toggles backlog (for conduction_delay)
        self.toggle_history = []

        # Log conduction amplitude each step
        self.conduction_amplitude_history = []

    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"

########################################
# 4) Propagation & Ion Channel Logic
########################################

def binary_propagate(segments, conduction_delay=1):
    """
    Gentle conduction approach:
      shift_amount = bit_depth - 14 => keep up to 14 bits
      conduction_strength = (field_bits * 3)>>1
      cap at 0x1FFF => 8191
    Then store conduction_strength in conduction_amplitude_history,
    and apply conduction_delay (pop from toggle_history).
    """
    n = len(segments)
    toggles = [0]*n
    conduction_strengths = [0]*n

    for i, seg in enumerate(segments):
        shift_amount = max(0, seg.bit_depth - 14)
        field_bits   = (seg.wave_state >> shift_amount)

        # multiply by ~1.5
        conduction_strength = (field_bits * 3) >> 1
        # cap at 8191
        if conduction_strength > 0x1FFF:
            conduction_strength = 0x1FFF

        conduction_strengths[i] = conduction_strength

        # pass toggles left & right
        if conduction_strength > 0:
            if i > 0:
                toggles[i-1] = conduction_strength
            if i < n-1:
                toggles[i+1] = conduction_strength

    # Store conduction amplitude & apply conduction delay
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_strengths[i])

        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val > seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Ion channel logic:
    - Ca push if wave_state>ca_threshold => partial difference >>4
    - K pull if wave_state>k_threshold => partial difference >>3, plus extra shift
    """
    # Ca push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 4
        # additional fine control from lower 13 bits
        ca_current += (segment.wave_state & 0x1FFF) >> 4

        new_val = segment.wave_state + ca_current
        if new_val > segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # K pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 3
        k_current += (segment.wave_state >> (segment.bit_depth // 4))
        new_val = segment.wave_state - k_current
        if new_val < 0:
            new_val = 0
        segment.wave_state = new_val

def partial_reflection(segments):
    """
    partial reflection => subtract wave_state >> shift_for_reflect
    ~30% reflection => shift_for_reflect ~ bit_depth//3
    """
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]

    shift_for_reflect = max(2, left_seg.bit_depth//3)

    # left boundary
    ref_left = left_seg.wave_state >> shift_for_reflect
    new_val  = left_seg.wave_state - ref_left
    if new_val < 0: 
        new_val = 0
    left_seg.wave_state = new_val

    # right boundary
    ref_right= right_seg.wave_state >> shift_for_reflect
    new_val2 = right_seg.wave_state - ref_right
    if new_val2<0:
        new_val2=0
    right_seg.wave_state= new_val2

def update_binary_wave(segments):
    # conduction
    binary_propagate(segments, conduction_delay=1)

    # ion channels
    for seg in segments:
        binary_ion_channels(seg)

    # partial reflection
    partial_reflection(segments)

########################################
# 5) Running the Simulation & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    """
    Create 'num_segments', run for 'steps', 
    store wave_state in 'history' each step,
    return segments + final history array.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    # artificially excite the first segment => ~5%
    segs[0].wave_state = segs[0].max_state // 20

    history = []
    for _ in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    history.append([seg.wave_state for seg in segs])
    return segs, np.array(history)

def plot_bizarro_results(segs, history):
    """
    Plot 6 subplots: 
      1) wave_state heatmap
      2) wave_state lines
      3) final wave_state distribution
      4) conduction amplitude heatmap
      5) conduction amplitude lines
      6) wave_state in mV lines
    """
    steps_plus = history.shape[0]
    num_segments = history.shape[1]
    time_axis = np.arange(steps_plus)

    # Build conduction amplitude array
    conduction_history = np.zeros((steps_plus, num_segments))
    for i, seg in enumerate(segs):
        for t, val in enumerate(seg.conduction_amplitude_history):
            if t < steps_plus:
                conduction_history[t, i] = val

    # Convert wave_state to mV
    wave_mV = np.zeros_like(history, dtype=float)
    for t in range(steps_plus):
        for i in range(num_segments):
            wave_mV[t, i] = binary_to_mV(history[t,i], segs[i].bit_depth, 
                                         range_mV=100.0, rest_mV=-70.0)

    fig, axes = plt.subplots(3,2, figsize=(18,15))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # 1) wave_state Heatmap
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment")

    # 2) wave_state line
    for i in range(num_segments):
        ax2.plot(time_axis, history[:, i], label=f"Seg {i}")
    ax2.set_title("Wave State Over Time")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State")
    ax2.grid(True)

    # 3) final wave_state distribution
    final_states = history[-1,:]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    # 4) conduction amplitude Heatmap
    sns.heatmap(conduction_history[:steps_plus-1].T, ax=ax4, cmap='magma')
    ax4.set_title("Conduction Amplitude Heatmap")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Segment")

    # 5) conduction amplitude line
    for i in range(num_segments):
        ax5.plot(np.arange(len(segs[i].conduction_amplitude_history)),
                 segs[i].conduction_amplitude_history, label=f"Seg {i}")
    ax5.set_title("Conduction Strength Over Time")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (toggles)")
    ax5.grid(True)
    ax5.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 6) wave_mV line
    for i in range(num_segments):
        ax6.plot(time_axis, wave_mV[:, i], label=f"Seg {i}")
    ax6.set_title("Membrane Potential (mV approx.)")
    ax6.set_xlabel("Time Step")
    ax6.set_ylabel("Voltage (mV)")
    ax6.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax6.grid(True)

    plt.tight_layout()
    return fig

########################################
# 6) Main
########################################

if __name__ == "__main__":
    segs, history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(segs, history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary -> mV Helper
########################################

def binary_to_mV(wave_state, bit_depth=16, range_mV=100.0, rest_mV=-70.0):
    """
    Convert wave_state in [0..(2^bit_depth -1)] to an approximate membrane potential.
    By default: [0..65535] -> [-70..+30] mV.
    """
    max_state = (1 << bit_depth) - 1
    fraction = wave_state / max_state
    return rest_mV + range_mV * fraction

########################################
# 3) BinaryWaveSegment Class
########################################

class BinaryWaveSegment:
    """
    Each segment has:
      - wave_state (16-bit integer, default)
      - thresholds: Ca (~3%), K (~6%)
      - conduction_amplitude_history for debugging
      - extremely low initial states to avoid early saturation
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1  # e.g. 65535

        # Very low initial wave_state => ~1.5%
        self.wave_state = self.max_state // 64

        # Ca threshold ~3%
        self.ca_threshold = self.max_state // 32
        # K threshold ~6%
        self.k_threshold  = self.max_state // 16

        # conduction toggles backlog (for conduction delay)
        self.toggle_history = []

        # store conduction amplitude each step
        self.conduction_amplitude_history = []

    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"


########################################
# 4) Propagation & Ion Channels
########################################

def binary_propagate(segments, conduction_delay=1):
    """
    Gentle conduction approach:
      shift_amount = bit_depth - 14 => keep up to 14 bits
      conduction_strength = min(field_bits * 3 >>1, 0x1FFF)
    Then store conduction_strength in each segment's conduction_amplitude_history,
    apply conduction_delay.
    """
    n = len(segments)
    toggles = [0]*n
    conduction_strengths = [0]*n

    for i, seg in enumerate(segments):
        # conduction amplitude
        shift_amount = max(0, seg.bit_depth - 14)
        field_bits   = (seg.wave_state >> shift_amount)

        # multiply by ~1.5
        conduction_strength = (field_bits * 3) >> 1
        # cap at 0x1FFF => 8191
        if conduction_strength > 0x1FFF:
            conduction_strength = 0x1FFF

        conduction_strengths[i] = conduction_strength

        if conduction_strength>0:
            # pass toggles left/right
            if i>0:
                toggles[i-1] = conduction_strength
            if i<n-1:
                toggles[i+1] = conduction_strength
    
    # store conduction amplitude & apply conduction delay
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_strengths[i])

        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Ion channel logic:
      - Ca push => wave_state>ca_threshold => partial difference >>4
      - K pull => wave_state>k_threshold => partial difference >>2 + extra shift
    """
    # Ca push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 4
        # plus some from lower bits
        ca_current += (segment.wave_state & 0x1FFF) >> 4

        new_val = segment.wave_state + ca_current
        if new_val > segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # Stronger K pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 2
        # additional factor
        k_current += (segment.wave_state >> (segment.bit_depth // 4))
        # subtract
        new_val = segment.wave_state - k_current
        if new_val < 0:
            new_val = 0
        segment.wave_state = new_val

def partial_reflection(segments):
    """
    partial reflection => subtract wave_state >> shift_for_reflect
    ~30% reflection => shift_for_reflect ~ bit_depth//3
    """
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]

    shift_for_reflect = max(2, left_seg.bit_depth//3)

    # left boundary
    ref_left = left_seg.wave_state >> shift_for_reflect
    new_val  = left_seg.wave_state - ref_left
    if new_val<0:
        new_val=0
    left_seg.wave_state= new_val

    # right boundary
    ref_right= right_seg.wave_state >> shift_for_reflect
    new_val2  = right_seg.wave_state - ref_right
    if new_val2<0:
        new_val2=0
    right_seg.wave_state= new_val2

def update_binary_wave(segments):
    # conduction
    binary_propagate(segments, conduction_delay=1)

    # ion channels
    for seg in segments:
        binary_ion_channels(seg)

    # partial reflection
    partial_reflection(segments)

########################################
# 5) Demo & Plot
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    # artificially excite segment[0] => ~5%
    segs[0].wave_state = segs[0].max_state // 20

    history = []
    for _ in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    history.append([seg.wave_state for seg in segs])
    return segs, np.array(history)

def plot_bizarro_results(segs, history):
    steps_plus = history.shape[0]
    num_segments = history.shape[1]
    time_axis = np.arange(steps_plus)

    # conduction amplitude array
    conduction_history = np.zeros((steps_plus, num_segments))
    for i, seg in enumerate(segs):
        for t, val in enumerate(seg.conduction_amplitude_history):
            if t<steps_plus:
                conduction_history[t,i] = val

    # wave_mV
    wave_mV = np.zeros_like(history, dtype=float)
    for t in range(steps_plus):
        for i in range(num_segments):
            wave_mV[t,i] = binary_to_mV(history[t,i], segs[i].bit_depth, 
                                         range_mV=100.0, rest_mV=-70.0)

    fig, axes = plt.subplots(3,2, figsize=(18,15))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # 1) wave_state Heatmap
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment")

    # 2) wave_state line
    for i in range(num_segments):
        ax2.plot(time_axis, history[:, i], label=f"Seg {i}")
    ax2.set_title("Wave State Over Time")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State")
    ax2.grid(True)

    # 3) final wave_state distribution
    final_states = history[-1,:]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    # 4) conduction amplitude Heatmap
    sns.heatmap(conduction_history[:steps_plus-1].T, ax=ax4, cmap='magma')
    ax4.set_title("Conduction Amplitude Heatmap")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Segment")

    # 5) conduction amplitude line
    for i in range(num_segments):
        ax5.plot(np.arange(len(segs[i].conduction_amplitude_history)),
                 segs[i].conduction_amplitude_history, label=f"Seg {i}")
    ax5.set_title("Conduction Strength Over Time")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (toggles)")
    ax5.grid(True)
    ax5.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 6) wave_mV line
    for i in range(num_segments):
        ax6.plot(time_axis, wave_mV[:, i], label=f"Seg {i}")
    ax6.set_title("Membrane Potential (mV approx.)")
    ax6.set_xlabel("Time Step")
    ax6.set_ylabel("Voltage (mV)")
    ax6.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax6.grid(True)

    plt.tight_layout()
    return fig

########################################
# 6) Main
########################################

if __name__ == "__main__":
    segs, history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(segs, history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary -> mV Helper
########################################

def binary_to_mV(wave_state, bit_depth=16, range_mV=50.0, rest_mV=-70.0):
    """
    Map wave_state in [0..(2^bit_depth -1)] to ~[-70..-20] mV range.
    By default, range_mV=50 => saturates at -20 mV (i.e. -70 + 50).
    """
    max_state = (1 << bit_depth) - 1
    fraction = wave_state / max_state
    return rest_mV + range_mV * fraction

########################################
# 3) BinaryWaveSegment Class
########################################

class BinaryWaveSegment:
    """
    Each segment:
      - wave_state (16-bit)
      - lower thresholds => faster activation
      - conduction_amplitude_history => debug
      - wave_state starts extremely low => ~1% => near -70mV
    """
    def __init__(self, bit_depth=16):
        self.bit_depth = bit_depth
        self.max_state = (1 << bit_depth) - 1

        # ~1% of max => near -70mV
        self.wave_state = self.max_state // 100

        # Ca threshold ~2% => faster spike
        self.ca_threshold = self.max_state // 50
        # K threshold ~3% => earlier repolarization
        self.k_threshold  = self.max_state // 33

        # conduction toggles backlog
        self.toggle_history = []

        # conduction amplitude each step
        self.conduction_amplitude_history = []

    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"

########################################
# 4) Propagation & Ion Channels
########################################

def binary_propagate(segments, conduction_delay=0):
    """
    Speed up conduction:
      shift_amount = bit_depth - 12 => keep 12 bits => up to 4095
      conduction_strength = (field_bits<<1), e.g. ~2x conduction
      conduction_delay=0 => toggles apply immediately => faster wave evolution
    """
    n = len(segments)
    toggles = [0]*n
    conduction_strengths = [0]*n

    for i, seg in enumerate(segments):
        shift_amount = max(0, seg.bit_depth - 12)  # keep up to 12 bits
        field_bits   = seg.wave_state >> shift_amount
        
        # multiply by 2
        conduction_strength = field_bits << 1
        # cap at 4095
        if conduction_strength > 0xFFF:
            conduction_strength = 0xFFF

        conduction_strengths[i] = conduction_strength

        # pass toggles left & right
        if conduction_strength>0:
            if i>0:
                toggles[i-1] = conduction_strength
            if i<n-1:
                toggles[i+1] = conduction_strength

    # record conduction amplitude
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_strengths[i])
        
        if conduction_delay>0 and len(seg.toggle_history)>=conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
            seg.toggle_history.append(toggles[i])
        else:
            # conduction_delay=0 => apply toggles immediately
            add_val = toggles[i]
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val

def binary_ion_channels(segment):
    """
    Ion channel logic with faster Ca, stronger K:
      - Ca push => wave_state>ca_threshold => partial difference >>2
      - K pull => wave_state>k_threshold => partial difference >>1 + shift => strong
    """
    # Ca push
    if segment.wave_state > segment.ca_threshold:
        # bigger chunk => wave_state>>2
        ca_current = (segment.wave_state - segment.ca_threshold) >> 2
        # plus some from lower 12 bits => wave_state&0xFFF>>2
        ca_current += (segment.wave_state & 0xFFF) >> 2
        
        new_val = segment.wave_state + ca_current
        if new_val>segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val

    # K pull
    if segment.wave_state > segment.k_threshold:
        # wave_state>>1 => strong K effect
        k_current = (segment.wave_state - segment.k_threshold) >> 1
        k_current += (segment.wave_state >> (segment.bit_depth//5))  # extra shift => bigger sub
        new_val = segment.wave_state - k_current
        if new_val<0:
            new_val = 0
        segment.wave_state=new_val

def partial_reflection(segments):
    """
    partial reflection => subtract wave_state >> shift_for_reflect
    ~40% reflection => shift_for_reflect= bit_depth//2.5 etc.
    We'll pick bit_depth//2 =>  ~1/4 wave?
    """
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]

    shift_for_reflect = max(1, int(left_seg.bit_depth//2.5))

    # left boundary
    ref_left = left_seg.wave_state >> shift_for_reflect
    new_val  = left_seg.wave_state - ref_left
    if new_val<0: 
        new_val=0
    left_seg.wave_state= new_val

    # right boundary
    ref_right= right_seg.wave_state >> shift_for_reflect
    new_val2 = right_seg.wave_state - ref_right
    if new_val2<0:
        new_val2=0
    right_seg.wave_state= new_val2

def update_binary_wave(segments):
    # conduction
    binary_propagate(segments, conduction_delay=0)  # immediate toggles => faster wave
    # ion channels
    for seg in segments:
        binary_ion_channels(seg)
    # partial reflection
    partial_reflection(segments)

########################################
# 5) Demo & Plotting
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    """
    Create wave chain, run for 'steps',
    store wave_state in 'history' each step,
    return final segments + wave_state array.
    """
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    # artificially excite segment[0] => ~2.5%
    segs[0].wave_state = segs[0].max_state // 40

    history = []
    for _ in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    # final
    history.append([seg.wave_state for seg in segs])
    return segs, np.array(history)

def plot_bizarro_results(segs, history):
    steps_plus = history.shape[0]
    num_segments= history.shape[1]
    time_axis   = np.arange(steps_plus)

    # conduction amplitude array
    conduction_history= np.zeros((steps_plus, num_segments))
    for i, seg in enumerate(segs):
        for t, val in enumerate(seg.conduction_amplitude_history):
            if t<steps_plus:
                conduction_history[t,i] = val

    # wave_mV
    wave_mV = np.zeros_like(history, dtype=float)
    for t in range(steps_plus):
        for i in range(num_segments):
            wave_mV[t,i] = binary_to_mV(history[t,i], segs[i].bit_depth, 
                                         range_mV=50.0, rest_mV=-70.0)
    #  => saturate near -20 mV at top

    fig, axes = plt.subplots(3,2, figsize=(18,15))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # 1) wave_state Heatmap
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap (Segment vs. Time Step)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment Index")

    # 2) wave_state line
    for i in range(num_segments):
        ax2.plot(time_axis, history[:, i], label=f"Seg {i}")
    ax2.set_title("Wave State Over Time")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State (0..max)")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) Final wave_state distribution
    final_states= history[-1,:]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    # 4) conduction amplitude Heatmap
    sns.heatmap(conduction_history[:steps_plus-1].T, ax=ax4, cmap='magma')
    ax4.set_title("Conduction Amplitude Heatmap")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Segment Index")

    # 5) conduction amplitude line
    for i in range(num_segments):
        ax5.plot(np.arange(len(segs[i].conduction_amplitude_history)),
                 segs[i].conduction_amplitude_history, label=f"Seg {i}")
    ax5.set_title("Conduction Strength Over Time")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (toggles)")
    ax5.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax5.grid(True)

    # 6) wave_mV line
    for i in range(num_segments):
        ax6.plot(time_axis, wave_mV[:, i], label=f"Seg {i}")
    ax6.set_title("Membrane Potential (mV approx.)")
    ax6.set_xlabel("Time Step")
    ax6.set_ylabel("Voltage (mV)")
    ax6.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax6.grid(True)

    plt.tight_layout()
    return fig

########################################
# 6) Main
########################################

if __name__ == "__main__":
    segs, history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(segs, history)
    plt.show()

In [ ]:
########################################
# 1) Imports
########################################
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

########################################
# 2) Binary -> mV Helper
########################################

def binary_to_mV(wave_state, bit_depth=16, range_mV=40.0, rest_mV=-70.0):
    """
    Convert wave_state in [0..(2^bit_depth -1)] to approx. membrane potential.
    By default, [0..65535] -> [-70..-30] mV (range=40).
    """
    max_state = (1 << bit_depth) - 1
    fraction = wave_state / max_state
    return rest_mV + range_mV * fraction

########################################
# 3) Binary Wave Segment
########################################

class BinaryWaveSegment:
    """
    Each segment:
      - wave_state (16-bit)
      - Very low initial wave_state => ~1% => near -70mV
      - Ca threshold ~2% => mild push
      - K threshold ~3% => mild repolarization
    """
    def __init__(self, bit_depth=16):
        self.bit_depth  = bit_depth
        self.max_state  = (1 << bit_depth) - 1

        # ~1% of max => near -70mV
        self.wave_state = self.max_state // 100

        # Ca threshold ~2%
        self.ca_threshold = self.max_state // 50
        # K threshold ~3%
        self.k_threshold  = self.max_state // 33

        # conduction toggles backlog
        self.toggle_history = []
        # conduction amplitude logging
        self.conduction_amplitude_history = []

    def __repr__(self):
        return f"BinaryWaveSegment(bit_depth={self.bit_depth}, wave_state={self.wave_state})"

########################################
# 4) Conduction & Ion Channels
########################################

def binary_propagate(segments, conduction_delay=1):
    """
    Gentle conduction:
     - shift_amount = bit_depth - 12 => keep 12 bits => up to 4095
     - conduction_strength = field_bits >> 2 => smaller toggles
     - conduction_delay=1 => wave toggles apply next step => mild wave front
    """
    n = len(segments)
    toggles = [0]*n
    conduction_strengths = [0]*n

    for i, seg in enumerate(segments):
        shift_amount = max(0, seg.bit_depth - 12)
        field_bits   = seg.wave_state >> shift_amount
        # conduction ~ field_bits>>2 => ~1/4
        conduction_strength = field_bits >> 2  # small toggles
        # clamp if needed
        if conduction_strength > 0xFFF:
            conduction_strength = 0xFFF

        conduction_strengths[i] = conduction_strength
        
        # pass toggles left/right
        if conduction_strength>0:
            if i>0:
                toggles[i-1] = conduction_strength
            if i<n-1:
                toggles[i+1] = conduction_strength

    # store conduction amplitude & apply conduction delay
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_strengths[i])

        if len(seg.toggle_history) >= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            new_val = seg.wave_state + add_val
            if new_val>seg.max_state:
                new_val = seg.max_state
            seg.wave_state = new_val
        seg.toggle_history.append(toggles[i])

def binary_ion_channels(segment):
    """
    Mild Ca push & stronger K pull, but limited amplitude => subthreshold waves
      - Ca if wave_state>ca_threshold => wave_state>>5
      - K if wave_state>k_threshold => wave_state>>3 + bit shift
    """
    # Ca push
    if segment.wave_state > segment.ca_threshold:
        ca_current = (segment.wave_state - segment.ca_threshold) >> 5
        ca_current += (segment.wave_state & 0xFFF) >> 5
        new_val = segment.wave_state + ca_current
        if new_val>segment.max_state:
            new_val = segment.max_state
        segment.wave_state = new_val
    
    # K pull
    if segment.wave_state > segment.k_threshold:
        k_current = (segment.wave_state - segment.k_threshold) >> 3
        # extra factor
        k_current += (segment.wave_state >> (segment.bit_depth // 5))
        new_val = segment.wave_state - k_current
        if new_val<0:
            new_val=0
        segment.wave_state=new_val

def partial_reflection(segments):
    """
    partial reflection => subtract wave_state>>shift_for_reflect
    ~20% reflection => shift_for_reflect = ~ bit_depth//4 or so
    """
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]

    shift_for_reflect = max(1, left_seg.bit_depth//4)

    # left boundary
    ref_left = left_seg.wave_state >> shift_for_reflect
    new_val  = left_seg.wave_state - ref_left
    if new_val<0:
        new_val=0
    left_seg.wave_state= new_val

    # right boundary
    ref_right= right_seg.wave_state >> shift_for_reflect
    new_val2  = right_seg.wave_state - ref_right
    if new_val2<0:
        new_val2=0
    right_seg.wave_state= new_val2

def update_binary_wave(segments):
    # conduction
    binary_propagate(segments, conduction_delay=1)
    # ion channels
    for seg in segments:
        binary_ion_channels(seg)
    # partial reflection
    partial_reflection(segments)

########################################
# 5) Demo & Plot
########################################

def run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16):
    segs = [BinaryWaveSegment(bit_depth=bit_depth) for _ in range(num_segments)]
    # artificially excite segment[0] => ~1.5% => we set wave_state = max_state//66 => ~1.5%
    segs[0].wave_state = segs[0].max_state // 66

    history = []
    for _ in range(steps):
        wave_snapshot = [seg.wave_state for seg in segs]
        history.append(wave_snapshot)
        update_binary_wave(segs)
    
    history.append([seg.wave_state for seg in segs])
    return segs, np.array(history)

def plot_bizarro_results(segs, history):
    steps_plus = history.shape[0]
    num_segments= history.shape[1]
    time_axis   = np.arange(steps_plus)

    # conduction array
    conduction_history= np.zeros((steps_plus, num_segments))
    for i, seg in enumerate(segs):
        for t, val in enumerate(seg.conduction_amplitude_history):
            if t<steps_plus:
                conduction_history[t,i] = val

    # wave_mV array
    wave_mV = np.zeros_like(history, dtype=float)
    for t in range(steps_plus):
        for i in range(num_segments):
            wave_mV[t,i] = binary_to_mV(history[t,i], segs[i].bit_depth, 
                                         range_mV=40.0, rest_mV=-70.0)

    fig, axes = plt.subplots(3,2, figsize=(18,15))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # 1) wave_state Heatmap
    sns.heatmap(history.T, ax=ax1, cmap='viridis')
    ax1.set_title("Wave States Heatmap")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Segment")

    # 2) wave_state lines
    for i in range(num_segments):
        ax2.plot(time_axis, history[:, i], label=f"Seg {i}")
    ax2.set_title("Wave State Over Time")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Wave State")
    ax2.grid(True)

    # 3) final wave_state distribution
    final_states = history[-1,:]
    ax3.hist(final_states, bins=50, alpha=0.7, color='orange')
    ax3.set_title("Final Wave State Distribution")
    ax3.set_xlabel("Wave State")
    ax3.set_ylabel("Count")
    ax3.grid(True)

    # 4) conduction amplitude Heatmap
    sns.heatmap(conduction_history[:steps_plus-1].T, ax=ax4, cmap='magma')
    ax4.set_title("Conduction Amplitude Heatmap")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Segment")

    # 5) conduction amplitude line
    for i in range(num_segments):
        ax5.plot(np.arange(len(segs[i].conduction_amplitude_history)),
                 segs[i].conduction_amplitude_history, label=f"Seg {i}")
    ax5.set_title("Conduction Strength Over Time")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (toggles)")
    ax5.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax5.grid(True)

    # 6) wave_mV lines
    for i in range(num_segments):
        ax6.plot(time_axis, wave_mV[:, i], label=f"Seg {i}")
    ax6.set_title("Membrane Potential (mV approx.)")
    ax6.set_xlabel("Time Step")
    ax6.set_ylabel("Voltage (mV)")
    ax6.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax6.grid(True)

    plt.tight_layout()
    return fig

########################################
# 6) Main
########################################

if __name__ == "__main__":
    segs, history = run_bizarro_c_elegans(num_segments=10, steps=25, bit_depth=16)
    fig = plot_bizarro_results(segs, history)
    plt.show()

In [ ]:
#!/usr/bin/env python3

import random
import time
from array import array

def limbs_to_int(limbs, chunk_bits):
    """Combine limb array into a single Python int."""
    val = 0
    shift = 0
    mask = (1 << chunk_bits) - 1
    for limb in limbs:
        val |= (limb & mask) << shift
        shift += chunk_bits
    return val

def int_to_limbs(value, chunk_bits):
    """Split a Python int into a list of fixed-size bit limbs."""
    limbs = []
    mask = (1 << chunk_bits) - 1
    while value > 0:
        limbs.append(value & mask)
        value >>= chunk_bits
    if not limbs:
        limbs.append(0)
    return limbs

def add_limbs(A, B, chunk_bits):
    """Add two limb arrays A+B -> result."""
    base = 1 << chunk_bits
    out = []
    carry = 0
    length = max(len(A), len(B))
    for i in range(length):
        av = A[i] if i < len(A) else 0
        bv = B[i] if i < len(B) else 0
        s_val = av + bv + carry
        carry = s_val >> chunk_bits
        out.append(s_val & (base-1))
    if carry:
        out.append(carry)
    return out

def mul_limbs(A, B, chunk_bits):
    """Multiply limb arrays A*B -> result."""
    base = 1 << chunk_bits
    out = [0]*(len(A) + len(B))
    for i in range(len(A)):
        carry = 0
        for j in range(len(B)):
            tmp = out[i+j] + A[i]*B[j] + carry
            out[i+j] = tmp & (base-1)
            carry = tmp >> chunk_bits
        if carry:
            out[i+len(B)] += carry
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def demo_binary_limb_arithmetic(chunk_bits=64, test_bits=512, trials=5):
    """Simple demonstration: random add and mul in chunk-based style."""
    print(f"=== Demo: chunk_bits={chunk_bits}, test_bits={test_bits} ===")
    for _ in range(trials):
        # Generate random test numbers
        A_val = random.getrandbits(test_bits)
        B_val = random.getrandbits(test_bits)
        A_arr = int_to_limbs(A_val, chunk_bits)
        B_arr = int_to_limbs(B_val, chunk_bits)

        # Timed addition
        t0 = time.perf_counter()
        sum_limbs = add_limbs(A_arr, B_arr, chunk_bits)
        dt_add = time.perf_counter() - t0
        sum_val = limbs_to_int(sum_limbs, chunk_bits)
        assert sum_val == (A_val + B_val), "Addition mismatch!"

        # Timed multiplication
        t1 = time.perf_counter()
        prod_limbs = mul_limbs(A_arr, B_arr, chunk_bits)
        dt_mul = time.perf_counter() - t1
        prod_val = limbs_to_int(prod_limbs, chunk_bits)
        assert prod_val == (A_val*B_val), "Multiplication mismatch!"

        print(f"Add {test_bits}-bit => {dt_add*1e6:.1f} µs, "
              f"Mul => {dt_mul*1e6:.1f} µs, OK")

def main():
    # Try 32-bit limbs
    demo_binary_limb_arithmetic(chunk_bits=32, test_bits=512, trials=5)
    # Try 64-bit limbs
    demo_binary_limb_arithmetic(chunk_bits=64, test_bits=512, trials=5)

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import random
import math
import time
from array import array

import matplotlib.pyplot as plt
import seaborn as sns

##################################################
# 1) Short HPC-limb snippet
##################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    if value < 0:
        raise ValueError("No negative in this snippet. Store sign separately if needed.")
    limbs = array('Q')
    while value > 0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q', [0]*(out_len+1))
    carry = 0
    for i in range(out_len):
        av = A[i] if i < len(A) else 0
        bv = B[i] if i < len(B) else 0
        s_val = av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry = s_val >> CHUNK_BITS
    if carry:
        out[out_len] = carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    # assume A >= B
    out_len = max(len(A), len(B))
    out = array('Q', [0]*out_len)
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff = av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry = 1
        else:
            carry = 0
        out[i] = diff & CHUNK_MASK
    # remove trailing zero limbs
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array) -> int:
    # compare A,B => 1(A>B),0(=),-1(A<B)
    la, lb = len(A), len(B)
    if la>lb:
        if A[-1]!=0: return 1
    elif lb>la:
        if B[-1]!=0: return -1
    top = max(la, lb)
    for i in range(top-1, -1, -1):
        av = A[i] if i<la else 0
        bv = B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits: int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out = array('Q', A)
    limb_shifts = shift_bits // CHUNK_BITS
    bit_shifts  = shift_bits % CHUNK_BITS
    if limb_shifts>=len(out):
        return array('Q',[0])
    out = out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = (out[i]) | (carry<<CHUNK_BITS)
        out[i] = (cur >> bit_shifts) & CHUNK_MASK
        carry = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

##################################################
# HPCWave_to_mV => interpret wave_state as fraction
##################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    """Map HPC-limb wave_state => float in [rest_mV..rest_mV+range_mV]."""
    val = limbs_to_int(wave_limb)
    max_val = limbs_to_int(wave_max)
    fraction = val/max_val if max_val>0 else 0.0
    return rest_mV + fraction*range_mV

##################################################
# 2) HPCWaveSegment
##################################################

class HPCWaveSegment:
    def __init__(self, wave_max_val=1_000_000):
        # We'll define wave_max=1,000,000 => top means fraction=1.0 => -30mV
        self.wave_max = int_to_limbs(1_000_000)
        # start ~ 5% of wave_max
        start_val = int(wave_max_val * 0.05)
        # wave_state ~1% => 10k
        self.wave_state = int_to_limbs(start_val)

        # if you want exactly -30 => 0.364, -35 => 0.318. Let's pick:
        k_f = 0.40    # ~ -30 mV
        ca_f = 0.35   # ~ -35 mV
        k_thr = int(wave_max_val * k_f)
        ca_thr = int(wave_max_val * ca_f)
        # Ca threshold ~2% => 20k
        self.ca_threshold = int_to_limbs(ca_thr)
        # K threshold ~3% => 30k
        self.k_threshold = int_to_limbs(k_thr)

        self.toggle_history = []
        self.conduction_amplitude_history = []

    def __repr__(self):
        w_int = limbs_to_int(self.wave_state)
        return f"<HPCWaveSegment wave_state={w_int}>"

##################################################
# 3) HPC wave conduction & channels
##################################################

def hpc_binary_propagate(segments, conduction_delay=1):
    n = len(segments)
    zero_limb = int_to_limbs(0)
    toggles   = [zero_limb]*n
    conduction_list = [zero_limb]*n

    for i, seg in enumerate(segments):
        # wave_state//8 => HPC shift?
        # simpler approach: do hpc_shr(seg.wave_state, 3)
        conduction_val = hpc_shr(seg.wave_state, 3)
        conduction_list[i] = conduction_val

        # pass toggles
        if hpc_compare(conduction_val, zero_limb)>0:
            if i>0:
                toggles[i-1]= conduction_val
            if i<n-1:
                toggles[i+1]= conduction_val

    # store conduction amplitude & apply conduction delay
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_list[i])
        if len(seg.toggle_history)>=conduction_delay:
            add_val = seg.toggle_history.pop(0)
            seg.wave_state = hpc_add(seg.wave_state, add_val)
        seg.toggle_history.append(toggles[i])

def hpc_ion_channels(seg: HPCWaveSegment):
    # if wave_state>ca_threshold => wave_state += wave_state//16
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        # HPC wave_state//16 => shift by 4 bits?
        plus_val = hpc_shr(seg.wave_state, 4)
        seg.wave_state = hpc_add(seg.wave_state, plus_val)

    # if wave_state>k_threshold => wave_state -= wave_state//8
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val = hpc_shr(seg.wave_state, 3)
        # HPC sub
        seg.wave_state = hpc_sub(seg.wave_state, minus_val)

def partial_reflection_hpc(segments):
    # wave_state -= wave_state//16 => reflection
    zero_limb = int_to_limbs(0)
    if not segments:
        return
    left_seg  = segments[0]
    right_seg = segments[-1]
    # left
    subL = hpc_shr(left_seg.wave_state, 4)
    left_seg.wave_state = hpc_sub(left_seg.wave_state, subL)
    # right
    subR = hpc_shr(right_seg.wave_state, 4)
    right_seg.wave_state= hpc_sub(right_seg.wave_state, subR)

def update_hpc_wave(segments):
    hpc_binary_propagate(segments, conduction_delay=1)
    for seg in segments:
        hpc_ion_channels(seg)
    partial_reflection_hpc(segments)

##################################################
# 4) HPC wave simulation & plotting
##################################################

def run_hpc_wave(num_segments=10, steps=20):
    segs = [HPCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg[0] => wave_state=20000 => 2%
    segs[0].wave_state = int_to_limbs(20_000)
    history=[]
    for step in range(steps):
        wave_snapshot = []
        for s in segs:
            wave_snapshot.append(array('Q', s.wave_state))  # copy
        history.append(wave_snapshot)
        update_hpc_wave(segs)
    # final
    history.append([array('Q', s.wave_state) for s in segs])
    return segs, history

def plot_hpc_results(segs, history):
    steps_plus = len(history)
    num_segments= len(segs)
    wave_float = []
    for t in range(steps_plus):
        rowf=[]
        for i, seg_obj in enumerate(segs):
            w_limb = history[t][i]
            # HPC -> mV
            v_mV = HPCWave_to_mV(w_limb, seg_obj.wave_max, range_mV=40.0, rest_mV=-70.0)
            rowf.append(v_mV)
        wave_float.append(rowf)

    conduction_arrays = []
    max_amp_steps = 0
    for seg in segs:
        max_amp_steps= max(max_amp_steps, len(seg.conduction_amplitude_history))
    for step_idx in range(max_amp_steps):
        rowC=[]
        for seg in segs:
            if step_idx<len(seg.conduction_amplitude_history):
                c_limb = seg.conduction_amplitude_history[step_idx]
                rowC.append(limbs_to_int(c_limb))
            else:
                rowC.append(0)
        conduction_arrays.append(rowC)

    import numpy as np
    wave_np = np.array(wave_float)
    cond_np = np.array(conduction_arrays)

    fig, axes = plt.subplots(3,1, figsize=(12,12))

    # 1) HPC wave line
    ax1=axes[0]
    time_axis=np.arange(steps_plus)
    for seg_i in range(num_segments):
        ax1.plot(time_axis, wave_np[:,seg_i], label=f"Seg {seg_i}")
    ax1.set_title("HPC-limb wave in Membrane Potential (mV approx.)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax1.grid(True)

    # 2) final distribution
    ax2=axes[1]
    final_vals= wave_np[-1,:]
    ax2.hist(final_vals, bins=20, color='orange', alpha=0.7)
    ax2.set_title("Final HPC Wave States Distribution (mV)")
    ax2.set_xlabel("Voltage (mV)")
    ax2.set_ylabel("Count")
    ax2.grid(True)

    # 3) conduction amplitude line
    ax3=axes[2]
    t_c= np.arange(len(cond_np))
    for seg_i in range(num_segments):
        ax3.plot(t_c, cond_np[:,seg_i], label=f"Seg {seg_i}")
    ax3.set_title("Conduction Strength Over Time (64-bit HPC-limb ints)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude (int)")
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax3.grid(True)

    plt.tight_layout()
    return fig

def main():
    segs, hist= run_hpc_wave(num_segments=8, steps=25)
    fig= plot_hpc_results(segs, hist)
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import time
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from array import array

#######################################
# 1) HPC-limb code (add, sub, shr, etc.)
#######################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    if value < 0:
        raise ValueError("This snippet does not handle negative ints. Store sign separately.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q', [0]*(out_len+1))
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val = av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry = s_val >> CHUNK_BITS
    if carry:
        out[out_len] = carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    # Assume A>=B
    out_len = max(len(A), len(B))
    out = array('Q', [0]*out_len)
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff = av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry = 1
        else:
            carry = 0
        out[i] = diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array) -> int:
    la, lb = len(A), len(B)
    if la>lb:
        if A[-1]!=0: return 1
    elif lb>la:
        if B[-1]!=0: return -1
    top = max(la, lb)
    for i in range(top-1, -1, -1):
        av = A[i] if i<la else 0
        bv = B[i] if i<lb else 0
        if av>bv: return 1
        elif av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits: int) -> array:
    """Right shift HPC-limb array by shift_bits bits total."""
    if shift_bits<=0:
        return array('Q', A)
    out = array('Q', A)
    limb_shifts = shift_bits//CHUNK_BITS
    bit_shifts  = shift_bits%CHUNK_BITS
    if limb_shifts>=len(out):
        return array('Q',[0])
    out = out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur >> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

def HPCwave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    """Interpret HPC wave_limb as fraction of wave_max => map [-70..-30]."""
    val = limbs_to_int(wave_limb)
    mv  = limbs_to_int(wave_max)
    fraction = val/mv if mv>0 else 0
    return rest_mV + range_mV*fraction

#######################################
# 2) HPC Wave Model
#######################################

class HPCWaveSegment:
    def __init__(self):
        self.wave_max = int_to_limbs(1_000_000)  # if wave_state=1e6 => fraction=1 => -30mV
        # wave_state ~1% => 10k
        self.wave_state= int_to_limbs(10_000)

        # thresholds: Ca=20k(2%), K=30k(3%)
        self.ca_threshold= int_to_limbs(20_000)
        self.k_threshold= int_to_limbs(30_000)

        self.toggle_history = []
        self.conduction_amplitude_history = []

def hpc_binary_propagate(segments, conduction_delay=1):
    n = len(segments)
    zeroL = int_to_limbs(0)
    toggles = [zeroL]*n
    conduction_list=[zeroL]*n

    for i, seg in enumerate(segments):
        # conduction => wave_state//8 => shift by 3
        conduction_val = hpc_shr(seg.wave_state, 3)
        conduction_list[i] = conduction_val
        if hpc_compare(conduction_val, zeroL)>0:
            if i>0:
                toggles[i-1]= conduction_val
            if i<n-1:
                toggles[i+1]= conduction_val

    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_list[i])
        if len(seg.toggle_history)>=conduction_delay:
            add_val = seg.toggle_history.pop(0)
            seg.wave_state = hpc_add(seg.wave_state, add_val)
        seg.toggle_history.append(toggles[i])

def hpc_ion_channels(seg: HPCWaveSegment):
    # if wave>ca => wave += wave//16
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4)
        seg.wave_state= hpc_add(seg.wave_state, plus_val)
    # if wave>k => wave -= wave//8
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3)
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def partial_reflection_hpc(segments):
    if not segments:
        return
    # wave -= wave//16 => boundary reflection
    subH= hpc_shr(segments[0].wave_state,4)
    segments[0].wave_state= hpc_sub(segments[0].wave_state, subH)

    subR= hpc_shr(segments[-1].wave_state,4)
    segments[-1].wave_state= hpc_sub(segments[-1].wave_state, subR)

def update_hpc_wave(segments):
    hpc_binary_propagate(segments, conduction_delay=1)
    for seg in segments:
        hpc_ion_channels(seg)
    partial_reflection_hpc(segments)

def run_hpc_wave(num_segments=8, steps=25):
    segs = [HPCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg0 => wave_state=20000 => 2%
    segs[0].wave_state= int_to_limbs(20_000)
    history=[]
    for step in range(steps):
        # record HPC wave
        wave_snapshot=[array('Q', s.wave_state) for s in segs]
        history.append(wave_snapshot)
        update_hpc_wave(segs)
    # final
    history.append([array('Q', s.wave_state) for s in segs])
    return segs, history

#######################################
# 3) Normal LTC wave Model
#######################################
class LTCWaveSegment:
    """
    Normal LTC wave with float-based approach:
      wave_state in [-70..-30] mV
      conduction = wave_state/8
      thresholds => Ca= -65, K= -60
    """
    def __init__(self):
        # wave_state ~ -69 => near rest
        self.wave_state = -69.0
        self.ca_thresh  = -65.0
        self.k_thresh   = -60.0
        self.toggle_history = []
        self.conduction_amplitude_history= []

def ltc_propagate(segments, conduction_delay=1):
    n = len(segments)
    toggles = [0.0]*n
    conduction_list=[0.0]*n
    for i, seg in enumerate(segments):
        conduction_val= seg.wave_state/8.0
        conduction_list[i]= conduction_val
        if conduction_val>0:
            if i>0:
                toggles[i-1]+= conduction_val
            if i<n-1:
                toggles[i+1]+= conduction_val
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_list[i])
        if len(seg.toggle_history)>=conduction_delay:
            addv= seg.toggle_history.pop(0)
            seg.wave_state+= addv
        seg.toggle_history.append(toggles[i])

def ltc_ion_channels(seg: LTCWaveSegment):
    # if wave>-65 => wave += wave/16 => mild push
    if seg.wave_state> seg.ca_thresh:
        seg.wave_state+= seg.wave_state/16.0
    # if wave>-60 => wave -= wave/8 => mild pull
    if seg.wave_state> seg.k_thresh:
        seg.wave_state-= seg.wave_state/8.0

def partial_reflection_ltc(segments):
    # wave -= wave/16 at boundaries
    if not segments:
        return
    segA= segments[0]
    segA.wave_state-= segA.wave_state/16.0
    segB= segments[-1]
    segB.wave_state-= segB.wave_state/16.0

def update_ltc_wave(segments):
    ltc_propagate(segments, conduction_delay=1)
    for s in segments:
        ltc_ion_channels(s)
    partial_reflection_ltc(segments)

def run_ltc_wave(num_segments=8, steps=25):
    segs= [LTCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg0 => wave_state= -65 => mild spike
    segs[0].wave_state= -65.0
    history=[]
    for step in range(steps):
        wave_snapshot=[ s.wave_state for s in segs ]
        history.append(wave_snapshot)
        update_ltc_wave(segs)
    history.append([ s.wave_state for s in segs])
    return segs, history

#######################################
# 4) Combined Dashboard
#######################################
def main():
    # HPC-limb wave
    start_hpc= time.perf_counter()
    hpc_segs, hpc_hist= run_hpc_wave(num_segments=8, steps=25)
    end_hpc= time.perf_counter()
    hpc_time= (end_hpc- start_hpc)*1e3

    # LTC wave
    start_ltc= time.perf_counter()
    ltc_segs, ltc_hist= run_ltc_wave(num_segments=8, steps=25)
    end_ltc= time.perf_counter()
    ltc_time= (end_ltc- start_ltc)*1e3

    print(f"HPC-limb wave took {hpc_time:.3f} ms total => {hpc_time/25:.3f} ms/step")
    print(f"LTC wave took     {ltc_time:.3f} ms total => {ltc_time/25:.3f} ms/step")

    # HPC => wave_mV
    steps_plus_hpc= len(hpc_hist)
    wave_mV_hpc=[]
    for t in range(steps_plus_hpc):
        rowf=[]
        for seg_i in range(len(hpc_segs)):
            v_mV= HPCwave_to_mV(hpc_hist[t][seg_i], hpc_segs[seg_i].wave_max)
            rowf.append(v_mV)
        wave_mV_hpc.append(rowf)

    # LTC => wave_state is already mV
    steps_plus_ltc= len(ltc_hist)

    # Plot side-by-side
    fig, axes= plt.subplots(2,2, figsize=(14,10))
    ax_hpc= axes[0][0]
    ax_ltc= axes[0][1]
    ax_hpc_dist= axes[1][0]
    ax_ltc_dist= axes[1][1]

    # HPC lines
    hpc_np= np.array(wave_mV_hpc)
    time_axis= np.arange(steps_plus_hpc)
    for seg_i in range(len(hpc_segs)):
        ax_hpc.plot(time_axis, hpc_np[:, seg_i], label=f"Seg {seg_i}")
    ax_hpc.set_title("HPC-limb Wave (mV)")
    ax_hpc.set_xlabel("Time Step")
    ax_hpc.set_ylabel("Voltage (mV)")
    ax_hpc.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax_hpc.grid(True)

    # HPC final distribution
    final_hpc= hpc_np[-1,:]
    ax_hpc_dist.hist(final_hpc, bins=10, color='orange')
    ax_hpc_dist.set_title("Final HPC-limb Distribution")
    ax_hpc_dist.set_xlabel("Voltage (mV)")
    ax_hpc_dist.set_ylabel("Count")
    ax_hpc_dist.grid(True)

    # LTC lines
    ltc_np= np.array(ltc_hist)
    time_axis_ltc= np.arange(steps_plus_ltc)
    for seg_i in range(len(ltc_segs)):
        ax_ltc.plot(time_axis_ltc, ltc_np[:, seg_i], label=f"Seg {seg_i}")
    ax_ltc.set_title("LTC Wave (mV float-based)")
    ax_ltc.set_xlabel("Time Step")
    ax_ltc.set_ylabel("Voltage (mV)")
    ax_ltc.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax_ltc.grid(True)

    # LTC final distribution
    final_ltc= ltc_np[-1,:]
    ax_ltc_dist.hist(final_ltc, bins=10, color='green')
    ax_ltc_dist.set_title("Final LTC Distribution")
    ax_ltc_dist.set_xlabel("Voltage (mV)")
    ax_ltc_dist.set_ylabel("Count")
    ax_ltc_dist.grid(True)

    plt.tight_layout()
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import time
from array import array
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#######################################################
# 1) HPC-limb code: int_to_limbs, add, sub, compare, shift
#######################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """
    Convert a nonnegative Python int -> array of 64-bit limbs (lowest limb first).
    """
    if value < 0:
        raise ValueError("Negative ints not supported in this snippet.")
    limbs = array('Q')  # 'Q' => unsigned long long
    while value > 0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    """
    Combine array of 64-bit limbs -> a single Python int.
    Limb[0] = lowest chunk, limb[-1] = highest chunk.
    """
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    """
    HPC-limb addition: A + B -> out, each 64-bit chunk in 'Q'.
    """
    out_len = max(len(A), len(B))
    out = array('Q', [0]*(out_len+1))
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val = av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry  = s_val >> CHUNK_BITS
    if carry:
        out[out_len] = carry
    else:
        out.pop()  # remove last if unused
    return out

def hpc_sub(A: array, B: array) -> array:
    """
    HPC-limb subtraction: A - B -> out, assume A >= B (nonnegative).
    """
    out_len = max(len(A), len(B))
    out = array('Q', [0]*out_len)
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff = av - bv - carry
        if diff < 0:
            diff += CHUNK_BASE
            carry = 1
        else:
            carry = 0
        out[i] = diff & CHUNK_MASK
    # remove trailing zeros
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array) -> int:
    """
    Compare HPC-limb arrays A,B => 1(A>B), 0(=), -1(A<B).
    No sign, so just compare lengths & top limbs.
    """
    la, lb = len(A), len(B)
    if la>lb:
        if A[-1]!=0: return 1
    elif lb>la:
        if B[-1]!=0: return -1
    top = max(la, lb)
    for i in range(top-1, -1, -1):
        av = A[i] if i<la else 0
        bv = B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits: int) -> array:
    """
    HPC-limb right shift by shift_bits bits total.
    Removes entire limbs first, then partial bit shift.
    """
    if shift_bits<=0:
        return array('Q', A)
    out = array('Q', A)
    limb_shifts = shift_bits // CHUNK_BITS
    bit_shifts  = shift_bits % CHUNK_BITS
    if limb_shifts>=len(out):
        return array('Q',[0])
    out = out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur >> bit_shifts) & CHUNK_MASK
        carry = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

#######################################################
# 2) HPCWave_to_mV => interpret HPC wave-limb as fraction
#######################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    """
    Convert HPC wave_limb => float in [rest_mV..(rest_mV + range_mV)],
    based on fraction= wave_limb / wave_max.
    """
    val = limbs_to_int(wave_limb)
    max_val = limbs_to_int(wave_max)
    fraction= val/max_val if max_val>0 else 0.0
    return rest_mV + range_mV*fraction

#######################################################
# 3) HPC WaveSegment + wave logic
#######################################################

class HPCWaveSegment:
    def __init__(self):
        # wave_max => define "1.0 fraction" => e.g. 1,000,000
        self.wave_max = int_to_limbs(1_000_000)
        # wave_state => ~1% => 10,000
        self.wave_state= int_to_limbs(10_000)
        # threshold => Ca=20k(2%), K=30k(3%)
        self.ca_threshold= int_to_limbs(20_000)
        self.k_threshold= int_to_limbs(30_000)

        self.toggle_history = []
        self.conduction_amplitude_history = []

def hpc_binary_propagate(segments, conduction_delay=1):
    """
    conduction => wave_state//8 => hpc_shr(wave_state,3)
    partial toggles => stored for conduction_delay steps
    """
    zeroL = int_to_limbs(0)
    n = len(segments)
    toggles  = [zeroL]*n
    condvals = [zeroL]*n

    for i, seg in enumerate(segments):
        conduction_val = hpc_shr(seg.wave_state, 3)  # wave_state//8
        condvals[i] = conduction_val
        # pass toggles
        if hpc_compare(conduction_val, zeroL)>0:
            if i>0:
                toggles[i-1]= conduction_val
            if i<n-1:
                toggles[i+1]= conduction_val

    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(condvals[i])
        if len(seg.toggle_history)>= conduction_delay:
            add_val= seg.toggle_history.pop(0)
            seg.wave_state = hpc_add(seg.wave_state, add_val)
        seg.toggle_history.append(toggles[i])

def hpc_ion_channels(seg: HPCWaveSegment):
    """
    Ca push => if wave>ca => wave += wave//16
    K pull => if wave>k => wave -= wave//8
    """
    zeroL= int_to_limbs(0)
    # if wave>ca
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4)  # wave//16
        seg.wave_state= hpc_add(seg.wave_state, plus_val)
    # if wave>k
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3) # wave//8
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def partial_reflection_hpc(segments):
    """At boundaries: wave -= wave//16 => HPC-limb style"""
    if not segments:
        return
    segA, segB = segments[0], segments[-1]
    subA = hpc_shr(segA.wave_state,4)
    segA.wave_state= hpc_sub(segA.wave_state, subA)
    subB = hpc_shr(segB.wave_state,4)
    segB.wave_state= hpc_sub(segB.wave_state, subB)

def update_hpc_wave(segments):
    hpc_binary_propagate(segments, conduction_delay=1)
    for seg in segments:
        hpc_ion_channels(seg)
    partial_reflection_hpc(segments)

def run_hpc_wave(num_segments=8, steps=25):
    # init segments
    segs = [HPCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg0 => wave_state= 20,000 => 2%
    segs[0].wave_state= int_to_limbs(20_000)

    history=[]
    for step in range(steps):
        snap=[array('Q', s.wave_state) for s in segs]  # copy HPC-limbs
        history.append(snap)
        update_hpc_wave(segs)
    # final
    final_snap= [array('Q', s.wave_state) for s in segs]
    history.append(final_snap)
    return segs, history

#######################################################
# 4) Plot HPC wave results
#######################################################

def plot_hpc_results(segs, history):
    steps_plus= len(history)
    num_segments= len(segs)
    wave_mV = []
    for t in range(steps_plus):
        rowf=[]
        for i, seg_obj in enumerate(segs):
            # interpret HPC-limb => fraction => [-70..-30]
            v = HPCWave_to_mV(history[t][i], seg_obj.wave_max, range_mV=40.0, rest_mV=-70.0)
            rowf.append(v)
        wave_mV.append(rowf)
    wave_np= np.array(wave_mV)

    # conduction amplitude
    max_steps= 0
    for s in segs:
        max_steps= max(max_steps, len(s.conduction_amplitude_history))
    conduction_array= []
    for step_idx in range(max_steps):
        rowC=[]
        for seg in segs:
            if step_idx< len(seg.conduction_amplitude_history):
                c_val= seg.conduction_amplitude_history[step_idx]
                c_int= limbs_to_int(c_val)
                rowC.append(c_int)
            else:
                rowC.append(0)
        conduction_array.append(rowC)
    cond_np= np.array(conduction_array)

    fig, axes= plt.subplots(3,1, figsize=(12,12))

    # 1) HPC wave line
    ax1=axes[0]
    time_axis= np.arange(steps_plus)
    for seg_i in range(num_segments):
        ax1.plot(time_axis, wave_np[:, seg_i], label=f"Seg {seg_i}")
    ax1.set_title("HPC-limb wave (mV approx.)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax1.grid(True)

    # 2) final wave distribution
    ax2=axes[1]
    final_vals= wave_np[-1,:]
    ax2.hist(final_vals, bins=20, color='orange', alpha=0.7)
    ax2.set_title("Final HPC-limb wave distribution (mV)")
    ax2.set_xlabel("Voltage (mV)")
    ax2.set_ylabel("Count")
    ax2.grid(True)

    # 3) conduction amplitude over time
    ax3=axes[2]
    c_t= np.arange(len(cond_np))
    for seg_i in range(num_segments):
        ax3.plot(c_t, cond_np[:, seg_i], label=f"Seg {seg_i}")
    ax3.set_title("Conduction amplitude (HPC-limb integers)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude (int)")
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax3.grid(True)

    plt.tight_layout()
    return fig

#######################################################
# 5) main
#######################################################

def main():
    # run HPC-limb wave
    start_t= time.perf_counter()
    segs, hist= run_hpc_wave(num_segments=8, steps=25)
    end_t= time.perf_counter()
    total_ms= (end_t- start_t)*1e3
    print(f"HPC-limb wave took {total_ms:.3f} ms total => {total_ms/25:.3f} ms/step")

    # Plot
    fig= plot_hpc_results(segs, hist)
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
def hpc_limb_to_mV(wave_limb, max_limb, range_mV=40.0, rest_mV=-70.0):
    """
    Convert HPC-limb wave_limb => float in the range [rest_mV..(rest_mV+range_mV)]
    by fraction = wave_limb / max_limb, then map fraction => [-70..-30] etc.
    """
    val = limbs_to_int(wave_limb)
    max_val = limbs_to_int(max_limb)
    fraction = val / max_val if max_val > 0 else 0.0
    return rest_mV + range_mV * fraction

def plot_biological_characteristics(history, segs):
    """
    Enhanced plotting with C. elegans-like features, but HPC-limb aware:
      - HPC-limb wave states stored in history[t][seg_idx]
      - HPC-limb wave_max stored in segs[seg_idx].wave_max
    """
    fig, axes = plt.subplots(4, 1, figsize=(15, 12))
    
    time_axis = np.arange(len(history))
    num_segs  = len(segs)

    # 1) Membrane potential with biological ranges
    ax1 = axes[0]
    for seg_idx in range(num_segs):
        # HPC-limb -> float (mV)
        v_trace = [
            hpc_limb_to_mV(
                h[seg_idx],        # HPC-limb wave state
                segs[seg_idx].wave_max,  # HPC-limb max
                range_mV=40.0,     # typical Ca spike amplitude
                rest_mV=-70.0
            )
            for h in history
        ]
        ax1.plot(time_axis, v_trace, label=f"Seg {seg_idx}")
    ax1.set_title("Membrane Potential (C. elegans Range, HPC-limb based)")
    ax1.set_ylabel("Voltage (mV)")
    ax1.set_ylim([-72, -30])  # Biological range
    ax1.grid(True)
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 2) Calcium dynamics
    # For demonstration, we interpret the wave state's integer value,
    # then "extract" the lower 12 bits as a pseudo "Ca channel" measure.
    # HPC-limb => convert to int, do & 0xFFF
    ax2 = axes[1]
    for seg_idx in range(num_segs):
        ca_trace = []
        for h in history:
            val = limbs_to_int(h[seg_idx])
            ca_trace.append(val & 0xFFF)  # lower 12 bits => "Ca2+"
        ax2.plot(time_axis, ca_trace, label=f"Seg {seg_idx}")
    ax2.set_title("Calcium Activity (UNC-2 Channel, HPC-limb bits)")
    ax2.set_ylabel("Ca2+ Activity (a.u.)")
    ax2.grid(True)
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 3) Potassium dynamics
    # Similarly, bits [12..23] => "K channel"
    ax3 = axes[2]
    for seg_idx in range(num_segs):
        k_trace = []
        for h in history:
            val = limbs_to_int(h[seg_idx])
            # shift right 12, & 0xFFF
            k_trace.append((val >> 12) & 0xFFF)
        ax3.plot(time_axis, k_trace, label=f"Seg {seg_idx}")
    ax3.set_title("Potassium Activity (EXP-2 Channel, HPC-limb bits)")
    ax3.set_ylabel("K+ Activity (a.u.)")
    ax3.grid(True)
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 4) Wave propagation velocity
    # We'll define "velocity" as the derivative of the mean HPC-limb integer wave_state across segments
    ax4 = axes[3]
    mean_vals = []
    for h in history:
        # average HPC-limb integer across all segs
        sum_val = 0
        for seg_idx in range(num_segs):
            sum_val += limbs_to_int(h[seg_idx])
        mean_val = sum_val / num_segs
        mean_vals.append(mean_val)
    # velocity => diff of mean_vals
    prop_vel = np.diff(mean_vals)
    ax4.plot(time_axis[1:], prop_vel)
    ax4.set_title("Wave Propagation Velocity (Δ of mean HPC-limb states)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Velocity (states/step)")
    ax4.grid(True)

    plt.tight_layout()
    return fig

if __name__ == "__main__":
    # Run HPC-limb wave
    segs, hist = run_hpc_wave(num_segments=8, steps=25)
    # Plot with biological characteristics
    fig = plot_biological_characteristics(hist, segs)
    plt.show()

In [ ]:
#!/usr/bin/env python3

import time
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from array import array

###########################################################
# 1) HPC-limb code: minimal add/sub/compare/shift
###########################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    if value < 0:
        raise ValueError("Negative ints not handled here. Store sign separately if needed.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q',[0]*(out_len+1))
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i]= s_val & CHUNK_MASK
        carry = s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    # Assume A>=B
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i]= diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array) -> int:
    la, lb= len(A), len(B)
    if la>lb:
        if A[-1]!=0: return 1
    elif lb>la:
        if B[-1]!=0: return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        elif av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits: int) -> array:
    """Right shift HPC-limb array by shift_bits bits."""
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts = shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out: out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur= out[i] | (carry<<CHUNK_BITS)
        out[i]= (cur >> bit_shifts) & CHUNK_MASK
        carry= cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out: out.append(0)
    return out

###########################################################
# HPC -> mV
###########################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0) -> float:
    val= limbs_to_int(wave_limb)
    mv = limbs_to_int(wave_max)
    fraction= val/mv if mv>0 else 0.0
    return rest_mV + range_mV*fraction

###########################################################
# 2) HPCWaveSegment (normal wave neuron)
###########################################################

class HPCWaveSegment:
    """Regular HPC-limb wave segment"""
    def __init__(self):
        # wave_max => define 1.0 fraction => 1e6
        self.wave_max = int_to_limbs(1_000_000)
        # wave_state => ~1% => 10k
        self.wave_state= int_to_limbs(10_000)
        # Ca threshold => 20k(2%), K => 30k(3%)
        self.ca_threshold= int_to_limbs(20_000)
        self.k_threshold= int_to_limbs(30_000)
        self.toggle_history = []
        self.conduction_amplitude_history= []

def hpc_binary_propagate(segments, conduction_delay=1):
    """
    wave_state//8 => conduction => HPC-limb approach
    partial reflection might happen separately
    """
    zeroL= int_to_limbs(0)
    n= len(segments)
    toggles= [zeroL]*n
    conduction_list=[zeroL]*n
    for i, seg in enumerate(segments):
        conduction_val= hpc_shr(seg.wave_state, 3)  # wave//8
        conduction_list[i]= conduction_val
        if hpc_compare(conduction_val, zeroL)>0:
            if i>0: toggles[i-1]= conduction_val
            if i<n-1: toggles[i+1]= conduction_val
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_list[i])
        if len(seg.toggle_history)>= conduction_delay:
            add_val= seg.toggle_history.pop(0)
            seg.wave_state= hpc_add(seg.wave_state, add_val)
        seg.toggle_history.append(toggles[i])

def hpc_ion_channels(seg: HPCWaveSegment):
    # if wave>ca => wave += wave//16
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4)
        seg.wave_state= hpc_add(seg.wave_state, plus_val)
    # if wave>k => wave -= wave//8
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3)
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def partial_reflection_hpc(segments):
    """At boundaries wave -= wave//16 => HPC-limb style."""
    if not segments:
        return
    segA= segments[0]
    segB= segments[-1]
    subA= hpc_shr(segA.wave_state,4)
    segA.wave_state= hpc_sub(segA.wave_state, subA)
    subB= hpc_shr(segB.wave_state,4)
    segB.wave_state= hpc_sub(segB.wave_state, subB)

def update_hpc_wave(segments):
    hpc_binary_propagate(segments, conduction_delay=1)
    for seg in segments:
        hpc_ion_channels(seg)
    partial_reflection_hpc(segments)

###########################################################
# 3) MotorSegment + motor activation
###########################################################

class MotorSegment(HPCWaveSegment):
    """Motor neuron segment with tension/contraction state"""
    def __init__(self):
        super().__init__()
        # Tension HPC-limb
        self.tension = int_to_limbs(0)
        # Additional threshold for motor activation => 2.5% => 25k
        self.motor_threshold= int_to_limbs(25_000)
        self.force_history= []

def motor_activation(motor_seg: MotorSegment):
    """ Convert wave state => muscle tension:
        if wave>motor_threshold => tension += (wave - threshold)//16
        else => tension decays by tension//8
    """
    if hpc_compare(motor_seg.wave_state, motor_seg.motor_threshold)>0:
        # tension += (wave_state - threshold)//16
        excess= hpc_sub(motor_seg.wave_state, motor_seg.motor_threshold)
        delta= hpc_shr(excess,4)
        motor_seg.tension= hpc_add(motor_seg.tension, delta)
    else:
        # tension decays => tension -= tension//8
        decay= hpc_shr(motor_seg.tension,3)
        # assume tension >= decay
        motor_seg.tension= hpc_sub(motor_seg.tension, decay)

def update_ncp_control(sensory_input: list, motor_segments: list):
    """
    NCP control loop:
      - Sensory modifies wave states
      - HPC wave conduction
      - motor activation
      - store tension
    """
    for i, seg in enumerate(motor_segments):
        # if we have i < len(sensory_input), apply some HPC addition
        if i< len(sensory_input):
            # e.g. wave_state += sensory_input[i]>>3 => mild effect
            mild= hpc_shr(sensory_input[i],3)
            seg.wave_state= hpc_add(seg.wave_state, mild)

        # normal HPC wave step for each motor segment
        # if you want them to wave each individually:
        hpc_binary_propagate([seg], conduction_delay=1)
        hpc_ion_channels(seg)
        motor_activation(seg)

        # store tension
        seg.force_history.append(array('Q', seg.tension))

###########################################################
# 4) Plot motor & HPC states
###########################################################

def plot_motor_characteristics(motor_segments: list):
    """
    Plot motor neuron wave states + tension over time
    """
    fig, axes= plt.subplots(2,1, figsize=(12,8))
    ax1, ax2= axes

    # time => length of force_history or wave state's conduction
    steps= len(motor_segments[0].force_history) if motor_segments else 0
    time_axis= np.arange(steps)

    # 1) HPC wave membrane potential lines
    for i, seg in enumerate(motor_segments):
        v_trace=[]
        # we didn't store wave_state history, so let's approximate from force_history steps
        # If we want wave states at each step, we can store them in update_ncp_control too
        # For demonstration, let's track wave_state now:
        # We'll do a partial approach => each step in force_history => HPCWave_to_mV?
        # We'll store wave states in seg's conduction_amplitude_history or a new seg attribute.
        # For now, assume wave_state didn't change? We'll skip or track the last wave_state
        # We'll just do tension for the second plot. Or if we want wave states, store them too.

        # We'll do a dummy single wave state => HPCWave_to_mV
        wave_val= HPCWave_to_mV(seg.wave_state, seg.wave_max)
        # just plot a horizontal line for demonstration
        v_trace= [wave_val]*steps
        ax1.plot(time_axis, v_trace, label=f"Motor Seg {i}")

    ax1.set_title("Motor Neuron HPC-limb Membrane Potential (approx final)")
    ax1.set_ylabel("Voltage (mV)")
    ax1.grid(True)
    ax1.legend()

    # 2) Tension development
    for i, seg in enumerate(motor_segments):
        t_trace= []
        for f_limb in seg.force_history:
            # HPC-limb => int => tension
            t_int= limbs_to_int(f_limb)
            t_trace.append(t_int)
        ax2.plot(time_axis, t_trace, label=f"Muscle {i}")

    ax2.set_title("Muscle Tension Development (HPC-limb integers)")
    ax2.set_ylabel("Tension (units)")
    ax2.set_xlabel("Time Step")
    ax2.grid(True)
    ax2.legend()

    plt.tight_layout()
    return fig

###########################################################
# 5) Demo main
###########################################################

def run_motor_simulation(num_segments=4, steps=50):
    """Create motor segments, run the control loop with mild sensory input, return them."""
    from random import getrandbits

    # Create motor segments
    motors= [MotorSegment() for _ in range(num_segments)]
    # We'll do a stable sensory input => HPC-limb(15k)
    sensor_val= int_to_limbs(15_000)
    sensory_list= [sensor_val]*(num_segments)  # same input for each motor seg

    for _ in range(steps):
        update_ncp_control(sensory_list, motors)

    return motors

def main():
    # Run motor simulation
    motors= run_motor_simulation(num_segments=3, steps=30)

    # Plot motor results
    fig= plot_motor_characteristics(motors)
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

def animate_worm(segments, history, interval=100):
    """Animate C. elegans-like worm using existing HPC wave data"""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Initialize worm segments as scatter points
    segments_plot = ax.scatter([], [], c=[], cmap='RdBu', s=200, vmin=-70, vmax=-30)
    
    # Add a head marker (bigger point)
    head = ax.scatter([], [], c='k', s=300)
    
    def init():
        ax.set_xlim(-10, 10)
        ax.set_ylim(-5, 5)
        ax.set_title('C. elegans HPC Wave Animation')
        return segments_plot, head
    
    def update(frame):
        # Get x-positions (fixed spacing)
        x = np.linspace(-8, 8, len(segments))
        
        # Calculate y positions based on wave state
        voltages = [HPCWave_to_mV(history[frame][i], segments[i].wave_max) 
                   for i in range(len(segments))]
        
        # Add sinusoidal movement based on voltage
        phase = 2 * np.pi * frame/20
        y = 0.5 * np.sin(x + phase)
        
        # Update segment positions and colors
        segments_plot.set_offsets(np.c_[x, y])
        segments_plot.set_array(np.array(voltages))
        
        # Update head position
        head.set_offsets(np.c_[x[0], y[0]])
        
        return segments_plot, head
    
    anim = FuncAnimation(fig, update, frames=len(history), 
                        init_func=init, interval=interval, 
                        blit=True, repeat=True)
    
    plt.colorbar(segments_plot, label='Membrane Potential (mV)')
    plt.close()  # Prevent double display in Jupyter
    return anim

# Run your existing simulation
segs, hist = run_hpc_wave(num_segments=8, steps=25)

# Create and display animation
anim = animate_worm(segs, hist)
from IPython.display import HTML
HTML(anim.to_jshtml())

In [ ]:
%pip install seaborn

In [ ]:
#!/usr/bin/env python3

import time
import math
import numpy as np
from array import array
import matplotlib.pyplot as plt
import seaborn as sns

############################################################
# 1) HPC-limb code: minimal add/sub/shift for 64-bit array('Q')
############################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    if value < 0:
        raise ValueError("No negative ints in this snippet.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q',[0]*(out_len+1))
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i]= s_val & CHUNK_MASK
        carry = s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    # A>=B
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i]= diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array) -> int:
    la, lb= len(A), len(B)
    if la>lb:
        if A[-1]!=0: return 1
    elif lb>la:
        if B[-1]!=0: return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        elif av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits: int) -> array:
    """Right shift HPC-limb array by shift_bits bits."""
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts = shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out: out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur= out[i] | (carry<<CHUNK_BITS)
        out[i]= (cur >> bit_shifts) & CHUNK_MASK
        carry= cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out: out.append(0)
    return out

def hpc_mul_int(A: array, n: int) -> array:
    """
    Multiply HPC-limb A by a small integer n. For bigger n, you'd do HPC-limb code.
    """
    if n<0:
        raise ValueError("No negative n in this snippet.")
    out = array('Q', A)
    carry=0
    for i in range(len(out)):
        prod= out[i]*n + carry
        out[i]= prod & CHUNK_MASK
        carry= prod >> CHUNK_BITS
    if carry:
        out.append(carry)
    return out

############################################################
# 2) HPCWaveSegment with the new Unified + Combined Equations
############################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0) -> float:
    val= limbs_to_int(wave_limb)
    mv = limbs_to_int(wave_max)
    fraction= val/mv if mv>0 else 0.0
    return rest_mV + fraction*range_mV

class HPCWaveSegment:
    """Now we unify conduction+reflection => P(t), and Ca/K => A(W)."""
    def __init__(self):
        self.wave_max= int_to_limbs(1_000_000)
        self.wave_state= int_to_limbs(10_000)   # ~1%
        # HPC-limb coefficients for alpha, beta, gamma
        # We'll pick alpha=1 => no shift needed
        # Or store them as arrays
        self.alpha= int_to_limbs(1)  # damping => 1 => equals wave_state >> rS
        self.beta=  int_to_limbs(1)  # Ca => 1 => wave_state >> caS
        self.gamma= int_to_limbs(1)  # K  => 1 => wave_state >> kS
        # define shift amounts
        self.cS=3   # conduction shift
        self.rS=4   # reflection shift
        self.caS=4  # Ca shift => wave_state >>4
        self.kS=3   # K shift => wave_state >>3
        self.history= []

def wave_operator_P(W: array, cS: int, rS: int, alpha: array) -> array:
    """
    P(t) = floor(W/2^cS) - alpha * floor(W/2^rS)
    We'll do HPC-limb for conduction & reflection, then multiply reflection by alpha.
    """
    conduction= hpc_shr(W, cS)
    reflect=    hpc_shr(W, rS)
    alpha_ref=  hpc_mul_int(reflect, limbs_to_int(alpha)) # assume alpha is small
    return hpc_sub(conduction, alpha_ref)

def threshold_operator_A(W: array, caS: int, kS: int, beta: array, gamma: array) -> array:
    """
    A(W)= W + beta*(W >> caS) - gamma*(W >> kS)
    """
    # HPC-limb shifts
    exc= hpc_shr(W, caS)
    inh= hpc_shr(W, kS)
    exc_scaled= hpc_mul_int(exc, limbs_to_int(beta))
    inh_scaled= hpc_mul_int(inh, limbs_to_int(gamma))
    # W + exc_scaled - inh_scaled
    tmp= hpc_add(W, exc_scaled)
    return hpc_sub(tmp, inh_scaled)

def update_wave_unified(seg: HPCWaveSegment):
    """
    W(t+1)= W(t) + P(W(t)) + [A(W(t)) - W(t)]
          = P(W(t)) + A(W(t))
       if we treat W(t)+... => or we define a single step approach
       We'll do => wave_state= W + P(W)+[A(W)-W]? etc.
       But let's keep it simpler: wave_state= W(t) + P(t), then wave_state= A(...)
    """
    W0= seg.wave_state
    # 1) P(t)
    p_val= wave_operator_P(W0, seg.cS, seg.rS, seg.alpha)
    # wave_state= W0 + P(t)
    step1= hpc_add(W0, p_val)
    # 2) A(W0) => or we can do A(step1). We'll do A on final
    # It's flexible. The user might want A(W(t)) or A(W(t+1)).
    # We'll do A on step1 => wave_state= step1 + [some portion]
    new_val= threshold_operator_A(step1, seg.caS, seg.kS, seg.beta, seg.gamma)
    seg.wave_state= new_val
    # store in seg.history
    seg.history.append(seg.wave_state)

############################################################
# 3) Demo: run HPC-limb with the new equations
############################################################

def run_unified_sim(num_segments=5, steps=20):
    segs= [HPCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg0 => wave_state=20000 =>2%
    segs[0].wave_state= int_to_limbs(20_000)

    hist_all= []
    for _ in range(steps):
        # store snapshot
        snap=[]
        for s in segs:
            snap.append(array('Q', s.wave_state))
        hist_all.append(snap)

        # update each segment with the new unified eq
        for s in segs:
            update_wave_unified(s)

    # final
    final_snap= [array('Q', s.wave_state) for s in segs]
    hist_all.append(final_snap)
    return segs, hist_all

def plot_unified_results(segs, hist_all):
    steps_plus= len(hist_all)
    num_seg= len(segs)
    wave_mV= []

    for t in range(steps_plus):
        rowf=[]
        for i, seg in enumerate(segs):
            val_mV= HPCWave_to_mV(hist_all[t][i], seg.wave_max, range_mV=40.0, rest_mV=-70.0)
            rowf.append(val_mV)
        wave_mV.append(rowf)

    wave_np= np.array(wave_mV)
    fig, ax= plt.subplots(figsize=(10,6))
    time_axis= np.arange(steps_plus)
    for i in range(num_seg):
        ax.plot(time_axis, wave_np[:, i], label=f"Seg {i}")
    ax.set_title("Unified HPC-Limb Wave (Conduction+Reflection+Threshold in One Eq)")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Voltage (mV approx.)")
    ax.grid(True)
    ax.legend()
    plt.tight_layout()
    return fig

def main():
    segs, hist_all= run_unified_sim(num_segments=5, steps=25)
    fig= plot_unified_results(segs, hist_all)
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from array import array
from typing import Tuple, Union

###############################################################################
# 1) HPC-limb MegaNumber with 16-bit chunks
###############################################################################

class MegaNumber:
    """
    A chunk-based big integer using 16-bit limbs (unsigned short).
    No negative support in the wave logic snippet, though we store a 'negative' flag
    for potential expansions. For large HPC-limb arithmetic only.
    """
    _chunk_code = 'H'       # 16-bit (unsigned short)
    _global_chunk_size = 16  # bits per limb
    _base = 1 << 16          # 65536
    _mask = (1 << 16) - 1    # 65535

    def __init__(
        self,
        mantissa: array = None,
        negative: bool = False,
    ):
        if mantissa is None:
            mantissa = array(self._chunk_code, [0])
        self.mantissa = array(self._chunk_code, mantissa)
        self.negative = negative
        self._normalize()

    @classmethod
    def from_int(cls, val: int) -> "MegaNumber":
        """Convert Python int => HPC-limb big-int with 16-bit chunks."""
        if val < 0:
            raise ValueError("Negatives not handled in this snippet.")
        if val == 0:
            return cls(array(cls._chunk_code, [0]), negative=False)
        out = array(cls._chunk_code)
        tmp = val
        while tmp > 0:
            out.append(tmp & cls._mask)
            tmp >>= cls._global_chunk_size
        return cls(out, negative=False)

    def to_int(self) -> int:
        """Convert HPC-limb => Python int."""
        val = 0
        shift = 0
        for limb in self.mantissa:
            val += (limb << shift)
            shift += self._global_chunk_size
        return val

    def copy(self) -> "MegaNumber":
        return MegaNumber(self.mantissa[:], self.negative)

    def _normalize(self):
        """Strip trailing zero limbs, ensure at least 1 limb remains."""
        while len(self.mantissa) > 1 and self.mantissa[-1] == 0:
            self.mantissa.pop()

    def __repr__(self):
        return f"<MegaNumber limbs={list(self.mantissa)} neg={self.negative}>"

###############################################################################
# 2) HPC-limb arithmetic: add, sub, mul_small, div_small, shift-based pow2
###############################################################################

def hpc_add(A: MegaNumber, B: MegaNumber) -> MegaNumber:
    """A+B HPC-limb, 16-bit chunk-based, ignoring sign."""
    la, lb = len(A.mantissa), len(B.mantissa)
    max_len = max(la, lb)
    out = array(A._chunk_code, [0]*(max_len+1))
    carry = 0
    for i in range(max_len):
        av = A.mantissa[i] if i<la else 0
        bv = B.mantissa[i] if i<lb else 0
        s = av + bv + carry
        out[i] = s & A._mask
        carry  = s >> A._global_chunk_size
    if carry:
        out[max_len] = carry
    else:
        out.pop()
    ret = MegaNumber(out)
    return ret

def hpc_sub(A: MegaNumber, B: MegaNumber) -> MegaNumber:
    """A-B HPC-limb, assume A>=B, ignoring sign."""
    la, lb = len(A.mantissa), len(B.mantissa)
    max_len = max(la, lb)
    out = array(A._chunk_code, [0]*max_len)
    carry=0
    for i in range(max_len):
        av = A.mantissa[i] if i<la else 0
        bv = B.mantissa[i] if i<lb else 0
        diff = av - bv - carry
        if diff<0:
            diff += A._base
            carry=1
        else:
            carry=0
        out[i] = diff & A._mask
    ret = MegaNumber(out)
    ret._normalize()
    return ret

def hpc_mul_small(A: MegaNumber, n: int) -> MegaNumber:
    """HPC-limb multiply by small int n."""
    if n < 0:
        raise ValueError("No negative n in snippet.")
    out = A.copy()
    carry=0
    for i in range(len(out.mantissa)):
        prod = out.mantissa[i]*n + carry
        out.mantissa[i] = prod & A._mask
        carry= prod >> A._global_chunk_size
    if carry:
        out.mantissa.append(carry)
    out._normalize()
    return out

def hpc_div_small(A: MegaNumber, d: int)-> MegaNumber:
    """HPC-limb integer floor divide A by small int d>0."""
    if d<=0:
        raise ZeroDivisionError("d<=0 not allowed.")
    out= array(A._chunk_code, [0]*len(A.mantissa))
    remainder=0
    for i in reversed(range(len(A.mantissa))):
        cur= (remainder<<A._global_chunk_size) + A.mantissa[i]
        q= cur // d
        remainder= cur % d
        out[i]= q & A._mask
    ret= MegaNumber(out)
    ret._normalize()
    return ret

def hpc_div_pow2(A: MegaNumber, bits: int)->MegaNumber:
    """Divide HPC-limb by 2^bits => HPC-limb result."""
    if bits<=0:
        return A.copy()
    divisor= 1<<bits
    return hpc_div_small(A, divisor)

###############################################################################
# 3) HPC-limb fraction for plotting: wave / wave_max => fraction <=1
###############################################################################

def compare_hpc(A: MegaNumber, B: MegaNumber)->int:
    """
    Return -1 if A<B, 0 if A==B, +1 if A>B, ignoring sign.
    """
    la, lb= len(A.mantissa), len(B.mantissa)
    if la>lb:
        return 1
    if la<lb:
        return -1
    for i in reversed(range(la)):
        if A.mantissa[i]>B.mantissa[i]:
            return 1
        elif A.mantissa[i]<B.mantissa[i]:
            return -1
    return 0

def hpc_lshift(A: MegaNumber, bits: int)->MegaNumber:
    """Left shift HPC-limb by bits => multiply by 2^bits."""
    if bits<=0:
        return A.copy()
    # We'll do it with small int mul approach if bits<16:
    # For simplicity, we'll chunk-lshift lumps of 16 bits if needed.
    out= A.copy()
    # lumps
    lumps= bits// A._global_chunk_size
    remain= bits% A._global_chunk_size
    # lumps => push zero limbs in front
    if lumps>0:
        # each lumps => out.mantissa => shift up lumps limbs
        extended= array(A._chunk_code, [0]*lumps)
        extended.extend(out.mantissa)
        out.mantissa= extended
    
    if remain>0:
        carry=0
        for i in range(len(out.mantissa)):
            cur= (out.mantissa[i]<<remain) + carry
            out.mantissa[i]= cur & A._mask
            carry= cur>> A._global_chunk_size
        if carry:
            out.mantissa.append(carry)
    out._normalize()
    return out

def wave_fraction_limb(W: MegaNumber, Wmax: MegaNumber, SHIFT_BITS=32)->float:
    """
    wave_f = ( (W<<SHIFT_BITS) // Wmax ) / 2^SHIFT_BITS
    clamp wave_f => <=1.0
    """
    # if W>=Wmax => fraction=>1
    c = compare_hpc(W, Wmax)
    if c>=0:
        return 1.0
    # W<< SHIFT_BITS => HPC-limb
    scaled_num= hpc_lshift(W, SHIFT_BITS)
    # HPC-limb => HPC-limb => scaled_num // Wmax
    denom_int= Wmax.to_int() # can be big but python handles big int
    if denom_int<=0:
        return 0.0
    scaled_q= hpc_div_small(scaled_num, denom_int)
    # convert scaled_q => python int => float
    scaled_q_int= scaled_q.to_int()
    frac= scaled_q_int/ float(1<<SHIFT_BITS)
    if frac>1.0:
        frac=1.0
    return frac

def hpc_limb_to_mV(W: MegaNumber, Wmax: MegaNumber, range_mV=40.0, rest_mV=-70.0)->float:
    frac= wave_fraction_limb(W, Wmax, SHIFT_BITS=32)
    return rest_mV + range_mV*frac

###############################################################################
# 4) HPC-limb reflection factor => toy "cordic_sin"
###############################################################################

def hpc_cordic_sin(nomega: int)-> MegaNumber:
    """Toy HPC-limb wave => sign flips every half period=512, amplitude=2000."""
    period=1024
    amplitude= MegaNumber.from_int(2000)
    idx= nomega % period
    sign_val=1 if idx<512 else -1
    if sign_val<0:
        zero= MegaNumber.from_int(0)
        return hpc_sub(zero, amplitude)
    else:
        return amplitude

###############################################################################
# 5) HPCWaveSegment
###############################################################################

class HPCWaveSegment:
    def __init__(self):
        self.wave_state= MegaNumber.from_int(20_000)
        self.wave_max=   MegaNumber.from_int(1_000_000)
        self.cS=3
        self.rS=4
        self.caS=4
        self.kS=3
        self.beta=  MegaNumber.from_int(1)
        self.gamma= MegaNumber.from_int(1)
        self.omega=1
        self.history=[]

def update_hpc_segment(seg: HPCWaveSegment, step_n: int):
    """
    HPC-limb formula:
    W_{n+1}= W_n + (W_n>>cS) - alpha(n)*(W_n>>rS)
                   + [ beta*(W_n>>caS) - gamma*(W_n>>kS) ].
    """
    W0= seg.wave_state

    cond=   hpc_div_pow2(W0, seg.cS)
    alpha_n= hpc_cordic_sin(step_n*seg.omega)
    refl=   hpc_div_pow2(W0, seg.rS)
    # multiply HPC-limb by alpha_n's int => alpha_n not huge => to_int() safe
    alpha_int= alpha_n.to_int()  # up to ~2000
    refl_scaled= hpc_mul_small(refl, alpha_int)

    wca=   hpc_div_pow2(W0, seg.caS)
    wca_s= hpc_mul_small(wca, seg.beta.to_int())
    wk=    hpc_div_pow2(W0, seg.kS)
    wk_s=  hpc_mul_small(wk, seg.gamma.to_int())

    tmp1= hpc_add(W0, cond)
    tmp2= hpc_sub(tmp1, refl_scaled)
    tmp3= hpc_add(tmp2, wca_s)
    newval= hpc_sub(tmp3, wk_s)

    seg.wave_state= newval
    seg.history.append(newval)

###############################################################################
# 6) main wave run
###############################################################################

def run_hpc_wave(num_segments=4, steps=25):
    import random
    segs= [HPCWaveSegment() for _ in range(num_segments)]
    # vary freq or initial wave
    for i, s in enumerate(segs):
        s.omega= random.randint(0,2)
        s.wave_state= MegaNumber.from_int( 10_000*(i+1) )
    history=[]
    for stp in range(steps):
        snap=[]
        for s in segs:
            snap.append(s.wave_state.copy())
        history.append(snap)
        for s in segs:
            update_hpc_segment(s, stp)
    final_snap=[]
    for s in segs:
        final_snap.append(s.wave_state.copy())
    history.append(final_snap)
    return segs, history

###############################################################################
# 7) Plot code
###############################################################################

def plot_biological_characteristics(history, segs):
    fig, axes= plt.subplots(4,1, figsize=(15,12))
    time_axis= np.arange(len(history))
    num_segs= len(segs)

    # 1) Membrane potential
    ax1= axes[0]
    for seg_idx in range(num_segs):
        v_trace=[]
        for step_snap in history:
            wv= step_snap[seg_idx]
            val_mV= hpc_limb_to_mV(wv, segs[seg_idx].wave_max, range_mV=40.0, rest_mV=-70.0)
            v_trace.append(val_mV)
        ax1.plot(time_axis, v_trace, label=f"Seg {seg_idx}")
    ax1.set_title("Membrane Potential (C. elegans Range, HPC-limb based)")
    ax1.set_ylabel("Voltage (mV)")
    ax1.set_ylim([-72, -30])
    ax1.grid(True)
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 2) Calcium => bits [0..11]
    ax2= axes[1]
    for seg_idx in range(num_segs):
        ca_trace=[]
        for step_snap in history:
            val_int= step_snap[seg_idx].to_int()
            ca_trace.append(val_int & 0xFFF)
        ax2.plot(time_axis, ca_trace, label=f"Seg {seg_idx}")
    ax2.set_title("Calcium Activity (UNC-2 Channel, HPC-limb bits)")
    ax2.set_ylabel("Ca2+ (a.u.)")
    ax2.grid(True)
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 3) Potassium => bits [12..23]
    ax3= axes[2]
    for seg_idx in range(num_segs):
        k_trace=[]
        for step_snap in history:
            val_int= step_snap[seg_idx].to_int()
            k_trace.append((val_int>>12)&0xFFF)
        ax3.plot(time_axis, k_trace, label=f"Seg {seg_idx}")
    ax3.set_title("Potassium Activity (EXP-2 Channel, HPC-limb bits)")
    ax3.set_ylabel("K+ (a.u.)")
    ax3.grid(True)
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 4) velocity => HPC-limb approach for mean => fraction
    ax4= axes[3]
    mean_vals=[]
    for step_snap in history:
        # sum HPC-limb => float in HPC-limb fraction
        # 1) accumulate HPC-limb sum
        ssum= MegaNumber.from_int(0)
        for seg_idx in range(num_segs):
            ssum= hpc_add(ssum, step_snap[seg_idx])
        # 2) HPC-limb fraction => ssum / num_segs
        # do wave_fraction_limb => but we pass ssum as wave & num_segs as HPC-limb
        segs_limb= MegaNumber.from_int(num_segs)
        frac_f= wave_fraction_limb(ssum, segs_limb, SHIFT_BITS=32)
        mean_vals.append(frac_f)
    # velocity => np.diff
    velocity= np.diff(mean_vals)
    ax4.plot(time_axis[1:], velocity)
    ax4.set_title("Wave Propagation Velocity (Δ HPC-limb mean wave states => fraction)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Velocity (dimensionless frac/step)")
    ax4.grid(True)

    plt.tight_layout()
    return fig

###############################################################################
# 8) main
###############################################################################

if __name__=="__main__":
    segs, hist= run_hpc_wave(num_segments=6, steps=25)
    fig= plot_biological_characteristics(hist, segs)
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

# Configurations for Rhythms and Ion Channels
@dataclass
class RhythmConfig:
    theta_freq: float = 5.0
    gamma_freq: float = 40.0
    beta_freq: float = 20.0
    alpha_freq: float = 10.0
    theta_duty: float = 0.5
    gamma_duty: float = 0.3
    beta_duty: float = 0.4
    alpha_duty: float = 0.45

@dataclass
class IonChannelConfig:
    calcium_freq: float = 12000.0
    sodium_freq: float = 15000.0
    potassium_freq: float = 17000.0
    ion_duty: float = 0.3

# Neural Rhythms
class NeuralRhythms:
    def __init__(self, config: RhythmConfig):
        self.config = config
        self.theta_state = 0
        self.gamma_state = 0
        self.beta_state = 0
        self.alpha_state = 0

    def update(self, t: float):
        self.theta_state = self._generate_wave(t, self.config.theta_freq, self.config.theta_duty)
        self.gamma_state = self._generate_wave(t, self.config.gamma_freq, self.config.gamma_duty)
        self.beta_state = self._generate_wave(t, self.config.beta_freq, self.config.beta_duty)
        self.alpha_state = self._generate_wave(t, self.config.alpha_freq, self.config.alpha_duty)

    def _generate_wave(self, t: float, freq: float, duty: float) -> int:
        phase = (t * freq) % 1.0
        return 1 if phase < duty else 0

# Ion Channels
class IonChannels:
    def __init__(self, config: IonChannelConfig):
        self.config = config
        self.calcium_state = 0
        self.sodium_state = 0
        self.potassium_state = 0

    def update(self, t: float, input_val: int):
        self.calcium_state ^= self._generate_wave(t, self.config.calcium_freq, self.config.ion_duty) & input_val
        self.sodium_state ^= self._generate_wave(t, self.config.sodium_freq, self.config.ion_duty) & input_val
        self.potassium_state ^= self._generate_wave(t, self.config.potassium_freq, self.config.ion_duty) & input_val

    def _generate_wave(self, t: float, freq: float, duty: float) -> int:
        phase = (t * freq) % 1.0
        return 1 if phase < duty else 0

# Combined WaveMemory
class WaveMemoryEnhanced:
    def __init__(self, rhythm_config: RhythmConfig, ion_config: IonChannelConfig):
        self.rhythms = NeuralRhythms(rhythm_config)
        self.ions = IonChannels(ion_config)
        self.combined_state = 0
        self.history = []

    def update(self, t: float, input_val: int):
        self.rhythms.update(t)
        self.ions.update(t, input_val)

        combined_rhythm = self.rhythms.theta_state ^ self.rhythms.gamma_state ^ self.rhythms.beta_state ^ self.rhythms.alpha_state
        combined_ions = self.ions.calcium_state ^ self.ions.sodium_state ^ self.ions.potassium_state
        self.combined_state = combined_rhythm ^ combined_ions

        self.history.append({
            't': t,
            'combined_state': self.combined_state,
            'rhythm': combined_rhythm,
            'ions': combined_ions
        })

# Visualization
def plot_combined_waves(memory: WaveMemoryEnhanced, duration: float, sample_rate: int):
    times = [h['t'] for h in memory.history]
    combined = [h['combined_state'] for h in memory.history]
    plt.figure(figsize=(12, 6))
    plt.plot(times, combined, label="Combined State")
    plt.title("Combined Wave Dynamics (Rhythms + Ion Channels)")
    plt.xlabel("Time (s)")
    plt.ylabel("State")
    plt.legend()
    plt.grid(True)
    plt.show()

# Simulation
if __name__ == "__main__":
    rhythm_config = RhythmConfig()
    ion_config = IonChannelConfig()
    memory = WaveMemoryEnhanced(rhythm_config, ion_config)

    duration = 0.1  # seconds
    sample_rate = 44100
    steps = int(duration * sample_rate)
    for step in range(steps):
        t = step / sample_rate
        input_val = 1 if np.random.rand() < 0.3 else 0
        memory.update(t, input_val)

    plot_combined_waves(memory, duration, sample_rate)

In [ ]:
def plot_enhanced_waves(memory: WaveMemoryEnhanced, duration: float, sample_rate: int):
    """
    Enhanced visualization of combined state with rhythms and ion channels.
    """
    times = [h['t'] for h in memory.history]
    combined = [h['combined_state'] for h in memory.history]
    rhythm = [h['rhythm'] for h in memory.history]
    ions = [h['ions'] for h in memory.history]

    fig, axes = plt.subplots(3, 1, figsize=(15, 12))

    # Plot 1: Combined state
    axes[0].plot(times, combined, label="Combined State", color='black', alpha=0.8)
    axes[0].set_title("Combined State Dynamics (Rhythms + Ion Channels)")
    axes[0].set_ylabel("State")
    axes[0].grid(True)
    axes[0].legend()

    # Plot 2: Neural rhythms
    axes[1].plot(times, rhythm, label="Neural Rhythms", color='blue', alpha=0.6)
    axes[1].set_title("Neural Rhythms Contribution")
    axes[1].set_ylabel("State")
    axes[1].grid(True)
    axes[1].legend()

    # Plot 3: Ion channels
    axes[2].plot(times, ions, label="Ion Channels", color='orange', alpha=0.6)
    axes[2].set_title("Ion Channels Contribution")
    axes[2].set_ylabel("State")
    axes[2].set_xlabel("Time (s)")
    axes[2].grid(True)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Simulation and enhanced plotting
if __name__ == "__main__":
    rhythm_config = RhythmConfig()
    ion_config = IonChannelConfig()
    memory = WaveMemoryEnhanced(rhythm_config, ion_config)

    duration = 0.1  # seconds
    sample_rate = 44100
    steps = int(duration * sample_rate)
    for step in range(steps):
        t = step / sample_rate
        input_val = 1 if np.random.rand() < 0.3 else 0
        memory.update(t, input_val)

    plot_enhanced_waves(memory, duration, sample_rate)

In [ ]:
def plot_enhanced_analog_waves(memory: WaveMemoryEnhanced, duration: float, sample_rate: int):
    """
    Enhanced visualization converting PWM duty cycle states to sinusoidal analog waves.
    """
    times = [h['t'] for h in memory.history]
    combined = [h['combined_state'] for h in memory.history]
    rhythm = [h['rhythm'] for h in memory.history]
    ions = [h['ions'] for h in memory.history]

    # Function to convert digital states into modulated sinusoidal waves
    def pwm_to_sine(times, states, freq, phase=0):
        sine_wave = np.sin(2 * np.pi * freq * np.array(times) + phase)
        modulated = sine_wave * np.array(states)
        return modulated

    # Frequencies for visualization
    freq_rhythm = 20.0  # Example visualization frequency for rhythms
    freq_ions = 100.0   # Example visualization frequency for ion channels
    freq_combined = 10.0  # Example for combined

    # Convert to analog
    analog_combined = pwm_to_sine(times, combined, freq_combined)
    analog_rhythm = pwm_to_sine(times, rhythm, freq_rhythm, phase=np.pi / 4)
    analog_ions = pwm_to_sine(times, ions, freq_ions, phase=np.pi / 2)

    # Plot
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))

    # Plot 1: Combined state (analog)
    axes[0].plot(times, analog_combined, label="Combined State (Analog)", color='black', alpha=0.8)
    axes[0].set_title("Combined State Dynamics (Analog Representation)")
    axes[0].set_ylabel("Amplitude")
    axes[0].grid(True)
    axes[0].legend()

    # Plot 2: Neural rhythms (analog)
    axes[1].plot(times, analog_rhythm, label="Neural Rhythms (Analog)", color='blue', alpha=0.6)
    axes[1].set_title("Neural Rhythms Contribution (Analog)")
    axes[1].set_ylabel("Amplitude")
    axes[1].grid(True)
    axes[1].legend()

    # Plot 3: Ion channels (analog)
    axes[2].plot(times, analog_ions, label="Ion Channels (Analog)", color='orange', alpha=0.6)
    axes[2].set_title("Ion Channels Contribution (Analog)")
    axes[2].set_ylabel("Amplitude")
    axes[2].set_xlabel("Time (s)")
    axes[2].grid(True)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Run the simulation and enhanced plotting
if __name__ == "__main__":
    rhythm_config = RhythmConfig()
    ion_config = IonChannelConfig()
    memory = WaveMemoryEnhanced(rhythm_config, ion_config)

    duration = 0.1  # seconds
    sample_rate = 44100
    steps = int(duration * sample_rate)
    for step in range(steps):
        t = step / sample_rate
        input_val = 1 if np.random.rand() < 0.3 else 0
        memory.update(t, input_val)

    plot_enhanced_analog_waves(memory, duration, sample_rate)

In [ ]:
%pip install scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

def pwm_to_smooth_sine(times, pwm_signal, freq, phase=0, cutoff=50, fs=44100):
    """
    Convert PWM binary signals to a smoothed sine wave using low-pass filtering.
    """
    # Generate sine wave
    sine_wave = np.sin(2 * np.pi * freq * np.array(times) + phase)
    
    # Multiply sine wave with PWM to keep binary transitions
    modulated = sine_wave * pwm_signal
    
    # Apply low-pass Butterworth filter for smooth transition
    b, a = butter(3, cutoff / (0.5 * fs), btype='low')
    smoothed = filtfilt(b, a, modulated)
    return smoothed

def plot_pwm_to_sinusoidal(memory, duration, sample_rate):
    """
    Enhanced PWM to sinusoidal wave visualization.
    """
    times = [h['t'] for h in memory.history]
    combined = [h['combined_state'] for h in memory.history]
    rhythm = [h['rhythm'] for h in memory.history]
    ions = [h['ions'] for h in memory.history]

    # Smoothed analog representations
    smooth_combined = pwm_to_smooth_sine(times, combined, freq=10)
    smooth_rhythm = pwm_to_smooth_sine(times, rhythm, freq=20, phase=np.pi/4)
    smooth_ions = pwm_to_smooth_sine(times, ions, freq=40, phase=np.pi/2)

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))

    # Combined state
    axes[0].plot(times, smooth_combined, label="Combined (Smoothed)", color='black', alpha=0.8)
    axes[0].set_title("Combined State Dynamics (Smoothed Analog)")
    axes[0].set_ylabel("Amplitude")
    axes[0].grid(True)
    axes[0].legend()

    # Neural rhythms
    axes[1].plot(times, smooth_rhythm, label="Neural Rhythms (Smoothed)", color='blue', alpha=0.6)
    axes[1].set_title("Neural Rhythms Contribution (Smoothed)")
    axes[1].set_ylabel("Amplitude")
    axes[1].grid(True)
    axes[1].legend()

    # Ion channels
    axes[2].plot(times, smooth_ions, label="Ion Channels (Smoothed)", color='orange', alpha=0.6)
    axes[2].set_title("Ion Channels Contribution (Smoothed)")
    axes[2].set_ylabel("Amplitude")
    axes[2].set_xlabel("Time (s)")
    axes[2].grid(True)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Run the enhanced simulation and visualization
if __name__ == "__main__":
    rhythm_config = RhythmConfig()
    ion_config = IonChannelConfig()
    memory = WaveMemoryEnhanced(rhythm_config, ion_config)

    duration = 0.1  # seconds
    sample_rate = 44100
    steps = int(duration * sample_rate)
    for step in range(steps):
        t = step / sample_rate
        input_val = 1 if np.random.rand() < 0.3 else 0
        memory.update(t, input_val)

    plot_pwm_to_sinusoidal(memory, duration, sample_rate)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# PWM to Sinusoidal Conversion Function
def pwm_to_sinusoidal(times, pwm_signal, freq, phase=0, cutoff=50, fs=44100):
    """
    Convert PWM binary signal to sinusoidal using low-pass filtering.
    """
    sine_wave = np.sin(2 * np.pi * freq * np.array(times) + phase)
    modulated = sine_wave * pwm_signal  # Modulate sine wave with PWM signal
    b, a = butter(3, cutoff / (0.5 * fs), btype='low')  # Low-pass filter
    return filtfilt(b, a, modulated)  # Apply filter for smoothing

# Generate Combined Wave Representation
def plot_combined_waves(memory, duration, sample_rate):
    """
    Combine all waves (neural rhythms, ion channels, etc.) into a single plot.
    """
    times = [h['t'] for h in memory.history]
    rhythms = [h['rhythm'] for h in memory.history]
    ions = [h['ions'] for h in memory.history]

    # Convert individual components to sinusoidal
    rhythm_sine = pwm_to_sinusoidal(times, rhythms, freq=10, phase=0)
    ion_sine = pwm_to_sinusoidal(times, ions, freq=20, phase=np.pi / 3)

    # Combine waves (weighted sum)
    combined_sine = rhythm_sine + ion_sine

    # Plot results
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))

    ax.plot(times, combined_sine, label="Combined State (Smoothed Analog)", color="black", alpha=0.9)
    ax.plot(times, rhythm_sine, label="Neural Rhythms (Analog)", color="blue", alpha=0.6)
    ax.plot(times, ion_sine, label="Ion Channels (Analog)", color="orange", alpha=0.6)

    ax.set_title("Combined Wave Dynamics (Sinusoidal + Weighted Input Contributions)")
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Amplitude")
    ax.grid(True)
    ax.legend()

    plt.tight_layout()
    plt.show()

# Run the Updated Simulation and Visualization
if __name__ == "__main__":
    rhythm_config = RhythmConfig()
    ion_config = IonChannelConfig()
    memory = WaveMemoryEnhanced(rhythm_config, ion_config)

    duration = 0.1  # seconds
    sample_rate = 44100
    steps = int(duration * sample_rate)

    for step in range(steps):
        t = step / sample_rate
        input_val = 1 if np.random.rand() < 0.3 else 0
        memory.update(t, input_val)

    plot_combined_waves(memory, duration, sample_rate)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Utility Functions
def generate_duty_cycle_wave(times, freq, duty_cycle):
    """Generate binary PWM wave for a given frequency and duty cycle."""
    return np.array([(t % (1 / freq)) < (duty_cycle / freq) for t in times], dtype=int)

def pwm_to_sinusoidal(times, pwm_signal, freq, phase=0, cutoff=50, fs=44100):
    """Convert PWM binary signal to sinusoidal using low-pass filtering."""
    sine_wave = np.sin(2 * np.pi * freq * np.array(times) + phase)
    modulated = sine_wave * pwm_signal  # Modulate sine wave with PWM signal
    b, a = butter(3, cutoff / (0.5 * fs), btype='low')  # Low-pass filter
    return filtfilt(b, a, modulated)  # Apply filter for smoothing

# Simulation Parameters
duration = 0.1  # seconds
sample_rate = 44100  # Hz
times = np.linspace(0, duration, int(duration * sample_rate))

# Ion Channel Gating
calcium_pwm = generate_duty_cycle_wave(times, freq=12_000, duty_cycle=0.3)
sodium_pwm = generate_duty_cycle_wave(times, freq=15_000, duty_cycle=0.3)
potassium_pwm = generate_duty_cycle_wave(times, freq=17_000, duty_cycle=0.3)

# Neural Frequencies
theta_pwm = generate_duty_cycle_wave(times, freq=5, duty_cycle=0.5)
gamma_pwm = generate_duty_cycle_wave(times, freq=40, duty_cycle=0.3)
beta_pwm = generate_duty_cycle_wave(times, freq=20, duty_cycle=0.4)
alpha_pwm = generate_duty_cycle_wave(times, freq=10, duty_cycle=0.45)

# Smoothed Sinusoidal Representations
calcium_sine = pwm_to_sinusoidal(times, calcium_pwm, freq=12_000)
sodium_sine = pwm_to_sinusoidal(times, sodium_pwm, freq=15_000)
potassium_sine = pwm_to_sinusoidal(times, potassium_pwm, freq=17_000)

theta_sine = pwm_to_sinusoidal(times, theta_pwm, freq=5)
gamma_sine = pwm_to_sinusoidal(times, gamma_pwm, freq=40)
beta_sine = pwm_to_sinusoidal(times, beta_pwm, freq=20)
alpha_sine = pwm_to_sinusoidal(times, alpha_pwm, freq=10)

# Combined Ion Channel Dynamics
ion_combined = calcium_sine + sodium_sine + potassium_sine

# Combined Neural Rhythms
rhythm_combined = theta_sine + gamma_sine + beta_sine + alpha_sine

# Reflective Dynamics (Decay)
reflective_dynamics = ion_combined * 0.9  # Example decay factor

# Final Neuron State
final_state = rhythm_combined + ion_combined + reflective_dynamics

# Visualization
fig, axes = plt.subplots(5, 1, figsize=(15, 18))

# Ion Channel Gating
axes[0].plot(times, calcium_sine, label="Calcium (Analog)", color="blue", alpha=0.6)
axes[0].plot(times, sodium_sine, label="Sodium (Analog)", color="orange", alpha=0.6)
axes[0].plot(times, potassium_sine, label="Potassium (Analog)", color="green", alpha=0.6)
axes[0].set_title("Ion Channel Gating (Analog Representation)")
axes[0].legend()
axes[0].grid(True)

# Neural Rhythms
axes[1].plot(times, theta_sine, label="Theta (Analog)", color="purple", alpha=0.6)
axes[1].plot(times, gamma_sine, label="Gamma (Analog)", color="red", alpha=0.6)
axes[1].plot(times, beta_sine, label="Beta (Analog)", color="cyan", alpha=0.6)
axes[1].plot(times, alpha_sine, label="Alpha (Analog)", color="magenta", alpha=0.6)
axes[1].set_title("Neural Rhythms (Analog Representation)")
axes[1].legend()
axes[1].grid(True)

# Reflective Dynamics
axes[2].plot(times, reflective_dynamics, label="Reflective Dynamics (Decay)", color="gray", alpha=0.6)
axes[2].set_title("Reflective Dynamics (Decay)")
axes[2].legend()
axes[2].grid(True)

# Combined Dynamics
axes[3].plot(times, ion_combined, label="Combined Ion Channels", color="orange", alpha=0.6)
axes[3].plot(times, rhythm_combined, label="Combined Neural Rhythms", color="blue", alpha=0.6)
axes[3].set_title("Combined Dynamics (Ion Channels + Neural Rhythms)")
axes[3].legend()
axes[3].grid(True)

# Final Neuron State
axes[4].plot(times, final_state, label="Final Neuron State", color="black", alpha=0.9)
axes[4].set_title("Final Neuron State (Integrated Dynamics)")
axes[4].legend()
axes[4].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Utility Functions
def generate_pwm_wave(times, freq, duty_cycle):
    """Generate binary PWM wave for a given frequency and duty cycle."""
    return np.array([(t % (1 / freq)) < (duty_cycle / freq) for t in times], dtype=int)

def logical_interference(wave1, wave2, operation="XOR"):
    """Combine two binary waves using logical operations."""
    if operation == "XOR":
        return np.logical_xor(wave1, wave2).astype(int)
    elif operation == "AND":
        return np.logical_and(wave1, wave2).astype(int)
    elif operation == "OR":
        return np.logical_or(wave1, wave2).astype(int)
    else:
        raise ValueError("Unsupported operation: choose XOR, AND, or OR.")

def pwm_to_sinusoidal(times, pwm_signal, freq, phase=0, cutoff=50, fs=44100):
    """Convert PWM binary signal to sinusoidal using low-pass filtering."""
    sine_wave = np.sin(2 * np.pi * freq * np.array(times) + phase)
    modulated = sine_wave * pwm_signal  # Modulate sine wave with PWM signal
    b, a = butter(3, cutoff / (0.5 * fs), btype='low')  # Low-pass filter
    return filtfilt(b, a, modulated)  # Apply filter for smoothing

# Simulation Parameters
duration = 0.1  # seconds
sample_rate = 44100  # Hz
times = np.linspace(0, duration, int(duration * sample_rate))

# Ion Channel Gating (PWM Waves)
calcium_pwm = generate_pwm_wave(times, freq=12_000, duty_cycle=0.3)
sodium_pwm = generate_pwm_wave(times, freq=15_000, duty_cycle=0.3)
potassium_pwm = generate_pwm_wave(times, freq=17_000, duty_cycle=0.3)

# Neural Rhythms (PWM Waves)
theta_pwm = generate_pwm_wave(times, freq=5, duty_cycle=0.5)
gamma_pwm = generate_pwm_wave(times, freq=40, duty_cycle=0.3)
beta_pwm = generate_pwm_wave(times, freq=20, duty_cycle=0.4)
alpha_pwm = generate_pwm_wave(times, freq=10, duty_cycle=0.45)

# Logical Interactions (Ion Channels)
ion_combined = logical_interference(calcium_pwm, sodium_pwm, "XOR")
ion_combined = logical_interference(ion_combined, potassium_pwm, "XOR")

# Logical Interactions (Neural Rhythms)
rhythm_combined = logical_interference(theta_pwm, gamma_pwm, "XOR")
rhythm_combined = logical_interference(rhythm_combined, beta_pwm, "XOR")
rhythm_combined = logical_interference(rhythm_combined, alpha_pwm, "XOR")

# Reflective Dynamics
reflective_decay = logical_interference(ion_combined, rhythm_combined, "AND")

# Final Neuron State
final_state = logical_interference(rhythm_combined, ion_combined, "XOR")
final_state = logical_interference(final_state, reflective_decay, "XOR")

# Visualization Conversion (PWM to Sinusoidal)
calcium_sine = pwm_to_sinusoidal(times, calcium_pwm, freq=12_000)
sodium_sine = pwm_to_sinusoidal(times, sodium_pwm, freq=15_000)
potassium_sine = pwm_to_sinusoidal(times, potassium_pwm, freq=17_000)

theta_sine = pwm_to_sinusoidal(times, theta_pwm, freq=5)
gamma_sine = pwm_to_sinusoidal(times, gamma_pwm, freq=40)
beta_sine = pwm_to_sinusoidal(times, beta_pwm, freq=20)
alpha_sine = pwm_to_sinusoidal(times, alpha_pwm, freq=10)

ion_combined_sine = pwm_to_sinusoidal(times, ion_combined, freq=100)  # Aggregate frequency
rhythm_combined_sine = pwm_to_sinusoidal(times, rhythm_combined, freq=100)
reflective_sine = pwm_to_sinusoidal(times, reflective_decay, freq=100)
final_state_sine = pwm_to_sinusoidal(times, final_state, freq=100)

# Visualization
fig, axes = plt.subplots(5, 1, figsize=(15, 18))

# Ion Channel Gating
axes[0].plot(times, calcium_sine, label="Calcium (Analog)", color="blue", alpha=0.6)
axes[0].plot(times, sodium_sine, label="Sodium (Analog)", color="orange", alpha=0.6)
axes[0].plot(times, potassium_sine, label="Potassium (Analog)", color="green", alpha=0.6)
axes[0].set_title("Ion Channel Gating (Sinusoidal Visualization)")
axes[0].legend()
axes[0].grid(True)

# Neural Rhythms
axes[1].plot(times, theta_sine, label="Theta (Analog)", color="purple", alpha=0.6)
axes[1].plot(times, gamma_sine, label="Gamma (Analog)", color="red", alpha=0.6)
axes[1].plot(times, beta_sine, label="Beta (Analog)", color="cyan", alpha=0.6)
axes[1].plot(times, alpha_sine, label="Alpha (Analog)", color="magenta", alpha=0.6)
axes[1].set_title("Neural Rhythms (Sinusoidal Visualization)")
axes[1].legend()
axes[1].grid(True)

# Reflective Dynamics
axes[2].plot(times, reflective_sine, label="Reflective Dynamics (Analog)", color="gray", alpha=0.6)
axes[2].set_title("Reflective Dynamics (Sinusoidal Visualization)")
axes[2].legend()
axes[2].grid(True)

# Combined Dynamics
axes[3].plot(times, ion_combined_sine, label="Combined Ion Channels (Analog)", color="orange", alpha=0.6)
axes[3].plot(times, rhythm_combined_sine, label="Combined Neural Rhythms (Analog)", color="blue", alpha=0.6)
axes[3].set_title("Combined Dynamics (Sinusoidal Visualization)")
axes[3].legend()
axes[3].grid(True)

# Final Neuron State
axes[4].plot(times, final_state_sine, label="Final Neuron State (Analog)", color="black", alpha=0.9)
axes[4].set_title("Final Neuron State (Sinusoidal Visualization)")
axes[4].legend()
axes[4].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from array import array
from scipy.signal import butter, filtfilt
from typing import List

# Constants for HPC-limb
CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

# Utility Functions for HPC-limb
def int_to_limbs(value: int) -> array:
    """Convert an integer to an HPC-limb array."""
    if value < 0:
        raise ValueError("Negative integers not supported.")
    limbs = array('Q')
    while value > 0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    return limbs if limbs else array('Q', [0])

def limbs_to_int(limbs: array) -> int:
    """Convert an HPC-limb array back to an integer."""
    return sum(limb << (CHUNK_BITS * i) for i, limb in enumerate(limbs))

def hpc_add(A: array, B: array) -> array:
    """HPC-limb addition."""
    out = array('Q', [0] * (max(len(A), len(B)) + 1))
    carry = 0
    for i in range(len(out)):
        a_val = A[i] if i < len(A) else 0
        b_val = B[i] if i < len(B) else 0
        sum_val = a_val + b_val + carry
        out[i] = sum_val & CHUNK_MASK
        carry = sum_val >> CHUNK_BITS
    return out if carry else out[:-1]

def hpc_sub(A: array, B: array) -> array:
    """HPC-limb subtraction."""
    out = array('Q', [0] * len(A))
    borrow = 0
    for i in range(len(A)):
        a_val = A[i]
        b_val = B[i] if i < len(B) else 0
        diff = a_val - b_val - borrow
        borrow = 0 if diff >= 0 else 1
        out[i] = diff & CHUNK_MASK if diff >= 0 else diff + CHUNK_BASE
    return out

def hpc_xor(A: array, B: array) -> array:
    """HPC-limb XOR operation."""
    out = array('Q', [A[i] ^ B[i] if i < len(B) else A[i] for i in range(len(A))])
    return out

# Neural Dynamics Classes
class HPCWaveSegment:
    """Represents a single neuron segment with binary wave propagation."""
    def __init__(self):
        self.wave_max = int_to_limbs(1_000_000)
        self.wave_state = int_to_limbs(10_000)
        self.ca_threshold = int_to_limbs(20_000)
        self.k_threshold = int_to_limbs(30_000)
        self.toggle_history = []
        self.conduction_amplitude_history = []

    def apply_ion_gating(self):
        """Ion channel gating logic with bit toggles."""
        if limbs_to_int(self.wave_state) > limbs_to_int(self.ca_threshold):
            self.wave_state = hpc_add(self.wave_state, hpc_shr(self.wave_state, 4))
        if limbs_to_int(self.wave_state) > limbs_to_int(self.k_threshold):
            self.wave_state = hpc_sub(self.wave_state, hpc_shr(self.wave_state, 3))

def hpc_shr(A: array, shift_bits: int) -> array:
    """HPC-limb right shift by a specified number of bits."""
    if shift_bits <= 0:
        return array('Q', A)
    out = array('Q', A)
    for _ in range(shift_bits // CHUNK_BITS):
        out.pop(0) if out else None
    shift_remainder = shift_bits % CHUNK_BITS
    carry = 0
    for i in reversed(range(len(out))):
        current = out[i]
        out[i] = (current >> shift_remainder) | carry
        carry = (current & ((1 << shift_remainder) - 1)) << (CHUNK_BITS - shift_remainder)
    return out

def hpc_binary_propagate(segments: List[HPCWaveSegment], conduction_delay: int = 1):
    """Binary wave propagation across segments."""
    zero_limb = int_to_limbs(0)
    toggles = [zero_limb] * len(segments)

    for i, seg in enumerate(segments):
        conduction = hpc_shr(seg.wave_state, 3)
        seg.conduction_amplitude_history.append(conduction)
        if i > 0:
            toggles[i - 1] = conduction
        if i < len(segments) - 1:
            toggles[i + 1] = conduction

    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            seg.wave_state = hpc_add(seg.wave_state, seg.toggle_history.pop(0))
        seg.toggle_history.append(toggles[i])

# Neural Network with STDP
class HPCNeuralNetwork:
    """Neural network with STDP and wave propagation."""
    def __init__(self, num_segments: int):
        self.segments = [HPCWaveSegment() for _ in range(num_segments)]

    def run_step(self):
        """Run one step of the network."""
        hpc_binary_propagate(self.segments)
        for seg in self.segments:
            seg.apply_ion_gating()

    def visualize(self, steps: int):
        """Visualize wave propagation over a specified number of steps."""
        history = []
        for _ in range(steps):
            self.run_step()
            history.append([limbs_to_int(seg.wave_state) for seg in self.segments])

        # Visualization
        time_axis = np.arange(steps)
        wave_states = np.array(history)

        plt.figure(figsize=(12, 8))
        for i in range(len(self.segments)):
            plt.plot(time_axis, wave_states[:, i], label=f"Segment {i}")
        plt.title("Wave Propagation in HPC Neural Network")
        plt.xlabel("Time Step")
        plt.ylabel("Wave State (HPC-limb integers)")
        plt.legend()
        plt.grid(True)
        plt.show()

# Main Execution
if __name__ == "__main__":
    neural_net = HPCNeuralNetwork(num_segments=8)
    neural_net.visualize(steps=50)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from array import array
from typing import List

# Constants for HPC-limb
CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

# Utility Functions for HPC-limb
def int_to_limbs(value: int) -> array:
    if value < 0:
        raise ValueError("Negative integers not supported.")
    limbs = array('Q')
    while value > 0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    return limbs if limbs else array('Q', [0])

def limbs_to_int(limbs: array) -> int:
    return sum(limb << (CHUNK_BITS * i) for i, limb in enumerate(limbs))

def hpc_add(A: array, B: array) -> array:
    out = array('Q', [0] * (max(len(A), len(B)) + 1))
    carry = 0
    for i in range(len(out)):
        a_val = A[i] if i < len(A) else 0
        b_val = B[i] if i < len(B) else 0
        sum_val = a_val + b_val + carry
        out[i] = sum_val & CHUNK_MASK
        carry = sum_val >> CHUNK_BITS
    return out if carry else out[:-1]

def hpc_sub(A: array, B: array) -> array:
    out = array('Q', [0] * len(A))
    borrow = 0
    for i in range(len(A)):
        a_val = A[i]
        b_val = B[i] if i < len(B) else 0
        diff = a_val - b_val - borrow
        borrow = 0 if diff >= 0 else 1
        out[i] = diff & CHUNK_MASK if diff >= 0 else diff + CHUNK_BASE
    return out

def hpc_xor(A: array, B: array) -> array:
    out = array('Q', [A[i] ^ B[i] if i < len(B) else A[i] for i in range(len(A))])
    return out

def hpc_shr(A: array, shift_bits: int) -> array:
    if shift_bits <= 0:
        return array('Q', A)
    out = array('Q', A)
    for _ in range(shift_bits // CHUNK_BITS):
        out.pop(0) if out else None
    shift_remainder = shift_bits % CHUNK_BITS
    carry = 0
    for i in reversed(range(len(out))):
        current = out[i]
        out[i] = (current >> shift_remainder) | carry
        carry = (current & ((1 << shift_remainder) - 1)) << (CHUNK_BITS - shift_remainder)
    return out

# Neural Dynamics Classes
class HPCWaveSegment:
    def __init__(self):
        self.wave_max = int_to_limbs(1_000_000)
        self.wave_state = int_to_limbs(10_000)
        self.ca_threshold = int_to_limbs(20_000)
        self.k_threshold = int_to_limbs(30_000)
        self.toggle_history = []
        self.conduction_amplitude_history = []
        self.reflective_toggles = []  # History of reflective dynamics

    def apply_ion_gating(self):
        if limbs_to_int(self.wave_state) > limbs_to_int(self.ca_threshold):
            self.wave_state = hpc_add(self.wave_state, hpc_shr(self.wave_state, 4))
        if limbs_to_int(self.wave_state) > limbs_to_int(self.k_threshold):
            self.wave_state = hpc_sub(self.wave_state, hpc_shr(self.wave_state, 3))

    def apply_reflective_decay(self):
        if self.reflective_toggles:
            decay_value = self.reflective_toggles.pop(0)
            self.wave_state = hpc_sub(self.wave_state, decay_value)

def hpc_binary_propagate(segments: List[HPCWaveSegment], conduction_delay: int = 1):
    zero_limb = int_to_limbs(0)
    toggles = [zero_limb] * len(segments)

    for i, seg in enumerate(segments):
        conduction = hpc_shr(seg.wave_state, 3)
        seg.conduction_amplitude_history.append(conduction)
        if i > 0:
            toggles[i - 1] = conduction
        if i < len(segments) - 1:
            toggles[i + 1] = conduction

    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            seg.wave_state = hpc_add(seg.wave_state, seg.toggle_history.pop(0))
        seg.toggle_history.append(toggles[i])
        seg.reflective_toggles.append(hpc_shr(seg.wave_state, 5))  # Add decay

# Neural Network with Reflective Decay
class HPCNeuralNetwork:
    def __init__(self, num_segments: int, layers: int = 1):
        self.layers = [[HPCWaveSegment() for _ in range(num_segments)] for _ in range(layers)]

    def run_step(self):
        for layer in self.layers:
            hpc_binary_propagate(layer)
            for seg in layer:
                seg.apply_ion_gating()
                seg.apply_reflective_decay()

    def visualize(self, steps: int):
        history = []
        for _ in range(steps):
            self.run_step()
            history.append([[limbs_to_int(seg.wave_state) for seg in layer] for layer in self.layers])

        # Visualization
        time_axis = np.arange(steps)
        fig, axes = plt.subplots(len(self.layers), 1, figsize=(12, 6 * len(self.layers)))

        for layer_idx, layer_history in enumerate(zip(*history)):
            wave_states = np.array(layer_history)
            for seg_idx in range(len(self.layers[layer_idx])):
                axes[layer_idx].plot(time_axis, wave_states[:, seg_idx], label=f"Segment {seg_idx}")
            axes[layer_idx].set_title(f"Wave Propagation in Layer {layer_idx + 1}")
            axes[layer_idx].set_xlabel("Time Step")
            axes[layer_idx].set_ylabel("Wave State (HPC-limb integers)")
            axes[layer_idx].legend()
            axes[layer_idx].grid(True)

        plt.tight_layout()
        plt.show()

# Main Execution
if __name__ == "__main__":
    neural_net = HPCNeuralNetwork(num_segments=8, layers=2)  # Two-layer network
    neural_net.visualize(steps=50)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_pcm_waves(
        sample_rate=44100, 
        duration=1.0,
        freqs=(6.0, 10.0, 20.0, 40.0),    # Theta, Alpha, Beta, Gamma
        amplitudes=(0.25, 0.3, 0.35, 0.4) # relative wave amplitudes
    ):
    """
    Generate a multi-wave PCM signal by summing sine waves
    at given frequencies and amplitudes, returning a 16-bit PCM array.
    
    :param sample_rate: sampling rate in Hz
    :param duration: total signal duration in seconds
    :param freqs: tuple of wave frequencies in Hz
    :param amplitudes: tuple of relative amplitudes (0..1)
    :return: combined np.array (int16) of PCM samples
    """
    # Time axis
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    # Use 16-bit PCM range
    max_val = 32767  # for signed int16

    # Initialize 32-bit accumulator to avoid overflow during summation
    combined = np.zeros_like(t, dtype=np.int32)

    for f, amp in zip(freqs, amplitudes):
        # Sine wave in float, scaled by amplitude
        wave_float = amp * np.sin(2 * np.pi * f * t)

        # Convert to int16 range
        wave_int = np.int16(np.round(max_val * wave_float))

        # Accumulate in 32-bit
        combined += wave_int

    # Now clip to [-32768..32767] to avoid overflow
    combined = np.clip(combined, -32768, 32767)

    # Finally convert to int16
    return combined.astype(np.int16), t

# Example usage / plotting
if __name__ == "__main__":
    sr = 44100
    dur = 1.0
    # Theta, Alpha, Beta, Gamma
    freqs = (6.0, 10.0, 20.0, 40.0)
    # Arbitrary amplitude factors
    amps = (0.25, 0.3, 0.35, 0.4)

    pcm_signal, t = generate_pcm_waves(
        sample_rate=sr, 
        duration=dur,
        freqs=freqs,
        amplitudes=amps
    )

    # Plot the result
    plt.figure(figsize=(10,4))
    plt.plot(t[:1000], pcm_signal[:1000], label="PCM Interference (first 1000 samples)")
    plt.title("PCM Wave Interference from Multi-Sinusoids")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (int16)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [10]:
def print_hpc_debug_table(step_input_history, output_history, segs, max_steps=10):
    """
    Print a table of HPC-limb input/output for each step (up to max_steps).
    Also optionally print segment HPC-limb weights if relevant.

    :param step_input_history: list of HPC-limb inputs at each step
    :param output_history: list of HPC-limb outputs from the last neuron
    :param segs: the HPCNeuronSegment objects (to show weights)
    :param max_steps: how many lines to print
    """
    # We'll show up to max_steps lines or the entire length if smaller
    n = min(len(step_input_history), max_steps)

    print("\n=== HPC Debug Table (first", n, "steps) ===")
    print("Step | Input(10)    Input(HPC)            -> Output(10)    Output(HPC)        Weights...")

    for i in range(n):
        inp_mn = step_input_history[i]
        out_mn = output_history[i]

        # Convert HPC-limb => python int => decimal
        inp_int = inp_mn.to_int(inp_mn)
        out_int = out_mn.to_int(out_mn)

        # For printing HPC-limb, we can do a short decimal string
        inp_str = inp_mn.to_decimal_string(20)
        out_str = out_mn.to_decimal_string(20)

        # Also show each segment's HPC-limb weight if you like
        weight_strs = []
        for s in segs:
            w_int = s.weight.to_int(s.weight)
            weight_strs.append(str(w_int))

        # Construct line
        print(f"{i:4d} | {inp_int:12d}  {inp_str:24s} -> {out_int:12d}  {out_str:24s}  {weight_strs}")


In [ ]:
import math
import matplotlib.pyplot as plt
from improvedmathv2 import MegaNumber

def hpc_pcm_demo(sample_rate=4410, duration=1.0):
    """
    Demo: Generate 4 sine waves (theta, alpha, beta, gamma),
    store each sample amplitude in HPC-limb form, sum them, then plot.

    sample_rate: samples per second (lowered from 44100 for brevity)
    duration: total seconds
    """
    # Frequencies (Hz) & scale factors
    freqs = [6.0, 10.0, 20.0, 40.0]       # theta, alpha, beta, gamma
    amps  = [0.25, 0.3, 0.35, 0.4]        # amplitude scale factors
    N = int(sample_rate * duration)       # total samples

    import numpy as np
    time_array = np.linspace(0, duration, N, endpoint=False)

    # We will store the HPC-limb results for the "combined wave" in a list
    # Each entry is a MegaNumber representing amplitude at that sample.
    from_meganum = []  # HPC-limb values for the final sum
    zero_mn = MegaNumber.from_int(0)

    for i in range(N):
        # Sum HPC-limb style
        # Start from HPC-limb zero
        sample_sum = zero_mn.copy()

        # For each wave, compute float amplitude => HPC-limb => add
        t = time_array[i]
        for f, a in zip(freqs, amps):
            # float amplitude => e.g. range ~ [-32767..32767] if you like
            val_float = a * math.sin(2*math.pi*f*t) 
            # Scale to e.g. 32767 for a typical PCM-ish range
            val_scaled = val_float * 32767
            # Round and clamp to int in normal Python
            val_int = int(round(val_scaled))
            # HPC-limb from that int
            val_mn = MegaNumber.from_int(abs(val_int))
            if val_int < 0:
                # If negative, in HPC-limb snippet we didn't fully do negative,
                # so we do a zero - val_mn approach
                # i.e. sample_sum = sample_sum - val_mn
                sample_sum = sample_sum.sub(val_mn)
            else:
                # add 
                sample_sum = sample_sum.add(val_mn)

        # Store HPC-limb result
        from_meganum.append(sample_sum)
    
    # Now we have HPC-limb wave array "from_meganum".
    # Let's convert each HPC-limb amplitude into a float for plotting.

    float_wave = []
    for mn in from_meganum:
        # Convert HPC-limb => decimal string => float
        # (We could parse directly to int, but let's follow a "string" approach.)
        dec_str = mn.to_decimal_string()   # e.g. "12345" or "12345 * 2^-7"
        
        # We can parse if there's a "* 2^X" part
        #  naive approach: 
        if '* 2^' in dec_str:
            base_str, pow_part = dec_str.split('* 2^')
            base_str = base_str.strip()
            pow_part = pow_part.strip()
            # convert base_str => int, then shift
            if base_str.startswith('-'):
                neg = True
                base_str = base_str[1:].strip()
            else:
                neg = False
            if base_str.startswith('...'):
                # truncated => can't parse easily 
                # let's fallback to HPC-limb => python int
                # (Simpler approach: val_int = mn.to_int())
                val_int = mn.to_int()
                # We handle exponent within HPC-limb for is_float? 
                # This might be simpler:
                float_val = float(val_int)
            else:
                base_int = int(base_str) 
                exponent_int = int(pow_part)
                # float_val = base_int * (2**exponent_int)
                # but if exponent_int < 0 => fraction
                float_val = base_int * (2**exponent_int)
            if neg:
                float_val = -float_val
        else:
            # parse direct int
            if dec_str.startswith('-'):
                float_val = float(int(dec_str))
            elif dec_str.startswith('...'):
                # truncated => fallback HPC-limb => int
                val_int = mn.to_int()
                float_val = float(val_int)
            else:
                float_val = float(int(dec_str))

        # Now scale back from 32767 if we want actual "amplitude" in [-1..+1]
        # But we never scaled up HPC-limb sums beyond the initial scaling => do so:
        float_val /= 32767.0
        float_wave.append(float_val)

    # Now we have float_wave with the final HPC-limb sum in float form
    # Plot
    plt.figure(figsize=(10,4))
    plt.plot(time_array, float_wave, label="HPC-limb sum => float")
    plt.title("PCM/HPC Hybrid Wave Interference Demo")
    plt.xlabel("Time (sec)")
    plt.ylabel("Amplitude (approx in [-1..+1])")
    plt.grid(True)
    plt.legend()
    plt.show()

# Minimal usage
if __name__=="__main__":
    # If your HPC-limb code is in the same file, just run:
    hpc_pcm_demo(sample_rate=200, duration=1.0)

In [ ]:
import math
import matplotlib.pyplot as plt
from improvedmathv2 import MegaNumber
# We'll rely on your HPC-limb code, e.g. MegaNumber, hpc_add, etc.
# from HPC_code import MegaNumber, hpc_is_zero, hpc_add, hpc_sub, ...

class HPCNeuronSegment:
    """
    Each neuron has:
      - HPC-limb 'weight'
      - HPC-limb 'threshold'
      - HPC-limb 'state' (like a membrane potential or wave amplitude)
    We'll do a simple feed-forward from 'input_mn' -> (input * weight),
    then if the result >= threshold => 'fires' => pass HPC-limb on to next.
    We'll also do a toy 'learning' rule: if it fires + the previous segment fired => weight++
    """

    def __init__(self, weight_mn: MegaNumber, threshold_mn: MegaNumber):
        self.weight = weight_mn
        self.threshold = threshold_mn
        self.state = MegaNumber.from_int(0)
        self.fired = False  # bool to track if this neuron fired last step

    def step(self, input_mn: MegaNumber):
        """
        HPC-limb forward pass:
          out = input_mn * weight
          state = out
          fired = (out >= threshold)
        Returns HPC-limb 'output' if fired, else HPC-limb zero
        """
        # Multiply HPC-limb
        out_mn = self.mul_hpc(input_mn, self.weight)
        self.state = out_mn

        # Compare HPC-limb to threshold
        c = self.compare_hpc(out_mn, self.threshold)
        if c >= 0:
            self.fired = True
            return out_mn
        else:
            self.fired = False
            return MegaNumber.from_int(0)

    def learn(self, prev_fired: bool):
        """
        Toy 'learning': if (this fired) AND (prev_fired) => weight++
        HPC-limb style => weight = weight + 1
        """
        if self.fired and prev_fired:
            one = MegaNumber.from_int(1)
            self.weight = self.weight.add(one)

    # Minimal HPC-limb multiply
    def mul_hpc(self, A: MegaNumber, B: MegaNumber) -> MegaNumber:
        return A.mul(B)

    # Minimal HPC-limb compare => +1 if A>B, 0 if A==B, -1 if A<B
    def compare_hpc(self, A: MegaNumber, B: MegaNumber) -> int:
        if len(A.mantissa) > len(B.mantissa):
            return 1
        if len(A.mantissa) < len(B.mantissa):
            return -1
        # same length => compare from top
        for i in reversed(range(len(A.mantissa))):
            if A.mantissa[i] > B.mantissa[i]:
                return 1
            elif A.mantissa[i] < B.mantissa[i]:
                return -1
        return 0

def hpc_chain_demo(num_segments=5, steps=50):
    """
    Create a chain of HPCNeuronSegment, feed in an HPC-limb wave input each step,
    run a simple learning rule, and record final output wave across steps.
    """
    # 1) Initialize segments with HPC-limb weights & thresholds
    # Let's pick random small weights, threshold around 100 for HPC-limb
    import random
    segs = []
    base_thr = MegaNumber.from_int(2000)  # some HPC-limb threshold
    for i in range(num_segments):
        w_init_val = random.randint(500, 1500)  # HPC-limb weight
        w_init = MegaNumber.from_int(w_init_val)
        segs.append(HPCNeuronSegment(weight_mn=w_init, threshold_mn=base_thr))

    # 2) We'll feed a repeated HPC-limb 'input wave' of amplitude e.g. 1200 rising to 2000
    # to see if chain eventually "learns" to pass more signals along
    # We'll store the final output from the last segment each step.
    import math
    output_history = []
    step_input_history = []
    for step_idx in range(steps):
        # HPC-limb input = let's do a sin wave => [500..2000]
        # We'll do a rough integer amplitude
        amplitude_val = int(1000 + 1000*math.sin(2*math.pi*(step_idx/10)))
        if amplitude_val < 0:
            amplitude_val = 0  # no negative
        input_mn = MegaNumber.from_int(amplitude_val)

        step_input_history.append(input_mn)

        # 3) Feed forward through chain
        prev_fired = False
        next_input = input_mn
        for seg in segs:
            out_mn = seg.step(next_input)
            # learn depends on (seg.fired, prev_fired)
            seg.learn(prev_fired)
            prev_fired = seg.fired
            next_input = out_mn

        # next_input is final from last segment
        output_history.append(next_input)

    # 4) Convert HPC-limb output => float for plotting
    float_out = []
    float_in = []
    for m_in, m_out in zip(step_input_history, output_history):
        f_in = float(m_in.to_int())  # direct HPC-limb => int => float
        f_out = float(m_out.to_int())
        float_in.append(f_in)
        float_out.append(f_out)

    # 5) Plot input & final output over steps
    plt.figure(figsize=(10,5))
    plt.plot(float_in, label="Input HPC-limb (as int float)", marker='o')
    plt.plot(float_out, label="Output HPC-limb (chain final)", marker='x')
    plt.title("HPC-Limb Neuron Chain with Toy Learning")
    plt.xlabel("Time Step")
    plt.ylabel("Amplitude (int scale)")
    plt.legend()
    plt.grid(True)
    plt.show()
    print_hpc_debug_table(step_input_history, output_history, segs, max_steps=10)

# Minimal usage
if __name__=="__main__":
    hpc_chain_demo(num_segments=5, steps=50)


In [ ]:
#!/usr/bin/env python3

import time
import random
import math
import os
import pickle
import array
import threading
from dataclasses import dataclass
from typing import List, Dict, Tuple, Union

import mpmath
mpmath.mp.prec = 20000  # used for HPC comparisons/demos

###############################################################################
# MegaNumber HPC-limb code (with updated mul()/div() negative handling)
###############################################################################

class MegaNumber:
    _global_chunk_size = None
    _base = None
    _mask = None
    _auto_detect_done = False
    _max_precision_bits = None

    def __init__(
        self,
        mantissa: List[int] = None,
        exponent: List[int] = None,
        negative: bool = False,
        is_float: bool = False,
        exponent_negative: bool = False
    ):
        # Possibly auto-pick chunk size
        if not self._auto_detect_done:
            self._auto_pick_chunk_size()
            type(self)._auto_detect_done = True

        if mantissa is None:
            mantissa = [0]
        if exponent is None:
            exponent = [0]

        self.mantissa = mantissa
        self.exponent = exponent
        self.negative = negative
        self.is_float = is_float
        self.exponent_negative = exponent_negative
        self._normalize()

    @classmethod
    def _int_to_chunklist(cls, val: int, csize: int) -> List[int]:
        """
        Convert a Python int 'val' => HPC chunk list of csize bits each, little-endian.
        """
        if val == 0:
            return [0]
        out = []
        mask = (1 << csize) - 1
        while val > 0:
            out.append(val & mask)
            val >>= csize
        return out

    @classmethod
    def from_int(cls, val: int) -> "MegaNumber":
        if val == 0:
            return cls([0], [0], negative=False)
        negative = (val < 0)
        val_abs = abs(val)
        if cls._global_chunk_size is None:
            cls._auto_pick_chunk_size()
            cls._auto_detect_done = True
        # Now we can safely call _int_to_chunklist
        limbs = cls._int_to_chunklist(val_abs, cls._global_chunk_size)
        return cls(mantissa=limbs, exponent=[0], negative=negative)

    @classmethod
    def _auto_pick_chunk_size(cls, candidates=None, test_bit_len=1024, trials=10):
        if candidates is None:
            candidates = [8, 16, 32, 64]
        best_csize = None
        best_time = float('inf')
        for csize in candidates:
            t = cls._benchmark_mul(csize, test_bit_len, trials)
            if t < best_time:
                best_time = t
                best_csize = csize
        cls._global_chunk_size = best_csize
        cls._base = 1 << best_csize
        cls._mask = cls._base - 1

    @classmethod
    def _benchmark_mul(cls, csize, bit_len, trials):
        import random, time
        base = 1 << csize
        start = time.time()
        for _ in range(trials):
            A_val = random.getrandbits(bit_len)
            B_val = random.getrandbits(bit_len)
            # now we can call _int_to_chunklist since it is defined
            A_limb = cls._int_to_chunklist(A_val, csize)
            B_limb = cls._int_to_chunklist(B_val, csize)
            for __ in range(3):
                _ = cls._mul_chunklists(A_limb, B_limb, csize, base)
        return time.time() - start

    def _normalize(self):
        # remove trailing zeros from mantissa
        while len(self.mantissa)>1 and self.mantissa[-1]==0:
            self.mantissa.pop()
        # if float => trim exponent
        if self.is_float:
            while len(self.exponent)>1 and self.exponent[-1]==0:
                self.exponent.pop()
        # if zero => unify sign & exponent
        if len(self.mantissa)==1 and self.mantissa[0]==0:
            self.negative=False
            self.exponent=[0]
            self.exponent_negative=False

    @property
    def max_precision_bits(self):
        return self._max_precision_bits

    def _check_precision_limit(self, num: "MegaNumber"):
        if self._max_precision_bits is not None:
            total_bits = len(num.mantissa) * self._global_chunk_size
            if total_bits > self._max_precision_bits:
                raise ValueError("Precision exceeded!")

    @classmethod
    def from_decimal_string(cls, dec_str: str) -> "MegaNumber":
        if cls._global_chunk_size is None:
            cls._auto_pick_chunk_size()
            cls._auto_detect_done = True

        s = dec_str.strip()
        if not s:
            return cls([0], [0], negative=False, is_float=False)

        negative = False
        if s.startswith('-'):
            negative = True
            s = s[1:].strip()

        point_pos = s.find('.')
        frac_len = 0
        if point_pos >= 0:
            frac_len = len(s) - (point_pos + 1)
            s = s.replace('.', '')

        mant_limb = [0]
        for ch in s:
            if not ('0' <= ch <= '9'):
                raise ValueError(f"Invalid decimal digit {ch}")
            digit_val = (ord(ch) - ord('0'))

            # multiply current by 10
            mant_limb = cls._mul_chunklists(
                mant_limb,
                cls._int_to_chunklist(10, cls._global_chunk_size),
                cls._global_chunk_size,
                cls._base
            )
            # add digit
            carry = digit_val
            idx = 0
            while carry != 0 or idx < len(mant_limb):
                if idx == len(mant_limb):
                    mant_limb.append(0)
                ssum = mant_limb[idx] + carry
                mant_limb[idx] = ssum & cls._mask
                carry = ssum >> cls._global_chunk_size
                idx += 1

        exp_limb = [0]
        exponent_negative = False
        is_float = False
        if frac_len > 0:
            shift_bits = int(math.ceil(frac_len * math.log2(10)))
            exp_limb = cls._int_to_chunklist(shift_bits, cls._global_chunk_size)
            exponent_negative = True
            is_float = True

        obj = cls(
            mantissa=mant_limb,
            exponent=exp_limb,
            negative=negative,
            is_float=is_float,
            exponent_negative=exponent_negative
        )
        obj._normalize()
        return obj

    def to_decimal_string(self, max_digits=None) -> str:
        if len(self.mantissa) == 1 and self.mantissa[0] == 0:
            return "0"
        sign_str = "-" if self.negative else ""
        is_exp_nonzero = (len(self.exponent) > 1 or self.exponent[0] != 0)
        exp_is_neg = self.exponent_negative

        if not is_exp_nonzero and not exp_is_neg:
            dec_str = self._chunk_to_dec_str(self.mantissa, max_digits)
            return sign_str + dec_str
        else:
            mant_str = self._chunk_to_dec_str(self.mantissa, max_digits)
            e_val = self._chunklist_to_small_int(self.exponent, self._global_chunk_size)
            if exp_is_neg:
                e_val = -e_val
            return f"{sign_str}{mant_str} * 2^{e_val}"

    @classmethod
    def _chunk_to_dec_str(cls, limbs: List[int], max_digits=None) -> str:
        if len(limbs) == 1 and limbs[0] == 0:
            return "0"
        temp = limbs[:]
        digits = []
        while not (len(temp) == 1 and temp[0] == 0):
            temp, r = cls._divmod_small(temp, 10)
            digits.append(str(r))
        digits.reverse()
        full_str = "".join(digits)
        if max_digits is None or max_digits >= len(full_str):
            return full_str
        else:
            return f"...{full_str[-max_digits:]}"

    def to_int(self, x: "MegaNumber") -> int:
        val = 0
        shift = 0
        for limb in x.mantissa:
            val += (limb << shift)
            shift += x._global_chunk_size
        if x.negative:
            val = -val
        return val

    ################################################################
    # HPC-limb arithmetic with negative fixes for mul()/div()
    ################################################################

    def add(self, other: "MegaNumber") -> "MegaNumber":
        if self.is_float or other.is_float:
            return self._add_float(other)
        # integer mode
        if self.negative == other.negative:
            sum_limb = self._add_chunklists(self.mantissa, other.mantissa)
            sign = self.negative
            result = MegaNumber(mantissa=sum_limb, negative=sign)
        else:
            c = self._compare_abs(self.mantissa, other.mantissa)
            if c == 0:
                return MegaNumber()
            elif c > 0:
                diff = self._sub_chunklists(self.mantissa, other.mantissa)
                result = MegaNumber(mantissa=diff, negative=self.negative)
            else:
                diff = self._sub_chunklists(other.mantissa, self.mantissa)
                result = MegaNumber(mantissa=diff, negative=other.negative)
        self._check_precision_limit(result)
        return result

    def sub(self, other: "MegaNumber") -> "MegaNumber":
        neg_other = MegaNumber(other.mantissa[:], other.exponent[:], not other.negative)
        return self.add(neg_other)

    def mul(self, other: "MegaNumber") -> "MegaNumber":
        # float fallback
        if self.is_float or other.is_float:
            combined_sign = (self.negative != other.negative)
            def exp_as_int(mn: MegaNumber):
                val = self._chunklist_to_int(mn.exponent)
                return -val if mn.exponent_negative else val
            expA = exp_as_int(self)
            expB = exp_as_int(other)
            sum_exp = expA + expB
            out_limb = self._mul_chunklists(
                self.mantissa, other.mantissa,
                self._global_chunk_size, self._base
            )
            exp_neg = (sum_exp < 0)
            sum_exp_abs = abs(sum_exp)
            new_exponent = (self._int_to_chunklist(sum_exp_abs, self._global_chunk_size) if sum_exp_abs else [0])
            result = MegaNumber(out_limb, new_exponent, combined_sign, is_float=True, exponent_negative=exp_neg)
            result._normalize()
            self._check_precision_limit(result)
            return result

        # integer multiply with sign fix
        combined_sign = (self.negative != other.negative)
        A_abs = self.copy(); A_abs.negative = False
        B_abs = other.copy(); B_abs.negative = False
        out_limb = self._mul_chunklists(A_abs.mantissa, B_abs.mantissa,
                                        self._global_chunk_size, self._base)
        result = MegaNumber(out_limb, [0], negative=combined_sign)
        result._normalize()
        self._check_precision_limit(result)
        return result

    def div(self, other: "MegaNumber") -> "MegaNumber":
        # float fallback
        if self.is_float or other.is_float:
            combined_sign = (self.negative != other.negative)
            def exp_as_int(mn: MegaNumber):
                val = self._chunklist_to_int(mn.exponent)
                return -val if mn.exponent_negative else val
            expA = exp_as_int(self)
            expB = exp_as_int(other)
            new_exponent_val = expA - expB

            if len(other.mantissa) == 1 and other.mantissa[0] == 0:
                raise ZeroDivisionError("division by zero")

            cmp_val = self._compare_abs(self.mantissa, other.mantissa)
            if cmp_val < 0:
                q_limb = [0]
            elif cmp_val == 0:
                q_limb = [1]
            else:
                q_limb, _ = self._div_chunk(self.mantissa, other.mantissa)

            exp_neg = (new_exponent_val < 0)
            new_exponent_val = abs(new_exponent_val)
            new_exponent = (self._int_to_chunklist(new_exponent_val, self._global_chunk_size)
                            if new_exponent_val != 0 else [0])

            result = MegaNumber(q_limb, new_exponent,
                                negative=combined_sign,
                                is_float=True,
                                exponent_negative=exp_neg)
            result._normalize()
            self._check_precision_limit(result)
            return result

        # integer division with negative fix
        if len(other.mantissa) == 1 and other.mantissa[0] == 0:
            raise ZeroDivisionError("division by zero")
        combined_sign = (self.negative != other.negative)
        c = self._compare_abs(self.mantissa, other.mantissa)
        if c < 0:
            return MegaNumber([0], [0], negative=False)
        elif c == 0:
            return MegaNumber([1], [0], negative=combined_sign)

        A_abs = self.copy(); A_abs.negative = False
        B_abs = other.copy(); B_abs.negative = False
        q_limb, _ = self._div_chunk(A_abs.mantissa, B_abs.mantissa)
        result = MegaNumber(q_limb, [0], negative=combined_sign)
        result._normalize()
        self._check_precision_limit(result)
        return result

    def _add_float(self, other: "MegaNumber") -> "MegaNumber":
        def exp_as_int(mn: MegaNumber):
            val = self._chunklist_to_int(mn.exponent)
            return -val if mn.exponent_negative else val
        expA = exp_as_int(self)
        expB = exp_as_int(other)
        if expA == expB:
            mantA, mantB = self.mantissa, other.mantissa
            final_exp = expA
        elif expA > expB:
            shift = expA - expB
            mantA = self.mantissa
            mantB = self._shift_right(other.mantissa, shift)
            final_exp = expA
        else:
            shift = expB - expA
            mantA = self._shift_right(self.mantissa, shift)
            mantB = other.mantissa
            final_exp = expB

        if self.negative == other.negative:
            sum_limb = self._add_chunklists(mantA, mantB)
            sign = self.negative
        else:
            c = self._compare_abs(mantA, mantB)
            if c == 0:
                return MegaNumber(is_float=True)
            elif c > 0:
                sum_limb = self._sub_chunklists(mantA, mantB)
                sign = self.negative
            else:
                sum_limb = self._sub_chunklists(mantB, mantA)
                sign = other.negative

        exp_neg = (final_exp < 0)
        final_exp_abs = abs(final_exp)
        exp_chunk = self._int_to_chunklist(final_exp_abs, self._global_chunk_size) if final_exp_abs else [0]
        out = MegaNumber(sum_limb, exp_chunk, sign, is_float=True, exponent_negative=exp_neg)
        out._normalize()
        self._check_precision_limit(out)
        return out

    def sqrt(self) -> "MegaNumber":
        if self.negative:
            raise ValueError("Cannot sqrt negative.")
        if len(self.mantissa) == 1 and self.mantissa[0] == 0:
            return MegaNumber([0], [0], False, is_float=self.is_float)
        if not self.is_float:
            A = self.mantissa
            low = [0]
            high = A[:]
            csize = self._global_chunk_size
            base = self._base
            while True:
                sum_lh = self._add_chunklists(low, high)
                mid = self._div2(sum_lh)
                c_lo = self._compare_abs(mid, low)
                c_hi = self._compare_abs(mid, high)
                if c_lo == 0 or c_hi == 0:
                    return MegaNumber(mid, [0], False)
                mid_sqr = self._mul_chunklists(mid, mid, csize, base)
                c_cmp = self._compare_abs(mid_sqr, A)
                if c_cmp == 0:
                    return MegaNumber(mid, [0], False)
                elif c_cmp < 0:
                    low = mid
                else:
                    high = mid
        else:
            return self._float_sqrt()

    def _float_sqrt(self) -> "MegaNumber":
        def exp_as_int(mn: MegaNumber):
            val = self._chunklist_to_int(mn.exponent)
            return -val if mn.exponent_negative else val
        total_exp = exp_as_int(self)
        remainder = total_exp & 1
        csize = self._global_chunk_size
        base = self._base
        work_mantissa = self.mantissa[:]

        if remainder != 0:
            if total_exp > 0:
                carry = 0
                for i in range(len(work_mantissa)):
                    doubled = (work_mantissa[i] << 1) + carry
                    work_mantissa[i] = doubled & self._mask
                    carry = doubled >> csize
                if carry != 0:
                    work_mantissa.append(carry)
                total_exp -= 1
            else:
                carry = 0
                for i in reversed(range(len(work_mantissa))):
                    cur_val = (carry << csize) + work_mantissa[i]
                    work_mantissa[i] = cur_val >> 1
                    carry = cur_val & 1
                while len(work_mantissa) > 1 and work_mantissa[-1] == 0:
                    work_mantissa.pop()
                total_exp += 1
        half_exp = total_exp // 2
        low, high = [0], work_mantissa[:]
        while True:
            sum_lh = self._add_chunklists(low, high)
            mid = self._div2(sum_lh)
            c_lo = self._compare_abs(mid, low)
            c_hi = self._compare_abs(mid, high)
            if c_lo == 0 or c_hi == 0:
                sqrt_mantissa = mid
                break
            mid_sqr = self._mul_chunklists(mid, mid, csize, base)
            c_cmp = self._compare_abs(mid_sqr, work_mantissa)
            if c_cmp == 0:
                sqrt_mantissa = mid
                break
            elif c_cmp < 0:
                low = mid
            else:
                high = mid
        exp_neg = (half_exp < 0)
        half_abs = abs(half_exp)
        new_exponent = (self._int_to_chunklist(half_abs, csize) if half_abs else [0])
        out = MegaNumber(sqrt_mantissa, new_exponent, negative=False, is_float=True, exponent_negative=exp_neg)
        out._normalize()
        self._check_precision_limit(out)
        return out

    @classmethod
    def from_int(cls, val: int) -> "MegaNumber":
        if val == 0:
            return cls([0], [0], negative=False)
        negative = (val < 0)
        val_abs = abs(val)
        if cls._global_chunk_size is None:
            cls._auto_pick_chunk_size()
            cls._auto_detect_done = True
        limbs = cls._int_to_chunklist(val_abs, cls._global_chunk_size)
        return cls(mantissa=limbs, exponent=[0], negative=negative)

    @classmethod
    def _chunklist_to_small_int(cls, limbs: List[int], csize: int) -> int:
        val = 0
        shift = 0
        for limb in limbs:
            val += (limb << shift)
            shift += csize
        return val

    @classmethod
    def _chunklist_to_int(cls, limbs: List[int]) -> int:
        if cls._global_chunk_size is None:
            cls._auto_pick_chunk_size()
            cls._auto_detect_done = True
        val = 0
        shift = 0
        for limb in limbs:
            val += (limb << shift)
            shift += cls._global_chunk_size
        return val

    @classmethod
    def _compare_abs(cls, A: List[int], B: List[int]) -> int:
        if len(A) > len(B):
            return 1
        if len(B) > len(A):
            return -1
        for i in range(len(A)-1, -1, -1):
            if A[i] > B[i]:
                return 1
            elif A[i] < B[i]:
                return -1
        return 0

    @classmethod
    def _mul_chunklists(cls, A: List[int], B: List[int], csize: int, base: int) -> List[int]:
        la, lb = len(A), len(B)
        out = [0]*(la+lb)
        for i in range(la):
            carry = 0
            av = A[i]
            for j in range(lb):
                mul_val = av * B[j] + out[i+j] + carry
                out[i+j] = mul_val & (base-1)
                carry = mul_val >> csize
            if carry:
                out[i+lb] += carry
        while len(out) > 1 and out[-1] == 0:
            out.pop()
        return out

    @classmethod
    def _div_chunk(cls, A: List[int], B: List[int]) -> Tuple[List[int], List[int]]:
        if cls._global_chunk_size is None:
            cls._auto_pick_chunk_size()
            cls._auto_detect_done = True
        if len(B) == 1 and B[0] == 0:
            raise ZeroDivisionError("divide by zero")
        c = cls._compare_abs(A, B)
        if c < 0:
            return ([0], A)
        if c == 0:
            return ([1], [0])
        Q = [0]*len(A)
        R = [0]
        base = 1 << cls._global_chunk_size
        for i in range(len(A)-1, -1, -1):
            R = cls._mul_chunklists(R, [base], cls._global_chunk_size, base)
            R = cls._add_chunklists(R, [A[i]])
            low, high = 0, base-1
            guess = 0
            while low <= high:
                mid = (low+high) >> 1
                mm = cls._mul_chunklists(B, [mid], cls._global_chunk_size, base)
                cmpv = cls._compare_abs(mm, R)
                if cmpv <= 0:
                    guess = mid
                    low = mid+1
                else:
                    high = mid-1
            if guess != 0:
                mm = cls._mul_chunklists(B, [guess], cls._global_chunk_size, base)
                R = cls._sub_chunklists(R, mm)
            Q[i] = guess
        while len(Q) > 1 and Q[-1] == 0:
            Q.pop()
        while len(R) > 1 and R[-1] == 0:
            R.pop()
        return (Q,R)

    @classmethod
    def _divmod_small(cls, A: List[int], small_val: int) -> Tuple[List[int], int]:
        remainder = 0
        out = [0]*len(A)
        csize = cls._global_chunk_size
        for i in reversed(range(len(A))):
            cur = (remainder << csize) + A[i]
            qd = cur // small_val
            remainder = cur % small_val
            out[i] = qd & cls._mask
        while len(out) > 1 and out[-1] == 0:
            out.pop()
        return (out, remainder)

    @classmethod
    def _add_chunklists(cls, A: List[int], B: List[int]) -> List[int]:
        out = []
        carry = 0
        max_len = max(len(A), len(B))
        for i in range(max_len):
            av = A[i] if i < len(A) else 0
            bv = B[i] if i < len(B) else 0
            s = av + bv + carry
            carry = s >> cls._global_chunk_size
            out.append(s & cls._mask)
        if carry:
            out.append(carry)
        while len(out) > 1 and out[-1] == 0:
            out.pop()
        return out

    @classmethod
    def _sub_chunklists(cls, A: List[int], B: List[int]) -> List[int]:
        out = []
        carry = 0
        max_len = max(len(A), len(B))
        for i in range(max_len):
            av = A[i] if i < len(A) else 0
            bv = B[i] if i < len(B) else 0
            diff = av - bv - carry
            if diff < 0:
                diff += cls._base
                carry = 1
            else:
                carry = 0
            out.append(diff & cls._mask)
        while len(out) > 1 and out[-1] == 0:
            out.pop()
        return out

    @classmethod
    def _div2(cls, limbs: List[int]) -> List[int]:
        out = []
        carry = 0
        csize = cls._global_chunk_size
        for i in reversed(range(len(limbs))):
            val = (carry << csize) + limbs[i]
            q = val >> 1
            r = val & 1
            carry = r
            out.append(q)
        out.reverse()
        while len(out) > 1 and out[-1] == 0:
            out.pop()
        return out

    def copy(self) -> "MegaNumber":
        return MegaNumber(self.mantissa[:], self.exponent[:], self.negative, self.is_float, self.exponent_negative)

    def __repr__(self):
        return f"<MegaNumber {self.to_decimal_string(50)}>"

###############################################################################
# HPC Utility
###############################################################################

def hpc_is_zero(x: MegaNumber) -> bool:
    return (len(x.mantissa) == 1 and x.mantissa[0] == 0)

def hpc_shr1(x: MegaNumber) -> MegaNumber:
    if x.negative:
        raise NotImplementedError("hpc_shr1 for negative not implemented.")
    new_mant = MegaNumber._div2(x.mantissa)
    out = MegaNumber(new_mant, [0], False)
    out._normalize()
    return out

###############################################################################
# HPCNeuronSegment + chain example
###############################################################################

class HPCNeuronSegment:
    """
    Each neuron has HPC-limb weight, HPC-limb threshold, HPC-limb state,
    and a 'fired' bool. We do a simple feed-forward and toy 'learn()' method.
    """
    def __init__(self, weight_mn: MegaNumber, threshold_mn: MegaNumber):
        self.weight = weight_mn
        self.threshold = threshold_mn
        self.state = MegaNumber.from_int(0)
        self.fired = False

    def step(self, input_mn: MegaNumber):
        """
        out = input_mn * weight
        state=out
        fired = (out >= threshold)
        returns HPC-limb out or HPC-limb zero
        """
        out_mn = self.mul_hpc(input_mn, self.weight)
        self.state = out_mn
        if self.compare_hpc(out_mn, self.threshold) >= 0:
            self.fired = True
            return out_mn
        else:
            self.fired = False
            return MegaNumber.from_int(0)

    def learn(self, prev_fired: bool):
        """
        If both me & prev fired => weight++
        HPC-limb style => weight=weight+1
        """
        if self.fired and prev_fired:
            one = MegaNumber.from_int(1)
            self.weight = self.weight.add(one)

    def mul_hpc(self, A: MegaNumber, B: MegaNumber)->MegaNumber:
        return A.mul(B)

    def compare_hpc(self, A: MegaNumber, B: MegaNumber)->int:
        #  -1 if A<B, 0 if A==B, +1 if A>B
        if len(A.mantissa)>len(B.mantissa):
            return 1
        if len(A.mantissa)<len(B.mantissa):
            return -1
        for i in reversed(range(len(A.mantissa))):
            if A.mantissa[i]>B.mantissa[i]:
                return 1
            elif A.mantissa[i]<B.mantissa[i]:
                return -1
        return 0

def print_hpc_debug_table(step_input_history, output_history, segs, max_steps=10):
    """
    Print a table of HPC-limb input/output for each step (up to max_steps).
    Also optionally show HPC-limb weights of each segment in integer form.
    """
    n = min(len(step_input_history), max_steps)
    print("\n=== HPC Debug Table (first", n, "steps) ===")
    print("Step | Input(10)     -> Output(10)     [Weigths in HPC-limb int]")

    for i in range(n):
        inp_mn = step_input_history[i]
        out_mn = output_history[i]
        # HPC-limb => python int
        inp_int = inp_mn.to_int(inp_mn)
        out_int = out_mn.to_int(out_mn)
        # gather weights
        w_str = []
        for s in segs:
            w_int = s.weight.to_int(s.weight)
            w_str.append(str(w_int))
        print(f"{i:4d} | {inp_int:12d} -> {out_int:12d}   w={[x for x in w_str]}")

def hpc_chain_demo(num_segments=5, steps=50):
    import numpy as np

    # 1) segs init
    base_thr = MegaNumber.from_int(2000)
    segs = []
    for i in range(num_segments):
        w_init_val = random.randint(500,1500)
        w_init = MegaNumber.from_int(w_init_val)
        segs.append(HPCNeuronSegment(w_init, base_thr))

    step_input_history=[]
    output_history=[]

    for step_idx in range(steps):
        # HPC-limb input => let's do a sin wave [0..2000]
        val_float = 1000 + 1000*math.sin(2*math.pi*(step_idx/10))
        if val_float<0: val_float=0
        val_int = int(round(val_float))
        input_mn = MegaNumber.from_int(val_int)
        step_input_history.append(input_mn)

        prev_fired=False
        next_in= input_mn
        for s in segs:
            out_mn= s.step(next_in)
            s.learn(prev_fired)
            prev_fired= s.fired
            next_in= out_mn

        output_history.append(next_in)

    # optionally debug print table for first steps
    print_hpc_debug_table(step_input_history, output_history, segs, max_steps=12)

    # convert HPC-limb => float for final plot
    float_in=[]
    float_out=[]
    for m_in, m_out in zip(step_input_history, output_history):
        fi= m_in.to_int(m_in)*1.0
        fo= m_out.to_int(m_out)*1.0
        float_in.append(fi)
        float_out.append(fo)

    import matplotlib.pyplot as plt
    plt.figure(figsize=(10,5))
    plt.plot(float_in, label="Input HPC-limb (as int float)", marker='o')
    plt.plot(float_out, label="Output HPC-limb (chain final)", marker='x')
    plt.title("HPC-Limb Neuron Chain with Toy Learning")
    plt.xlabel("Time Step")
    plt.ylabel("Amplitude (int scale)")
    plt.grid(True)
    plt.legend()
    plt.show()

###############################################################################
# Main usage
###############################################################################
def main():
    hpc_chain_demo(num_segments=5, steps=50)

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import math, random
from array import array
import numpy as np
import matplotlib.pyplot as plt

#######################################################
# HPC-limb code (64-bit chunks) - minimal version
#######################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """Convert nonnegative Python int -> array of 64-bit limbs."""
    if value < 0:
        raise ValueError("Negative not handled here.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    """Recombine HPC-limb => Python int."""
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q', [0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val = av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry  = s_val >> CHUNK_BITS
    if carry:
        out[out_len] = carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q', [0]*out_len)
    carry = 0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff = av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb = len(A), len(B)
    # Compare top limbs in reverse
    if la>lb:
        if A[-1]!=0: return 1
    elif lb>la:
        if B[-1]!=0: return -1
    top = max(la, lb)
    for i in range(top-1, -1, -1):
        av = A[i] if i<la else 0
        bv = B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out = array('Q', A)
    limb_shifts = shift_bits // CHUNK_BITS
    bit_shifts  = shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out = out[limb_shifts:]
    if bit_shifts==0:
        if not out: out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur >> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

#######################################################
# HPC-limb => "mV" mapping, for plotting
#######################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    val  = limbs_to_int(wave_limb)
    vmax = limbs_to_int(wave_max)
    fraction = val/vmax if vmax>0 else 0.0
    return rest_mV + range_mV*fraction

#######################################################
# HPCWaveSegment: HPC-limb wave states
#######################################################

class HPCWaveSegment:
    def __init__(self):
        # wave_max => "1.0 fraction" => e.g. 1_000_000
        self.wave_max = int_to_limbs(1_000_000)
        # wave_state => start ~1% => 10,000
        self.wave_state= int_to_limbs(10_000)
        # thresholds => Ca=2%, K=3%
        self.ca_threshold= int_to_limbs(20_000)
        self.k_threshold=  int_to_limbs(30_000)

        self.toggle_history = []
        self.conduction_amplitude_history = []

def conduction_step(segments, conduction_delay=1):
    n = len(segments)
    zeroA = int_to_limbs(0)
    toggles= [zeroA]*n
    conduction_vals= [zeroA]*n
    for i, seg in enumerate(segments):
        # conduction => wave_state//8
        conduction = hpc_shr(seg.wave_state,3)
        conduction_vals[i] = conduction
        # pass conduction to neighbors if >0
        if hpc_compare(conduction, zeroA)>0:
            if i>0: toggles[i-1] = conduction
            if i<n-1: toggles[i+1] = conduction
    # store conduction amplitude
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_vals[i])
        if len(seg.toggle_history)>= conduction_delay:
            add_val = seg.toggle_history.pop(0)
            seg.wave_state= hpc_add(seg.wave_state, add_val)
        seg.toggle_history.append(toggles[i])

def ion_channels(seg: HPCWaveSegment):
    # Ca push => wave>ca => wave+= wave//16
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4)
        seg.wave_state= hpc_add(seg.wave_state, plus_val)
    # K pull => wave>k => wave-= wave//8
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3)
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def partial_reflection(segments):
    """At boundaries: wave -= wave//16 => HPC-limb style"""
    if not segments:
        return
    sub_left = hpc_shr(segments[0].wave_state,4)
    segments[0].wave_state= hpc_sub(segments[0].wave_state, sub_left)
    sub_right= hpc_shr(segments[-1].wave_state,4)
    segments[-1].wave_state= hpc_sub(segments[-1].wave_state, sub_right)

#######################################################
# 1) Insert random/harmonic input
#######################################################

def random_harmonic_input(t:int, seg: HPCWaveSegment, freq_list=None):
    """
    Example driver: sum of random or harmonic waves => HPC-limb integer
    freq_list => e.g. [0.05, 0.12, 0.2] => frequencies in "1/time step" domain
    """
    if freq_list is None:
        freq_list = [0.03, 0.07, 0.11]  # random example

    # sum ~ amplitude => up to ~5000
    # can do e.g. amplitude = 1000 * (some sin freq)
    # or random pulses
    total = 0
    for freq in freq_list:
        phase = 2*math.pi*freq*t
        val_float = 1000.0*math.sin(phase)
        total += int(val_float)

    # add some random noise
    noise = random.randint(-50, 50)
    total += noise
    # clamp
    if total<0:
        # HPC-limb code snippet here only does nonnegative
        total=0

    # HPC-limb
    driver_limb = int_to_limbs(total)
    # add to wave_state
    seg.wave_state= hpc_add(seg.wave_state, driver_limb)

def update_hpc_wave(segments, step):
    """One time-step: conduction, channels, reflection, plus input wave on segment0."""
    # Insert random/harmonic wave in segment0
    random_harmonic_input(step, segments[0], freq_list=[0.05, 0.10])

    # conduction
    conduction_step(segments)
    # ion channels
    for seg in segments:
        ion_channels(seg)
    # reflection
    partial_reflection(segments)

#######################################################
# 2) Running + collecting HPC-limb states
#######################################################

def run_hpc_wave_with_input(num_segments=8, steps=50):
    segs = [HPCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg0 => wave_state= 20k => 2%
    segs[0].wave_state = int_to_limbs(20_000)

    history=[]
    for step in range(steps):
        # store HPC-limb snapshot
        snap= [array('Q', s.wave_state) for s in segs]
        history.append(snap)
        update_hpc_wave(segs, step)

    # final snap
    final_snap= [array('Q', s.wave_state) for s in segs]
    history.append(final_snap)
    return segs, history

#######################################################
# 3) Plot HPC-limb wave in mV
#######################################################

def plot_hpc_results(segs, history):
    steps_plus = len(history)
    num_segs = len(segs)
    wave_mV = []
    for t in range(steps_plus):
        rowf=[]
        for i, seg in enumerate(segs):
            mv_val= HPCWave_to_mV(history[t][i], seg.wave_max, range_mV=40.0, rest_mV=-70.0)
            rowf.append(mv_val)
        wave_mV.append(rowf)
    wave_arr= np.array(wave_mV)

    # conduction amplitude
    max_steps= 0
    for s in segs:
        max_steps= max(max_steps, len(s.conduction_amplitude_history))
    conduction_arr=[]
    for step_idx in range(max_steps):
        rowC=[]
        for seg in segs:
            if step_idx< len(seg.conduction_amplitude_history):
                c_val= limbs_to_int(seg.conduction_amplitude_history[step_idx])
                rowC.append(c_val)
            else:
                rowC.append(0)
        conduction_arr.append(rowC)
    conduction_np= np.array(conduction_arr)

    fig, axes= plt.subplots(2,1, figsize=(10,8))

    # HPC-limb wave line
    ax1= axes[0]
    time_axis= np.arange(steps_plus)
    for seg_i in range(num_segs):
        ax1.plot(time_axis, wave_arr[:, seg_i], label=f"Seg {seg_i}")
    ax1.set_title("HPC-limb wave with Random/Harmonic Input (mV approx.)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.legend()
    ax1.grid(True)

    # conduction amplitude lines
    ax2= axes[1]
    c_t= np.arange(len(conduction_np))
    for seg_i in range(num_segs):
        ax2.plot(c_t, conduction_np[:, seg_i], label=f"Seg {seg_i}")
    ax2.set_title("Conduction Amplitude Over Time (HPC-limb ints)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude (int)")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    return fig

#######################################################
# 4) main driver
#######################################################

def main():
    segs, hist= run_hpc_wave_with_input(num_segments=8, steps=50)
    fig= plot_hpc_results(segs, hist)
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import math, random
from array import array
import numpy as np
import matplotlib.pyplot as plt

###############################################################################
# 1) HPC-limb code (64-bit) - minimal version
###############################################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """
    Convert nonnegative Python int -> HPC-limb array of 64-bit 'Q'.
    """
    if value < 0:
        raise ValueError("Negative ints not supported here.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val=0
    shift=0
    for limb in limbs:
        val += (limb << shift)
        shift+= CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q', [0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry  = s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff+= CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    # remove trailing zeros
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    # compare HPC-limb arrays, ignoring sign
    la, lb= len(A), len(B)
    # check top limbs
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts=  shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out: out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<< bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

###############################################################################
# 2) HPC-limb wave -> approximate mV
###############################################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    val = limbs_to_int(wave_limb)
    vmax= limbs_to_int(wave_max)
    fraction= val/vmax if vmax>0 else 0.0
    return rest_mV + range_mV*fraction

###############################################################################
# 3) HPCWaveSegment & HPC-limb wave logic
###############################################################################

class HPCWaveSegment:
    def __init__(self, wave_max_val=1_000_000, start_val=10_000):
        # wave_max => "1.0 fraction"
        self.wave_max = int_to_limbs(wave_max_val)
        # wave_state => ~start_val
        self.wave_state= int_to_limbs(start_val)
        # thresholds => e.g. 2% & 3%
        thr_ca_val= int(wave_max_val*0.02)
        thr_k_val = int(wave_max_val*0.03)
        self.ca_threshold= int_to_limbs(thr_ca_val)
        self.k_threshold=  int_to_limbs(thr_k_val)

        # conduction backlog
        self.toggle_history = []
        self.conduction_amplitude_history = []

class HPCDebugData:
    """
    For each time step, store:
      - wave_pre (int), wave_post_ion (int), wave_final (int)
      - conduction_in (int)
      - reflection_sub (int) for boundary
      - ca_bits, k_bits from wave?
    We'll store them in a dict of dicts => debug_data[step][seg_idx] = {...}
    """
    def __init__(self):
        self.data= {}

    def record(self, step:int, seg_idx:int, key:str, val:int):
        if step not in self.data:
            self.data[step] = {}
        if seg_idx not in self.data[step]:
            self.data[step][seg_idx]= {}
        self.data[step][seg_idx][key]= val

    def get(self, step, seg_idx, key, default=0):
        return self.data.get(step,{}).get(seg_idx,{}).get(key, default)

def conduction_step(segments, conduction_delay:int, debug_data:HPCDebugData, step:int):
    """
    HPC-limb conduction => wave_state//8 => pass to neighbors
    """
    n= len(segments)
    zeroA= int_to_limbs(0)
    toggles= [zeroA]*n
    conduction_vals= [zeroA]*n

    for i, seg in enumerate(segments):
        conduction= hpc_shr(seg.wave_state, 3)  # wave//8
        conduction_vals[i]= conduction
        c_int= limbs_to_int(conduction)
        debug_data.record(step, i, "conduction_out", c_int)

        # pass conduction
        if hpc_compare(conduction, zeroA)>0:
            if i>0:
                toggles[i-1]= conduction
            if i<n-1:
                toggles[i+1]= conduction

    # apply conduction
    for i, seg in enumerate(segments):
        seg.conduction_amplitude_history.append(conduction_vals[i])
        if len(seg.toggle_history)>= conduction_delay:
            add_val= seg.toggle_history.pop(0)
            seg.wave_state= hpc_add(seg.wave_state, add_val)
            add_int= limbs_to_int(add_val)
            debug_data.record(step, i, "conduction_in", add_int)
        else:
            debug_data.record(step, i, "conduction_in", 0)
        seg.toggle_history.append(toggles[i])

def ion_channels(seg: HPCWaveSegment, debug_data:HPCDebugData, step:int, seg_idx:int):
    """
    HPC-limb partial Ca push, K pull
    """
    wave_int_pre= limbs_to_int(seg.wave_state)
    # get bits for ca, k
    ca_bits= wave_int_pre & 0xFFF
    k_bits= (wave_int_pre>>12) & 0xFFF
    debug_data.record(step, seg_idx, "ca_bits", ca_bits)
    debug_data.record(step, seg_idx, "k_bits", k_bits)

    # Ca push => wave>ca => wave += wave//16
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state, 4) # wave//16
        seg.wave_state= hpc_add(seg.wave_state, plus_val)

    # K pull => wave>k => wave -= wave//8
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state, 3) # wave//8
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def partial_reflection(segments, debug_data:HPCDebugData, step:int):
    """
    HPC-limb reflection at edges => wave -= wave//16
    store how much sub was removed => reflection_sub
    """
    if not segments:
        return
    left_seg= segments[0]
    right_seg= segments[-1]

    subL = hpc_shr(left_seg.wave_state, 4)
    subL_int= limbs_to_int(subL)
    left_seg.wave_state= hpc_sub(left_seg.wave_state, subL)
    debug_data.record(step, 0, "reflection_sub", subL_int)

    subR = hpc_shr(right_seg.wave_state,4)
    subR_int= limbs_to_int(subR)
    right_seg.wave_state= hpc_sub(right_seg.wave_state, subR)
    debug_data.record(step, len(segments)-1, "reflection_sub", subR_int)

def random_harmonic_input(t:int, seg: HPCWaveSegment, freq_list=None):
    """
    Example driver: sum of random or harmonic waves => HPC-limb int
    """
    if freq_list is None:
        freq_list = [0.05, 0.10]
    total= 0
    for freq in freq_list:
        phase= 2*math.pi*freq*t
        val_f= 1000.0*math.sin(phase)
        total += int(val_f)
    # random noise
    noise= random.randint(-50,50)
    total+= noise
    if total<0: total=0
    add_limb= int_to_limbs(total)
    seg.wave_state= hpc_add(seg.wave_state, add_limb)

def update_hpc_wave(segments, step, debug_data:HPCDebugData):
    # record pre state
    for i, seg in enumerate(segments):
        wave_int= limbs_to_int(seg.wave_state)
        debug_data.record(step, i, "wave_pre", wave_int)

    # inject random/harmonic wave on segment0
    random_harmonic_input(step, segments[0])

    # conduction
    conduction_step(segments, conduction_delay=1, debug_data=debug_data, step=step)

    # ion channels
    for i, seg in enumerate(segments):
        ion_channels(seg, debug_data, step, i)

    # record post-ion channel
    for i, seg in enumerate(segments):
        wave_int_post= limbs_to_int(seg.wave_state)
        debug_data.record(step, i, "wave_post_ion", wave_int_post)

    # reflection
    partial_reflection(segments, debug_data, step)

    # record final wave
    for i, seg in enumerate(segments):
        wave_int_final= limbs_to_int(seg.wave_state)
        debug_data.record(step, i, "wave_final", wave_int_final)

def run_hpc_wave_all_debug(num_segments=8, steps=50):
    segs= [HPCWaveSegment() for _ in range(num_segments)]
    # artificially excite seg0 => 2% => 20k
    segs[0].wave_state= int_to_limbs(20_000)
    debug_data= HPCDebugData()
    for step in range(steps):
        update_hpc_wave(segs, step, debug_data)
    return segs, debug_data

############################################################
# Plot with multiple subplots to show HPC-limb data
############################################################

def plot_all_debug(segs, debug_data:HPCDebugData, steps=50):
    """
    Subplots:
      1) HPC-limb wave final => interpret as mV
      2) conduction_in
      3) reflection_sub
      4) Ca bits, K bits
      5) wave_pre vs. wave_post_ion
      6) final distribution
    """
    num_segs= len(segs)
    time_axis= np.arange(steps)

    # Collect wave_final in mV
    wave_mV= np.zeros((steps, num_segs), dtype=float)
    conduction_in= np.zeros((steps, num_segs), dtype=float)
    reflection_sub= np.zeros((steps, num_segs), dtype=float)
    ca_bits_log= np.zeros((steps, num_segs), dtype=float)
    k_bits_log=  np.zeros((steps, num_segs), dtype=float)
    wave_pre=    np.zeros((steps, num_segs), dtype=float)
    wave_post=   np.zeros((steps, num_segs), dtype=float)

    for step in range(steps):
        for seg_i in range(num_segs):
            wave_f = debug_data.get(step, seg_i, "wave_final", 0)
            wave_prev= debug_data.get(step, seg_i, "wave_pre", 0)
            wave_postv=debug_data.get(step, seg_i, "wave_post_ion",0)
            cond_in = debug_data.get(step, seg_i, "conduction_in", 0)
            refl_sub= debug_data.get(step, seg_i, "reflection_sub",0)
            ca_b   = debug_data.get(step, seg_i, "ca_bits", 0)
            kb     = debug_data.get(step, seg_i, "k_bits", 0)

            # HPC-limb => fraction => mV
            wave_limb_f= int_to_limbs(wave_f)
            wave_mV[step, seg_i]= HPCWave_to_mV(wave_limb_f, segs[seg_i].wave_max)

            wave_pre[step, seg_i]= wave_prev
            wave_post[step, seg_i]= wave_postv
            conduction_in[step, seg_i]= cond_in
            reflection_sub[step, seg_i]= refl_sub
            ca_bits_log[step, seg_i]= ca_b
            k_bits_log[step, seg_i]= kb

    fig, axes= plt.subplots(3,2, figsize=(15,12))
    ax1, ax2, ax3, ax4, ax5, ax6= axes.flatten()

    # 1) HPC-limb wave (mV)
    for seg_i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:,seg_i], label=f"Seg {seg_i}")
    ax1.set_title("HPC-limb Wave Final => mV")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.grid(True)
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 2) conduction_in lines
    for seg_i in range(num_segs):
        ax2.plot(time_axis, conduction_in[:, seg_i], label=f"Seg {seg_i}")
    ax2.set_title("Conduction In (int HPC-limb)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude")
    ax2.grid(True)
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 3) reflection_sub lines
    for seg_i in range(num_segs):
        ax3.plot(time_axis, reflection_sub[:, seg_i], label=f"Seg {seg_i}")
    ax3.set_title("Reflection Sub (int HPC-limb)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude")
    ax3.grid(True)
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 4) Ca / K bits => let's do Ca in solid, K in dashed
    for seg_i in range(num_segs):
        ax4.plot(time_axis, ca_bits_log[:, seg_i], label=f"Seg{seg_i} Ca", alpha=0.6)
        ax4.plot(time_axis, k_bits_log[:, seg_i], '--', label=f"Seg{seg_i} K", alpha=0.6)
    ax4.set_title("Ca bits (solid) & K bits (dashed)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Bits (0..4095)")
    ax4.grid(True)
    ax4.legend(bbox_to_anchor=(1.05,1), loc="upper left")

    # 5) wave_pre vs wave_post => difference
    # We'll just pick seg0 for example
    seg0_pre  = wave_pre[:,0]
    seg0_post = wave_post[:,0]
    ax5.plot(time_axis, seg0_pre, label="Seg0 wave_pre", alpha=0.7)
    ax5.plot(time_axis, seg0_post, label="Seg0 wave_postIon", alpha=0.7)
    ax5.set_title("Segment0 wave_pre vs wave_postIon (int HPC-limb)")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Amplitude (int)")
    ax5.grid(True)
    ax5.legend()

    # 6) final distribution => wave_mV at last step
    final_vals= wave_mV[-1,:]
    ax6.hist(final_vals, bins=20, alpha=0.7)
    ax6.set_title("Final HPC-limb wave distribution (mV)")
    ax6.set_xlabel("Voltage (mV)")
    ax6.set_ylabel("Count")
    ax6.grid(True)

    plt.tight_layout()
    return fig

def main():
    segs, debug_data= run_hpc_wave_all_debug(num_segments=8, steps=50)
    fig= plot_all_debug(segs, debug_data, steps=50)
    plt.show()

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import math, random
from array import array
import numpy as np
import matplotlib.pyplot as plt
import time

###############################################################################
# 1) HPC-limb code (64-bit) - minimal version
###############################################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """Convert nonnegative Python int -> HPC-limb array of 64-bit 'Q'."""
    if value < 0:
        raise ValueError("Negative ints not supported here.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val=0
    shift=0
    for limb in limbs:
        val += (limb << shift)
        shift+= CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q', [0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry  = s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb= len(A), len(B)
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts=  shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out: out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<< bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

###############################################################################
# HPC-limb wave => approximate mV
###############################################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    val = limbs_to_int(wave_limb)
    vmax= limbs_to_int(wave_max)
    fraction= val/vmax if vmax>0 else 0.0
    return rest_mV + range_mV*fraction

###############################################################################
# HPCWaveSegment + HPC-limb wave logic
###############################################################################

class HPCWaveSegment:
    """
    HPC-limb wave segment. wave_state can be large. conduction backlog, thresholds, etc.
    We'll define a 'prev_wave_int' to track wave(t-1) => for velocity.
    """
    def __init__(self, wave_max_val=1_000_000, start_val=10_000):
        self.wave_max = int_to_limbs(wave_max_val)
        self.wave_state= int_to_limbs(start_val)
        ca_thr = int(wave_max_val*0.02)  # 2%
        k_thr  = int(wave_max_val*0.03)  # 3%
        self.ca_threshold= int_to_limbs(ca_thr)
        self.k_threshold=  int_to_limbs(k_thr)

        self.toggle_history = []
        self.conduction_amplitude_history = []
        # For velocity calc
        self.prev_wave_int = start_val

###############################################################################
# HPCDebugData for storing step-by-step details
###############################################################################

class HPCDebugData:
    def __init__(self):
        self.data = {}
        # we store velocity: wave(t) - wave(t-1)

    def record_val(self, step:int, seg_idx:int, key:str, val:int):
        """
        e.g. self.data[step][seg_idx][key]= val
        """
        if step not in self.data:
            self.data[step] = {}
        if seg_idx not in self.data[step]:
            self.data[step][seg_idx] = {}
        self.data[step][seg_idx][key] = val

    def fetch(self, step:int, seg_idx:int, key:str, default=0)->int:
        return self.data.get(step,{}).get(seg_idx,{}).get(key, default)

###############################################################################
# random + harmonic input
###############################################################################

def random_harmonic_input(t:int, seg: HPCWaveSegment, freq_list=None):
    if freq_list is None:
        freq_list=[0.05, 0.10]
    total=0
    # sum sin waves
    for freq in freq_list:
        phase= 2*math.pi*freq*t
        val_f= 1000.0*math.sin(phase)
        total += int(val_f)
    # random noise
    noise= random.randint(-50,50)
    total+= noise
    if total<0: total=0
    seg.wave_state= hpc_add(seg.wave_state, int_to_limbs(total))

###############################################################################
# conduction, ion channels, partial reflection
###############################################################################

def conduction_step(segments, conduction_delay:int, dbg, step:int):
    n = len(segments)
    zeroA = int_to_limbs(0)
    toggles = [zeroA]*n
    conduction_vals = [zeroA]*n

    for i, seg in enumerate(segments):
        conduction = hpc_shr(seg.wave_state, 3)  # wave//8
        conduction_vals[i] = conduction
        # Only record debug if dbg is not None
        if dbg is not None:
            dbg.record_val(step, i, "conduction_out", limbs_to_int(conduction))

    # Pass conduction to neighbors
    for i, seg in enumerate(segments):
        cval = conduction_vals[i]
        if hpc_compare(cval, zeroA) > 0:
            if i > 0:
                toggles[i-1] = cval
            if i < n-1:
                toggles[i+1] = cval

    # Apply conduction with delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            inflow = seg.toggle_history.pop(0)
            seg.wave_state = hpc_add(seg.wave_state, inflow)
            if dbg is not None:
                dbg.record_val(step, i, "conduction_in", limbs_to_int(inflow))
        else:
            if dbg is not None:
                dbg.record_val(step, i, "conduction_in", 0)
        seg.toggle_history.append(toggles[i])

def ion_channels(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_idx:int):
    """
    partial Ca push => wave>ca => wave += wave//16
    partial K pull => wave>k => wave -= wave//8
    record Ca bits, K bits, velocity
    """
    wave_int_pre= limbs_to_int(seg.wave_state)
    # Ca bits => wave_int_pre & 0xFFF
    # K bits => (wave_int_pre>>12)&0xFFF
    cab = wave_int_pre & 0xFFF
    kb  = (wave_int_pre>>12) & 0xFFF
    dbg.record_val(step, seg_idx, "ca_bits", cab)
    dbg.record_val(step, seg_idx, "k_bits", kb)

    # Ion thresholds
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4) # wave//16
        seg.wave_state= hpc_add(seg.wave_state, plus_val)

    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3) # wave//8
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def partial_reflection(segments, dbg:HPCDebugData, step:int):
    """
    HPC-limb reflection => wave -= wave//16
    record reflection_sub
    """
    if not segments:
        return
    left= segments[0]
    right= segments[-1]
    subL= hpc_shr(left.wave_state,4)
    left.wave_state= hpc_sub(left.wave_state, subL)
    dbg.record_val(step, 0, "reflection_sub", limbs_to_int(subL))

    subR= hpc_shr(right.wave_state,4)
    right.wave_state= hpc_sub(right.wave_state, subR)
    dbg.record_val(step, len(segments)-1, "reflection_sub", limbs_to_int(subR))

def compute_velocity(seg: HPCWaveSegment):
    """
    velocity => wave_int(t) - wave_int(t-1)
    store wave_int in seg.prev_wave_int
    """
    curr_int= limbs_to_int(seg.wave_state)
    vel= curr_int - seg.prev_wave_int
    seg.prev_wave_int= curr_int
    return vel

###############################################################################
# update step
###############################################################################

def update_hpc_wave(segments, step:int, dbg:HPCDebugData):
    # record wave_pre
    for i, seg in enumerate(segments):
        wv_int= limbs_to_int(seg.wave_state)
        dbg.record_val(step, i, "wave_pre", wv_int)

    # random/harmonic input => only seg0
    random_harmonic_input(step, segments[0])

    # conduction
    conduction_step(segments, conduction_delay=1, dbg=dbg, step=step)

    # ion channels
    for i, seg in enumerate(segments):
        ion_channels(seg, dbg, step, i)

    # wave_postIon
    for i, seg in enumerate(segments):
        wv_post= limbs_to_int(seg.wave_state)
        dbg.record_val(step, i, "wave_postIon", wv_post)

    # reflection
    partial_reflection(segments, dbg, step)

    # final wave + velocity
    for i, seg in enumerate(segments):
        wv_final= limbs_to_int(seg.wave_state)
        dbg.record_val(step, i, "wave_final", wv_final)
        # velocity
        vel= compute_velocity(seg)
        dbg.record_val(step, i, "velocity", vel)

###############################################################################
# run with debug
###############################################################################

def run_hpc_wave_debug(num_segments=8, steps=50):
    segs= []
    for i in range(num_segments):
        # start wave => e.g. 10k for all
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    # artificially excite seg0 => wave_state= 20k => 2%
    segs[0].wave_state= int_to_limbs(20_000)

    dbg= HPCDebugData()
    for step in range(steps):
        update_hpc_wave(segs, step, dbg)
    return segs, dbg

###############################################################################
# 2) Stress test => 3000 segments
###############################################################################

def run_hpc_wave_stress_test(num_segments=3000, steps=20):
    """
    We'll skip debug logs to avoid huge memory usage.
    We'll just measure timing & final wave states.
    """
    segs= []
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    # start seg0 => 2%
    segs[0].wave_state= int_to_limbs(20_000)
    t0= time.perf_counter()
    for step in range(steps):
        # partial update
        # conduction
        conduction_step(segs, conduction_delay=1, dbg=None, step=0)  # pass None => no logs
        # ion channels
        for s in segs:
            # minimal
            if hpc_compare(s.wave_state, s.ca_threshold)>0:
                plus_val= hpc_shr(s.wave_state,4)
                s.wave_state= hpc_add(s.wave_state, plus_val)
            if hpc_compare(s.wave_state, s.k_threshold)>0:
                minus_val= hpc_shr(s.wave_state,3)
                s.wave_state= hpc_sub(s.wave_state, minus_val)
        # reflection
        left= segs[0]
        right= segs[-1]
        subL= hpc_shr(left.wave_state,4)
        left.wave_state= hpc_sub(left.wave_state, subL)
        subR= hpc_shr(right.wave_state,4)
        right.wave_state= hpc_sub(right.wave_state, subR)
    t1= time.perf_counter()
    dur= (t1-t0)
    # measure final wave => sum
    final_sum= 0
    for s in segs:
        final_sum += limbs_to_int(s.wave_state)
    return dur, final_sum

###############################################################################
# Plot function
###############################################################################

def plot_debug_data(segs, dbg:HPCDebugData, steps=50):
    """
    multiple subplots => wave_final in mV, velocity, conduction_in,
    reflection_sub, Ca bits, K bits, final distribution, etc.
    """
    num_segs= len(segs)
    time_axis= np.arange(steps)

    # We'll store arrays
    wave_mV  = np.zeros((steps, num_segs), dtype=float)
    velocity = np.zeros((steps, num_segs), dtype=float)
    conduction_in= np.zeros((steps, num_segs), dtype=float)
    reflection_sub= np.zeros((steps, num_segs), dtype=float)
    ca_bits= np.zeros((steps, num_segs), dtype=float)
    k_bits=  np.zeros((steps, num_segs), dtype=float)

    for step in range(steps):
        for seg_i in range(num_segs):
            wv_final= dbg.fetch(step, seg_i, "wave_final", 0)
            wave_limb_final= int_to_limbs(wv_final)
            wave_mV[step, seg_i]= HPCWave_to_mV(wave_limb_final, segs[seg_i].wave_max)
            conduction_in[step, seg_i]= dbg.fetch(step, seg_i, "conduction_in", 0)
            reflection_sub[step, seg_i]= dbg.fetch(step, seg_i, "reflection_sub", 0)
            ca_bits[step, seg_i]= dbg.fetch(step, seg_i, "ca_bits", 0)
            k_bits[step, seg_i]= dbg.fetch(step, seg_i, "k_bits", 0)
            velocity[step, seg_i]= dbg.fetch(step, seg_i, "velocity", 0)

    fig, axes= plt.subplots(3,2, figsize=(16,14))
    ax1, ax2, ax3, ax4, ax5, ax6= axes.flatten()

    # 1) wave_mV line
    for seg_i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:, seg_i], label=f"Seg {seg_i}")
    ax1.set_title("HPC-limb wave => mV (final after reflection)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax1.grid(True)

    # 2) conduction_in
    for seg_i in range(num_segs):
        ax2.plot(time_axis, conduction_in[:, seg_i], label=f"Seg {seg_i}")
    ax2.set_title("Conduction In (HPC-limb int)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) reflection_sub
    for seg_i in range(num_segs):
        ax3.plot(time_axis, reflection_sub[:, seg_i], label=f"Seg {seg_i}")
    ax3.set_title("Reflection Sub at Boundaries (HPC-limb int)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude")
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax3.grid(True)

    # 4) Ca bits, K bits
    # overlay them => or separate? We'll do overlay
    for seg_i in range(num_segs):
        ax4.plot(time_axis, ca_bits[:, seg_i], label=f"S{seg_i} Ca", alpha=0.7)
        ax4.plot(time_axis, k_bits[:, seg_i], '--', label=f"S{seg_i} K", alpha=0.7)
    ax4.set_title("Ca bits (solid) & K bits (dashed)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("bits (0..4095)")
    ax4.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax4.grid(True)

    # 5) velocity
    # sum or line per segment
    # we'll do line per segment + total wave velocity
    total_vel= np.sum(velocity, axis=1)
    for seg_i in range(num_segs):
        ax5.plot(time_axis, velocity[:, seg_i], alpha=0.4)
    ax5.plot(time_axis, total_vel, 'k-', linewidth=2, label="Sum velocity(all segs)")
    ax5.set_title("Velocity = wave(t) - wave(t-1) per segment, sum in black")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Velocity (int HPC-limb)")
    ax5.grid(True)
    ax5.legend()

    # 6) final distribution => wave_mV at last step
    final_mV= wave_mV[-1,:]
    ax6.hist(final_mV, bins=30, alpha=0.7, color='orange')
    ax6.set_title("Final HPC-limb wave distribution (mV)")
    ax6.set_xlabel("Voltage (mV)")
    ax6.set_ylabel("Count")
    ax6.grid(True)

    plt.tight_layout()
    return fig

###############################################################################
# main demo
###############################################################################

def main():
    # 1) Demo with 8 segments, 50 steps => produce debug + plots
    segs_dbg, debug_data= run_hpc_wave_debug(num_segments=8, steps=50)
    fig= plot_debug_data(segs_dbg, debug_data, steps=50)
    plt.show()

    # 2) Stress test => 3000 segments, 20 steps => measure performance
    print("Running HPC-limb wave stress test with 3000 segments, 20 steps...")
    import time
    t0= time.perf_counter()
    dur, final_sum= run_hpc_wave_stress_test(num_segments=3000, steps=20)
    t1= time.perf_counter()
    print(f"Stress test took {dur:.4f}s for update loop, total real time {t1-t0:.4f}s")
    print(f"Final sum of wave states => {final_sum}")

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import math, random
from array import array
import numpy as np
import matplotlib.pyplot as plt

###############################################################################
# 1) HPC-limb code (64-bit) - minimal version
###############################################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """Convert nonnegative Python int -> HPC-limb array of 64-bit 'Q'."""
    if value < 0:
        raise ValueError("Negative ints not supported here.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val=0
    shift=0
    for limb in limbs:
        val += (limb << shift)
        shift+= CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q', [0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry  = s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av = A[i] if i<len(A) else 0
        bv = B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff+= CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    # remove trailing zeros
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb= len(A), len(B)
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts=  shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out: out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<< bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

###############################################################################
# HPC-limb wave => approximate mV
###############################################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array, range_mV=40.0, rest_mV=-70.0)->float:
    val = limbs_to_int(wave_limb)
    vmax= limbs_to_int(wave_max)
    fraction= val/vmax if vmax>0 else 0.0
    return rest_mV + range_mV*fraction

###############################################################################
# HPCWaveSegment + HPC-limb wave logic
###############################################################################

class HPCWaveSegment:
    """
    HPC-limb wave segment with conduction backlog, threshold logic,
    plus a "prev_wave_int" for velocity if desired.
    """
    def __init__(self, wave_max_val=1_000_000, start_val=10_000):
        self.wave_max = int_to_limbs(wave_max_val)
        self.wave_state= int_to_limbs(start_val)
        # threshold e.g. 2% & 3%
        ca_thr = int(wave_max_val*0.02)
        k_thr  = int(wave_max_val*0.03)
        self.ca_threshold= int_to_limbs(ca_thr)
        self.k_threshold=  int_to_limbs(k_thr)

        self.toggle_history= []
        self.conduction_amplitude_history= []
        self.prev_wave_int= start_val  # for velocity

###############################################################################
# HPCDebugData for storing step-by-step details
###############################################################################

class HPCDebugData:
    def __init__(self):
        self.data = {}

    def record_val(self, step:int, seg_idx:int, key:str, val:int):
        if step not in self.data:
            self.data[step] = {}
        if seg_idx not in self.data[step]:
            self.data[step][seg_idx] = {}
        self.data[step][seg_idx][key] = val

    def fetch(self, step:int, seg_idx:int, key:str, default=0)->int:
        return self.data.get(step, {}).get(seg_idx, {}).get(key, default)

###############################################################################
# random + harmonic input
###############################################################################

def random_harmonic_input(t:int, seg: HPCWaveSegment, freq_list=None):
    """Sum of sin waves + random noise => HPC-limb int => add to wave_state."""
    if freq_list is None:
        freq_list=[0.05, 0.10]
    total=0
    for freq in freq_list:
        phase= 2*math.pi*freq*t
        val_f= 1000.0*math.sin(phase)
        total += int(val_f)
    # random noise
    noise= random.randint(-50,50)
    total += noise
    if total<0: total=0
    seg.wave_state= hpc_add(seg.wave_state, int_to_limbs(total))

###############################################################################
# conduction, ion channels, partial reflection, voltage leak
###############################################################################

def conduction_step(segments, conduction_delay:int, dbg:HPCDebugData, step:int):
    """
    wave//8 => pass => toggles => HPC-limb
    record conduction_in, conduction_out if dbg not None
    """
    n= len(segments)
    zeroA= int_to_limbs(0)
    toggles= [zeroA]*n
    conduction_vals= [zeroA]*n

    for i, seg in enumerate(segments):
        conduction= hpc_shr(seg.wave_state,3) # wave//8
        conduction_vals[i]= conduction
        if dbg is not None:
            dbg.record_val(step, i, "conduction_out", limbs_to_int(conduction))
    # pass conduction to neighbors
    for i, seg in enumerate(segments):
        cval= conduction_vals[i]
        if hpc_compare(cval, zeroA)>0:
            if i>0:
                toggles[i-1]= cval
            if i<n-1:
                toggles[i+1]= cval

    # apply conduction w/ conduction_delay
    for i, seg in enumerate(segments):
        if len(seg.toggle_history)>= conduction_delay:
            inflow= seg.toggle_history.pop(0)
            seg.wave_state= hpc_add(seg.wave_state, inflow)
            if dbg is not None:
                dbg.record_val(step, i, "conduction_in", limbs_to_int(inflow))
        else:
            if dbg is not None:
                dbg.record_val(step, i, "conduction_in", 0)
        seg.toggle_history.append(toggles[i])

def ion_channels(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_idx:int):
    """partial Ca push => wave += wave//16 if wave>ca, partial K pull => wave-= wave//8 if wave>k"""
    wave_int_pre= limbs_to_int(seg.wave_state)
    ca_b= wave_int_pre & 0xFFF
    k_b = (wave_int_pre >>12)& 0xFFF
    if dbg is not None:
        dbg.record_val(step, seg_idx, "ca_bits", ca_b)
        dbg.record_val(step, seg_idx, "k_bits", k_b)

    # Ca push
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4) # wave//16
        seg.wave_state= hpc_add(seg.wave_state, plus_val)

    # K pull
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3) # wave//8
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

def voltage_leak(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_idx:int, leak_shift=5):
    """
    HPC-limb leak => wave_state -= wave_state//(2^leak_shift).
    e.g. wave_state//32 => ~3% leak each step if leak_shift=5.
    """
    wave_pre_int= limbs_to_int(seg.wave_state)
    if wave_pre_int>0:
        sub_val= hpc_shr(seg.wave_state, leak_shift)
        seg.wave_state= hpc_sub(seg.wave_state, sub_val)
        if dbg is not None:
            dbg.record_val(step, seg_idx, "leak_sub", limbs_to_int(sub_val))

def partial_reflection(segments, dbg:HPCDebugData, step:int):
    """wave -= wave//16 at boundaries => HPC-limb reflection"""
    if not segments:
        return
    left= segments[0]
    right= segments[-1]
    subL= hpc_shr(left.wave_state,4)
    left.wave_state= hpc_sub(left.wave_state, subL)
    if dbg is not None:
        dbg.record_val(step, 0, "reflection_sub", limbs_to_int(subL))
    subR= hpc_shr(right.wave_state,4)
    right.wave_state= hpc_sub(right.wave_state, subR)
    if dbg is not None:
        dbg.record_val(step, len(segments)-1, "reflection_sub", limbs_to_int(subR))

def compute_velocity(seg: HPCWaveSegment):
    curr_int= limbs_to_int(seg.wave_state)
    vel= curr_int - seg.prev_wave_int
    seg.prev_wave_int= curr_int
    return vel

###############################################################################
# update step
###############################################################################

def update_hpc_wave(segments, step:int, dbg:HPCDebugData, leak_shift=5):
    # record wave_pre
    for i, seg in enumerate(segments):
        wv_int= limbs_to_int(seg.wave_state)
        if dbg is not None:
            dbg.record_val(step, i, "wave_pre", wv_int)

    # inject random/harmonic wave => seg0
    random_harmonic_input(step, segments[0])

    # conduction
    conduction_step(segments, conduction_delay=1, dbg=dbg, step=step)

    # ion channels
    for i, seg in enumerate(segments):
        ion_channels(seg, dbg, step, i)

    # voltage leak => wave -= wave//(2^leak_shift)
    for i, seg in enumerate(segments):
        voltage_leak(seg, dbg, step, i, leak_shift=leak_shift)

    # wave_postIon
    if dbg is not None:
        for i, seg in enumerate(segments):
            wv_post= limbs_to_int(seg.wave_state)
            dbg.record_val(step, i, "wave_postIon", wv_post)

    # reflection
    partial_reflection(segments, dbg, step)

    # final wave + velocity
    for i, seg in enumerate(segments):
        wv_final= limbs_to_int(seg.wave_state)
        if dbg is not None:
            dbg.record_val(step, i, "wave_final", wv_final)
        vel= compute_velocity(seg)
        if dbg is not None:
            dbg.record_val(step, i, "velocity", vel)

###############################################################################
# run with debug
###############################################################################

def run_hpc_wave_debug(num_segments=8, steps=50, leak_shift=5):
    segs= []
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    # artificially excite seg0 => wave_state= 20k => 2%
    segs[0].wave_state= int_to_limbs(20_000)
    dbg= HPCDebugData()
    for step in range(steps):
        update_hpc_wave(segs, step, dbg, leak_shift=leak_shift)
    return segs, dbg

###############################################################################
# Stress test => 3000 segments
###############################################################################

def run_hpc_wave_stress_test(num_segments=3000, steps=20, leak_shift=5):
    import time
    segs=[]
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    # seg0 => wave_state=20k
    segs[0].wave_state= int_to_limbs(20_000)
    t0= time.perf_counter()
    for step in range(steps):
        # minimal => skip debug
        # conduction
        conduction_step(segs, conduction_delay=1, dbg=None, step=step)
        # ion channels
        for s in segs:
            # partial Ca => wave//16
            if hpc_compare(s.wave_state, s.ca_threshold)>0:
                plus_val= hpc_shr(s.wave_state,4)
                s.wave_state= hpc_add(s.wave_state, plus_val)
            # partial K => wave//8
            if hpc_compare(s.wave_state, s.k_threshold)>0:
                minus_val= hpc_shr(s.wave_state,3)
                s.wave_state= hpc_sub(s.wave_state, minus_val)
            # leak => wave//(2^leak_shift)
            if limbs_to_int(s.wave_state)>0:
                leak_val= hpc_shr(s.wave_state, leak_shift)
                s.wave_state= hpc_sub(s.wave_state, leak_val)
        # reflection
        left= segs[0]
        right= segs[-1]
        subL= hpc_shr(left.wave_state,4)
        left.wave_state= hpc_sub(left.wave_state, subL)
        subR= hpc_shr(right.wave_state,4)
        right.wave_state= hpc_sub(right.wave_state, subR)
    t1= time.perf_counter()
    dur= (t1-t0)
    final_sum=0
    for s in segs:
        final_sum += limbs_to_int(s.wave_state)
    return dur, final_sum

###############################################################################
# Plot debug data => velocity, conduction_in, reflection_sub, Ca bits, K bits
###############################################################################

def plot_debug_data(segs, dbg:HPCDebugData, steps=50):
    num_segs= len(segs)
    time_axis= np.arange(steps)

    # We'll store arrays
    wave_mV  = np.zeros((steps, num_segs), dtype=float)
    velocity = np.zeros((steps, num_segs), dtype=float)
    conduction_in= np.zeros((steps, num_segs), dtype=float)
    reflection_sub= np.zeros((steps, num_segs), dtype=float)
    ca_bits= np.zeros((steps, num_segs), dtype=float)
    k_bits=  np.zeros((steps, num_segs), dtype=float)

    for step in range(steps):
        for seg_i in range(num_segs):
            wv_final= dbg.fetch(step, seg_i, "wave_final", 0)
            wave_limb_final= int_to_limbs(wv_final)
            wave_mV[step, seg_i]= HPCWave_to_mV(wave_limb_final, segs[seg_i].wave_max)
            conduction_in[step, seg_i]= dbg.fetch(step, seg_i, "conduction_in", 0)
            reflection_sub[step, seg_i]= dbg.fetch(step, seg_i, "reflection_sub", 0)
            ca_bits[step, seg_i]= dbg.fetch(step, seg_i, "ca_bits", 0)
            k_bits[step, seg_i]= dbg.fetch(step, seg_i, "k_bits", 0)
            velocity[step, seg_i]= dbg.fetch(step, seg_i, "velocity", 0)

    fig, axes= plt.subplots(3,2, figsize=(16,14))
    ax1, ax2, ax3, ax4, ax5, ax6= axes.flatten()

    # 1) wave_mV line
    for seg_i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:,seg_i], label=f"Seg {seg_i}")
    ax1.set_title("HPC-limb wave => mV (final after reflection + leak)")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax1.grid(True)

    # 2) conduction_in
    for seg_i in range(num_segs):
        ax2.plot(time_axis, conduction_in[:, seg_i], label=f"Seg {seg_i}")
    ax2.set_title("Conduction In (HPC-limb int)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude")
    ax2.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax2.grid(True)

    # 3) reflection_sub
    for seg_i in range(num_segs):
        ax3.plot(time_axis, reflection_sub[:, seg_i], label=f"Seg {seg_i}")
    ax3.set_title("Reflection Sub at Boundaries (HPC-limb int)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude")
    ax3.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax3.grid(True)

    # 4) Ca bits, K bits
    for seg_i in range(num_segs):
        ax4.plot(time_axis, ca_bits[:, seg_i], label=f"S{seg_i} Ca", alpha=0.7)
        ax4.plot(time_axis, k_bits[:, seg_i], '--', label=f"S{seg_i} K", alpha=0.7)
    ax4.set_title("Ca bits (solid) & K bits (dashed)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("bits (0..4095)")
    ax4.legend(bbox_to_anchor=(1.05,1), loc="upper left")
    ax4.grid(True)

    # 5) velocity
    # sum across segments
    total_vel= np.sum(velocity, axis=1)
    for seg_i in range(num_segs):
        ax5.plot(time_axis, velocity[:, seg_i], alpha=0.4)
    ax5.plot(time_axis, total_vel, 'k-', linewidth=2, label="Sum velocity(all segs)")
    ax5.set_title("Velocity per segment, sum in black")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Velocity (int HPC-limb)")
    ax5.grid(True)
    ax5.legend()

    # 6) final distribution => wave_mV at last step
    final_mV= wave_mV[-1,:]
    ax6.hist(final_mV, bins=30, alpha=0.7, color='orange')
    ax6.set_title("Final HPC-limb wave distribution (mV)")
    ax6.set_xlabel("Voltage (mV)")
    ax6.set_ylabel("Count")
    ax6.grid(True)

    plt.tight_layout()
    return fig

###############################################################################
# main
###############################################################################

def main():
    # small run
    segs_dbg, dbg= run_hpc_wave_debug(num_segments=8, steps=50, leak_shift=5)
    fig= plot_debug_data(segs_dbg, dbg, steps=50)
    plt.show()

    # stress test 3000 segs
    import time
    print("Running HPC-limb wave stress test with 3000 segments, 20 steps (with leak).")
    t0= time.perf_counter()
    dur, final_sum= run_hpc_wave_stress_test(num_segments=3000, steps=20, leak_shift=5)
    t1= time.perf_counter()
    print(f"Stress test partial conduction + leak took {t1-t0:.4f}s total; wave update => {dur:.4f}s")
    print(f"Final sum of wave states => {final_sum}")

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

import math, random
from array import array
import numpy as np
import matplotlib.pyplot as plt
import time

###############################################################################
# 1) HPC-limb code (64-bit chunks)
###############################################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """Convert nonnegative Python int -> HPC-limb array of 64-bit 'Q'."""
    if value < 0:
        raise ValueError("Negative ints not supported here.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    """Combine HPC-limb array -> Python int."""
    val= 0
    shift= 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av= A[i] if i<len(A) else 0
        bv= B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry= s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av= A[i] if i<len(A) else 0
        bv= B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff+= CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb= len(A), len(B)
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    """Right shift HPC-limb array by shift_bits bits total."""
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts=  shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<< bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

###############################################################################
# 2) HPC-limb wave => approximate mV in [-70..+40] => 110 mV total swing
###############################################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array)->float:
    """
    Map wave_state in [0..wave_max] => [-70..+40], i.e. 110 mV total amplitude
    voltage = -70 + 110*(wave_state / wave_max).
    """
    val = limbs_to_int(wave_limb)
    vmax= limbs_to_int(wave_max)
    if vmax<=0:
        return -70.0
    fraction= val / vmax
    return -70.0 + 110.0*fraction

###############################################################################
# HPCWaveSegment => sim c.elegans references
###############################################################################

class HPCWaveSegment:
    """
    HPC-limb wave segment for c.elegans-like logic:
      - CaThreshold ~2% wave_max
      - KThreshold ~3% wave_max
      - leak => wave -= wave//32
      - conduction => wave//8 to neighbors
      - reflection => wave -= wave//16 at boundaries
    """
    def __init__(self, wave_max_val=1_000_000, start_val=10_000):
        self.wave_max = int_to_limbs(wave_max_val)
        self.wave_state= int_to_limbs(start_val)
        # Ca threshold => 2% => egl-19
        ca_thr= int(wave_max_val*0.02)
        # K threshold => 3% => sK, slo-2
        k_thr= int(wave_max_val*0.03)
        self.ca_threshold= int_to_limbs(ca_thr)
        self.k_threshold=  int_to_limbs(k_thr)

        self.toggle_history= []
        self.conduction_amplitude_history= []
        # For velocity calc
        self.prev_int= start_val

###############################################################################
# HPCDebugData => store step-by-step
###############################################################################

class HPCDebugData:
    def __init__(self):
        self.data= {}

    def record_val(self, step:int, seg_i:int, key:str, val:int):
        if step not in self.data:
            self.data[step] = {}
        if seg_i not in self.data[step]:
            self.data[step][seg_i]= {}
        self.data[step][seg_i][key]= val

    def fetch(self, step:int, seg_i:int, key:str, default=0)->int:
        return self.data.get(step, {}).get(seg_i, {}).get(key, default)

###############################################################################
# 3) Input => random/harmonic injection
###############################################################################

def random_harmonic_input(t:int, seg: HPCWaveSegment, freq_list=None):
    """ sum sin waves + small noise => HPC-limb => add to wave_state """
    if freq_list is None:
        freq_list=[0.05, 0.10]
    total=0
    for freq in freq_list:
        phase= 2*math.pi*freq*t
        val_f= 1000.0*math.sin(phase)
        total += int(val_f)
    # random noise
    noise= random.randint(-50,50)
    total+= noise
    if total<0: total=0
    seg.wave_state= hpc_add(seg.wave_state, int_to_limbs(total))

###############################################################################
# conduction => wave//8 => pass => 1-step backlog
###############################################################################

def conduction_step(segments, conduction_delay:int, dbg:HPCDebugData, step:int):
    n= len(segments)
    zeroA= int_to_limbs(0)
    toggles= [zeroA]*n
    conduction_vals= [zeroA]*n

    for i, seg in enumerate(segments):
        conduction= hpc_shr(seg.wave_state,3)  # wave//8
        conduction_vals[i]= conduction
        if dbg:
            dbg.record_val(step, i, "cond_out", limbs_to_int(conduction))

    # pass conduction => neighbors
    for i, seg in enumerate(segments):
        cval= conduction_vals[i]
        if hpc_compare(cval, zeroA)>0:
            if i>0:
                toggles[i-1]= cval
            if i<n-1:
                toggles[i+1]= cval

    # apply conduction w/delay=1
    for i, seg in enumerate(segments):
        if len(seg.toggle_history)>= conduction_delay:
            inflow= seg.toggle_history.pop(0)
            seg.wave_state= hpc_add(seg.wave_state, inflow)
            if dbg:
                dbg.record_val(step, i, "cond_in", limbs_to_int(inflow))
        else:
            if dbg:
                dbg.record_val(step, i, "cond_in", 0)
        seg.toggle_history.append(toggles[i])

###############################################################################
# Ion channels => Ca if wave>2%, K if wave>3%
###############################################################################

def ion_channels(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_i:int):
    wave_int_pre= limbs_to_int(seg.wave_state)
    if dbg:
        # record bits for debug
        cab= wave_int_pre & 0xFFF
        kb= (wave_int_pre>>12)&0xFFF
        dbg.record_val(step, seg_i, "ca_bits", cab)
        dbg.record_val(step, seg_i, "k_bits", kb)

    # Ca push => wave += wave//16 if wave>ca_threshold
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4) # wave//16
        seg.wave_state= hpc_add(seg.wave_state, plus_val)

    # K pull => wave -= wave//8 if wave>k_threshold
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3) # wave//8
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

###############################################################################
# 4) TWK-7 leak => wave -= wave//32
###############################################################################

def leak_current(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_i:int):
    wave_pre= limbs_to_int(seg.wave_state)
    if wave_pre>0:
        leak_val= hpc_shr(seg.wave_state,5)  # wave//32 => ~3%
        seg.wave_state= hpc_sub(seg.wave_state, leak_val)
        if dbg:
            dbg.record_val(step, seg_i, "leak_sub", limbs_to_int(leak_val))

###############################################################################
# partial reflection => wave -= wave//16 at boundaries
###############################################################################

def partial_reflection(segments, dbg:HPCDebugData, step:int):
    if not segments:
        return
    left= segments[0]
    right= segments[-1]

    subL= hpc_shr(left.wave_state,4)  # wave//16
    left.wave_state= hpc_sub(left.wave_state, subL)
    if dbg:
        dbg.record_val(step, 0, "reflect_sub", limbs_to_int(subL))

    subR= hpc_shr(right.wave_state,4)
    right.wave_state= hpc_sub(right.wave_state, subR)
    if dbg:
        dbg.record_val(step, len(segments)-1, "reflect_sub", limbs_to_int(subR))

###############################################################################
# velocity => wave_final - wave_prev
###############################################################################

def compute_velocity(seg: HPCWaveSegment):
    curr_int= limbs_to_int(seg.wave_state)
    vel= curr_int- seg.prev_int
    seg.prev_int= curr_int
    return vel

###############################################################################
# update step => conduction -> Ca/K -> leak -> reflection
###############################################################################

def update_hpc_wave(segments, dbg:HPCDebugData, step:int):
    # record wave_pre
    for i, seg in enumerate(segments):
        wave_pre= limbs_to_int(seg.wave_state)
        if dbg:
            dbg.record_val(step, i, "wave_pre", wave_pre)

    # random input => seg0
    random_harmonic_input(step, segments[0])

    # conduction
    conduction_step(segments, conduction_delay=1, dbg=dbg, step=step)

    # ion channels
    for i, seg in enumerate(segments):
        ion_channels(seg, dbg, step, i)

    # leak
    for i, seg in enumerate(segments):
        leak_current(seg, dbg, step, i)

    # wave_postIon
    if dbg:
        for i, seg in enumerate(segments):
            wv_post= limbs_to_int(seg.wave_state)
            dbg.record_val(step, i, "wave_postIon", wv_post)

    # reflection
    partial_reflection(segments, dbg, step)

    # final wave + velocity
    for i, seg in enumerate(segments):
        wv_final= limbs_to_int(seg.wave_state)
        if dbg:
            dbg.record_val(step, i, "wave_final", wv_final)
            vel= compute_velocity(seg)
            dbg.record_val(step, i, "velocity", vel)
        else:
            compute_velocity(seg)  # not storing

###############################################################################
# run small => debug + plots
###############################################################################

def run_c_elegans_small(num_segments=8, steps=50):
    segs= []
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    # seg0 => 2% => 20k
    segs[0].wave_state= int_to_limbs(20_000)

    dbg= HPCDebugData()
    for step in range(steps):
        update_hpc_wave(segs, dbg, step)
    return segs, dbg

###############################################################################
# run large => stress test
###############################################################################

def run_c_elegans_stress_test(num_segments=3000, steps=20):
    import time
    segs=[]
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    segs[0].wave_state= int_to_limbs(20_000)

    t0= time.perf_counter()
    for step in range(steps):
        # minimal => skip debug
        # conduction
        conduction_step(segs, conduction_delay=1, dbg=None, step=step)
        # ion channels
        for s in segs:
            # Ca if wave>2%
            if hpc_compare(s.wave_state, s.ca_threshold)>0:
                plus_val= hpc_shr(s.wave_state,4)
                s.wave_state= hpc_add(s.wave_state, plus_val)
            # K if wave>3%
            if hpc_compare(s.wave_state, s.k_threshold)>0:
                minus_val= hpc_shr(s.wave_state,3)
                s.wave_state= hpc_sub(s.wave_state, minus_val)
            # leak => wave//32
            wave_pre= limbs_to_int(s.wave_state)
            if wave_pre>0:
                leak_val= hpc_shr(s.wave_state,5)
                s.wave_state= hpc_sub(s.wave_state, leak_val)
        # reflection
        left= segs[0]
        right= segs[-1]
        subL= hpc_shr(left.wave_state,4)
        left.wave_state= hpc_sub(left.wave_state, subL)
        subR= hpc_shr(right.wave_state,4)
        right.wave_state= hpc_sub(right.wave_state, subR)
        # velocity => skip storing, just do compute
        for s in segs:
            cur_int= limbs_to_int(s.wave_state)
            _vel= cur_int- s.prev_int
            s.prev_int= cur_int
    t1= time.perf_counter()
    dur= (t1 - t0)
    # final sum
    final_sum=0
    for s in segs:
        final_sum+= limbs_to_int(s.wave_state)
    return dur, final_sum

###############################################################################
# plot => HPCWave_to_mV => c.elegans range
###############################################################################

def plot_debug_data_c_elegans(segs, dbg:HPCDebugData, steps=50):
    num_segs= len(segs)
    time_axis= np.arange(steps)

    wave_mV = np.zeros((steps, num_segs), dtype=float)
    velocity= np.zeros((steps, num_segs), dtype=float)
    conduction_in= np.zeros((steps, num_segs), dtype=float)
    reflect_sub= np.zeros((steps, num_segs), dtype=float)
    ca_bits= np.zeros((steps, num_segs), dtype=float)
    k_bits= np.zeros((steps, num_segs), dtype=float)

    for step in range(steps):
        for i in range(num_segs):
            wv_final= dbg.fetch(step, i, "wave_final", 0)
            wave_limb= int_to_limbs(wv_final)
            wave_mV[step, i]= HPCWave_to_mV(wave_limb, segs[i].wave_max)

            conduction_in[step, i]= dbg.fetch(step, i, "cond_in", 0)
            reflect_sub[step, i]= dbg.fetch(step, i, "reflect_sub", 0)
            ca_bits[step, i]= dbg.fetch(step, i, "ca_bits", 0)
            k_bits[step, i]= dbg.fetch(step, i, "k_bits", 0)
            velocity[step, i]= dbg.fetch(step, i, "velocity", 0)

    fig, axes= plt.subplots(3,2, figsize=(16,14))
    ax1, ax2, ax3, ax4, ax5, ax6= axes.flatten()

    # HPC-limb wave => mV
    for i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:, i], label=f"S{i}")
    ax1.set_title("C. elegans HPC-limb wave => [-70..+40] mV")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.grid(True)
    ax1.legend()

    # conduction_in
    for i in range(num_segs):
        ax2.plot(time_axis, conduction_in[:, i], alpha=0.7)
    ax2.set_title("Conduction In (wave//8) from Neighbors")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude (int)")
    ax2.grid(True)

    # reflection sub
    for i in range(num_segs):
        ax3.plot(time_axis, reflect_sub[:, i], alpha=0.7)
    ax3.set_title("Reflection Sub at Boundaries (wave//16)")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude (int)")
    ax3.grid(True)

    # Ca bits (solid) & K bits (dashed)
    for i in range(num_segs):
        ax4.plot(time_axis, ca_bits[:, i], alpha=0.7)
        ax4.plot(time_axis, k_bits[:, i], '--', alpha=0.7)
    ax4.set_title("Ca & K Bits from HPC-limb State")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Bit pattern (0..4095)")
    ax4.grid(True)

    # velocity => sum in black
    total_vel= np.sum(velocity, axis=1)
    for i in range(num_segs):
        ax5.plot(time_axis, velocity[:, i], alpha=0.4)
    ax5.plot(time_axis, total_vel, 'k-', linewidth=2, label="Sum velocity")
    ax5.set_title("Velocity per Segment => wave(t)-wave(t-1)")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Velocity (int HPC-limb)")
    ax5.grid(True)
    ax5.legend()

    # final distribution => wave_mV at last step
    final_mV= wave_mV[-1,:]
    ax6.hist(final_mV, bins=30, alpha=0.7, color='orange')
    ax6.set_title("Final HPC-limb wave distribution (mV)")
    ax6.set_xlabel("Voltage (mV)")
    ax6.set_ylabel("Count")
    ax6.grid(True)

    plt.tight_layout()
    return fig

###############################################################################
# main
###############################################################################

def main():
    print("1) Running small c.elegans HPC-limb wave with debug & plot (8 segments, 50 steps)")
    segs_small, dbg_small= run_c_elegans_small(num_segments=8, steps=50)
    fig= plot_debug_data_c_elegans(segs_small, dbg_small, steps=50)
    plt.show()

    print("\n2) Running HPC-limb wave stress test with 3000 segments, 20 steps...\n")
    t0= time.perf_counter()
    dur, final_sum= run_c_elegans_stress_test(num_segments=3000, steps=20)
    t1= time.perf_counter()
    print(f"Stress test code time => {t1-t0:.4f}s   wave update => {dur:.4f}s")
    print(f"Final sum of wave states => {final_sum}")

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3

"""
C. elegans-Inspired HPC-Limb Model

This code simulates a chain of neuron/muscle segments in discrete time,
using HPC-limb integer arithmetic to represent membrane potentials from
-70 mV to +40 mV (110 mV total range). Biologically, we approximate:

1) **EGL-19 (Ca2+ Inward Current)**:
   If the membrane crosses a certain threshold (~2% of wave_max),
   HPC-limb partial increment => wave += wave//16.

2) **SHK-1, SLO-2 (K+ Outward Current)**:
   If above a higher threshold (~3%), HPC-limb partial decrement => wave -= wave//8,
   modeling repolarization.

3) **TWK-7 (Leak Channel)**:
   Even below threshold, HPC-limb wave leaks by wave//32 each step (3%),
   preventing indefinite amplitude growth, mimicking stable resting potential.

4) **Gap-Junction Conduction**:
   Each segment passes wave//8 to neighbors with a 1-step backlog
   (“electrical coupling”). Larger conduction => faster spread of activity.

5) **Boundary Reflection**:
   The two ends lose wave//16 each step, representing partial sealing
   or impedance mismatch at the edges, causing partial wave reflection.

6) **Random/Harmonic Input**:
   Each step, segment 0 receives a small integer injection from a sum of sine waves
   plus random noise, modeling external drive or synaptic input.

We keep track of “velocity” = wave(t) - wave(t-1). Positive => net depolarization,
negative => net repolarization, possibly “backward traveling” wave.

References:
- EGL-19 is the major L-type Ca channel in worms (inward current).
- SHK-1, SLO-2, etc. are K+ channels for repolarizing outflow.
- TWK-7 is a K+ leak channel maintaining negative resting potential.
"""

import math
import random
import time
import mpmath  # for high-precision sine
from array import array
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Dict
mpmath.mp.prec = 200  # e.g. 200 bits of precision

###############################################################################
# 1) HPC-limb code (64-bit integer chunks)
###############################################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """Nonnegative Python int -> HPC-limb array (64-bit)."""
    if value<0:
        raise ValueError("Negative ints not supported.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val=0
    shift=0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av= A[i] if i<len(A) else 0
        bv= B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry= s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av= A[i] if i<len(A) else 0
        bv= B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb= len(A), len(B)
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts=  shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

###############################################################################
# B) HPC-limb fraction => HPCFraction
###############################################################################

class HPCFraction:
    """
    HPC-limb fraction => numerator, denominator each HPC-limb array (nonnegative).
    We store sign separately if needed, but let's keep it positive for angles.
    """
    __slots__ = ('num','den')

    def __init__(self, numerator: array, denominator: array):
        # assume gcd? or we can do it if we want
        if len(denominator)==1 and denominator[0]==0:
            raise ZeroDivisionError("Fraction denominator=0")
        self.num= numerator
        self.den= denominator

    @classmethod
    def from_int(cls, val: int) -> "HPCFraction":
        if val<0:
            raise ValueError("Negative HPCFraction not done in snippet.")
        return cls(int_to_limbs(val), int_to_limbs(1))

    def copy(self)->"HPCFraction":
        return HPCFraction(array('Q',self.num), array('Q',self.den))

def hpcf_add(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    """(A.num/A.den) + (B.num/B.den) => HPC-limb fraction."""
    # cross => A.num*B.den + B.num*A.den
    an= hpc_mul(A.num, B.den)
    bn= hpc_mul(B.num, A.den)
    ssum= hpc_add(an, bn)
    dden= hpc_mul(A.den, B.den)
    return HPCFraction(ssum, dden)

def hpcf_sub(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    an= hpc_mul(A.num, B.den)
    bn= hpc_mul(B.num, A.den)
    diff= hpc_sub(an, bn)
    dden= hpc_mul(A.den, B.den)
    # skip sign check for snippet
    return HPCFraction(diff, dden)

def hpcf_mul(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    nump= hpc_mul(A.num, B.num)
    denp= hpc_mul(A.den, B.den)
    return HPCFraction(nump, denp)

def hpcf_div(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    if len(B.num)==1 and B.num[0]==0:
        raise ZeroDivisionError("Divide fraction by 0 HPC-limb")
    nump= hpc_mul(A.num, B.den)
    denp= hpc_mul(A.den, B.num)
    return HPCFraction(nump, denp)

def hpc_mul(A: array, B: array)-> array:
    """Schoolbook HPC-limb multiply => array of size <= len(A)+len(B)."""
    la, lb= len(A), len(B)
    out= array('Q',[0]*(la+lb))
    for i in range(la):
        carry=0
        av= A[i]
        for j in range(lb):
            mul_val= av*B[j] + out[i+j] + carry
            out[i+j]= mul_val & CHUNK_MASK
            carry= mul_val >> CHUNK_BITS
        if carry:
            out[i+lb]+= carry
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpcf_to_mpmath(A: HPCFraction)-> mpmath.mpf:
    """
    Convert HPCFraction => mpmath.mpf for trig, etc.
    """
    an= limbs_to_int(A.num)
    ad= limbs_to_int(A.den)
    return mpmath.mpf(an)/mpmath.mpf(ad)

###############################################################################
# C) HPC-limb-based angle => caching for sin
###############################################################################

_sin_cache: Dict[Tuple[int,int], int] = {}  
# key = (num_int, den_int), value= HPC-limb int amplitude => e.g. scaled sin

def hpcf_sin(angle: HPCFraction, amplitude: int=10000)-> array:
    """
    Return HPC-limb integer for sin(angle * pi?), depending on how you define angle.
    We'll do sin(angle) in radians if angle is fraction of rad.
    amplitude => scale final => HPC-limb integer.

    We do caching:
      key = (num_of_angle, den_of_angle) => HPC-limb int
    """
    an= limbs_to_int(angle.num)
    ad= limbs_to_int(angle.den)
    key= (an, ad, amplitude)
    if key in _sin_cache:
        # Already computed
        ival= _sin_cache[key]
        return int_to_limbs(ival)

    # convert fraction => mpf
    angle_mpf= hpcf_to_mpmath(angle)
    s_val= mpmath.sin(angle_mpf)
    scaled= s_val* amplitude
    # convert => HPC-limb int
    # nearest integer
    i_val= int(mpmath.nint(scaled))
    _sin_cache[key]= i_val
    return int_to_limbs(i_val)

###############################################################################
# 2) HPC-limb wave => map to [-70..+40] mV (C. elegans membrane potential)
###############################################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array) -> float:
    """
    Interpret HPC-limb wave state in [0..wave_max] => [-70..+40] mV range (110 mV total).
    """
    wv = limbs_to_int(wave_limb)
    mx = limbs_to_int(wave_max)
    if mx==0:
        return -70.0
    fraction= wv/mx
    return -70.0 + 110.0*fraction

###############################################################################
# HPCWaveSegment => each segment's HPC-limb state + thresholds
###############################################################################

class HPCWaveSegment:
    """
    HPC-limb segment for modeling a C. elegans muscle/neuron chunk.

    wave_state: HPC-limb integer representing membrane potential in [0..wave_max],
                which translates to [-70..+40] mV in HPCWave_to_mV.

    We define:
      CaThreshold => ~2% of wave_max (EGL-19 L-type Ca2+ channel).
      KThreshold  => ~3% of wave_max (SHK-1, SLO-2 K+ channels).
      We'll apply a small conduction fraction (wave//8) to neighbors,
      a small reflection fraction (wave//16) at boundaries,
      plus a continuous leak wave//32 (TWK-7).
    """
    def __init__(self, wave_max_val=1_000_000, start_val=10_000):
        self.wave_max = int_to_limbs(wave_max_val)
        self.wave_state= int_to_limbs(start_val)

        ca_val= int(wave_max_val*0.02) # ~2%
        k_val = int(wave_max_val*0.03) # ~3%
        self.ca_threshold= int_to_limbs(ca_val)
        self.k_threshold=  int_to_limbs(k_val)

        # conduction backlog => wave from neighbors arrives next step
        self.toggle_history= []
        self.conduction_amplitude_history= []

        # velocity => wave(t)- wave(t-1) for debugging
        self.prev_int= start_val

###############################################################################
# HPCDebugData => store step-by-step logs (like conduction_in, reflection_sub, etc.)
###############################################################################

class HPCDebugData:
    def __init__(self):
        self.data= {}

    def record_val(self, step:int, seg_i:int, key:str, val:int):
        """
        Store an integer 'val' (like conduction_in, wave_final, velocity, etc.)
        under data[step][seg_i][key].
        """
        if step not in self.data:
            self.data[step] = {}
        if seg_i not in self.data[step]:
            self.data[step][seg_i] = {}
        self.data[step][seg_i][key] = val

    def fetch(self, step:int, seg_i:int, key:str, default=0)->int:
        return self.data.get(step,{}).get(seg_i,{}).get(key, default)

###############################################################################
# random/harmonic input => segment0 each step (like external synaptic drive)
###############################################################################

def random_harmonic_input(t:int, seg: HPCWaveSegment, freq_list=None):
    """
    Sum of 1 or 2 sines + random noise => HPC-limb integer => added to wave_state of seg0.
    This can represent external drive to the system at each time step.
    """
    if freq_list is None:
        freq_list = [0.05, 0.10]
    total=0
    for freq in freq_list:
        phase= 2*math.pi*freq*t
        val_f= 1000.0*math.sin(phase)
        total += int(val_f)

    # small random noise
    noise= random.randint(-50,50)
    total += noise
    if total<0:
        total=0
    # HPC-limb add
    seg.wave_state= hpc_add(seg.wave_state, int_to_limbs(total))

###############################################################################
# conduction => wave//8 to neighbors with 1-step delay
###############################################################################

def conduction_step(segments, conduction_delay:int, dbg:HPCDebugData, step:int):
    """
    Each segment's conduction_out => wave_state//8,
    stored for neighbors. Next time step, that conduction is added to wave_state
    (conduction_in).
    """
    n= len(segments)
    zeroA= int_to_limbs(0)
    toggles= [zeroA]*n
    conduction_vals= [zeroA]*n

    for i, seg in enumerate(segments):
        out_val= hpc_shr(seg.wave_state,3)  # wave//8
        conduction_vals[i]= out_val
        if dbg:
            dbg.record_val(step, i, "GapJ_Out", limbs_to_int(out_val))

    # pass conduction to neighbors
    for i, seg in enumerate(segments):
        cval= conduction_vals[i]
        if hpc_compare(cval, zeroA)>0:
            if i>0:
                toggles[i-1]= cval
            if i<n-1:
                toggles[i+1]= cval

    # conduction delay => apply old toggles
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            inflow= seg.toggle_history.pop(0)
            seg.wave_state= hpc_add(seg.wave_state, inflow)
            if dbg:
                dbg.record_val(step, i, "GapJ_In", limbs_to_int(inflow))
        else:
            if dbg:
                dbg.record_val(step, i, "GapJ_In", 0)
        seg.toggle_history.append(toggles[i])

###############################################################################
# Ion channels => EGL-19 Ca in => wave += wave//16, SLO-2 K out => wave -= wave//8
###############################################################################

def ion_channels(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_i:int):
    """
    If wave>ca_threshold => HPC-limb partial increment => Ca2+ inward current.
    If wave>k_threshold => HPC-limb partial decrement => K+ outflow.
    """
    wave_int_pre= limbs_to_int(seg.wave_state)
    if dbg:
        # record bits to interpret
        cab= wave_int_pre & 0xFFF
        kb = (wave_int_pre>>12) & 0xFFF
        dbg.record_val(step, seg_i, "Ca_bits", cab)
        dbg.record_val(step, seg_i, "K_bits", kb)

    # Ca => wave += wave//16
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4)
        seg.wave_state= hpc_add(seg.wave_state, plus_val)

    # K => wave -= wave//8
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3)
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

###############################################################################
# TWK-7 => leak => wave -= wave//32 => ~3%
###############################################################################

def leak_current(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, seg_i:int):
    """
    HPC-limb leak => wave -= wave//32, ensuring a stable resting potential
    if conduction + Ca push isn't strong enough.
    """
    wave_pre= limbs_to_int(seg.wave_state)
    if wave_pre>0:
        leak_val= hpc_shr(seg.wave_state,5)  # wave//32
        seg.wave_state= hpc_sub(seg.wave_state, leak_val)
        if dbg:
            dbg.record_val(step, seg_i, "Leak_Sub", limbs_to_int(leak_val))

###############################################################################
# partial reflection => wave -= wave//16 at boundaries
###############################################################################

def partial_reflection(segments, dbg:HPCDebugData, step:int):
    """
    HPC-limb reflection at left & right boundaries => wave -= wave//16,
    approximating partial sealing or boundary impedance.
    """
    if not segments:
        return
    left= segments[0]
    right= segments[-1]

    subL= hpc_shr(left.wave_state,4)
    left.wave_state= hpc_sub(left.wave_state, subL)
    if dbg:
        dbg.record_val(step, 0, "BoundaryReflect", limbs_to_int(subL))

    subR= hpc_shr(right.wave_state,4)
    right.wave_state= hpc_sub(right.wave_state, subR)
    if dbg:
        dbg.record_val(step, len(segments)-1, "BoundaryReflect", limbs_to_int(subR))

###############################################################################
# velocity => wave(t)- wave(t-1)
###############################################################################

def compute_velocity(seg: HPCWaveSegment):
    """
    HPC-limb velocity => final_wave(t) - final_wave(t-1).
    Negative means net repolarization or wave dropping,
    positive => net depolarization or rising wave amplitude.
    """
    curr_int= limbs_to_int(seg.wave_state)
    vel= curr_int - seg.prev_int
    seg.prev_int= curr_int
    return vel

###############################################################################
# main update => conduction -> ion channels -> leak -> reflection
###############################################################################

def update_hpc_wave(segments, dbg:HPCDebugData, step:int):
    # record wave_pre
    for i, seg in enumerate(segments):
        wave_pre= limbs_to_int(seg.wave_state)
        if dbg:
            dbg.record_val(step, i, "Wave_pre", wave_pre)

    # random/harmonic injection => seg0
    random_harmonic_input(step, segments[0])

    # conduction => wave//8
    conduction_step(segments, conduction_delay=1, dbg=dbg, step=step)

    # ion channels => Ca in, K out
    for i, seg in enumerate(segments):
        ion_channels(seg, dbg, step, i)

    # leak => wave//32
    for i, seg in enumerate(segments):
        leak_current(seg, dbg, step, i)

    # record wave_postIon
    if dbg:
        for i, seg in enumerate(segments):
            wv_post= limbs_to_int(seg.wave_state)
            dbg.record_val(step, i, "Wave_postIon", wv_post)

    # boundary reflection => wave//16 at ends
    partial_reflection(segments, dbg, step)

    # final wave & velocity
    for i, seg in enumerate(segments):
        wv_final= limbs_to_int(seg.wave_state)
        if dbg:
            dbg.record_val(step, i, "Wave_final", wv_final)
            vel= compute_velocity(seg)
            dbg.record_val(step, i, "Velocity", vel)
        else:
            compute_velocity(seg)  # not storing if no debug

###############################################################################
# small run => 8 segments, 50 steps => debug + plot
###############################################################################

def run_c_elegans_small(num_segments=8, steps=50):
    segs=[]
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    # excite seg0 => wave=20,000 => ~2%
    segs[0].wave_state= int_to_limbs(20_000)
    dbg= HPCDebugData()
    for step in range(steps):
        update_hpc_wave(segs, dbg, step)
    return segs, dbg

###############################################################################
# large => stress test => skip debug logs
###############################################################################

def run_c_elegans_stress_test(num_segments=3000, steps=20):
    segs=[]
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000, start_val=10_000))
    segs[0].wave_state= int_to_limbs(20_000)

    t0= time.perf_counter()
    for step in range(steps):
        update_hpc_wave(segs, None, step)
    t1= time.perf_counter()
    dur= (t1-t0)

    # final sum
    final_sum=0
    for s in segs:
        final_sum += limbs_to_int(s.wave_state)
    return dur, final_sum

###############################################################################
# plot => HPCWave_to_mV => c.elegans in [-70..+40]
###############################################################################

def plot_debug_data_c_elegans(segs, dbg:HPCDebugData, steps=50):
    """
    Show multiple subplots:
      1) HPC-limb wave => mV
      2) Gap-Junction In
      3) Boundary Reflection
      4) Ca & K bits
      5) Velocity => sum line
      6) Final distribution of wave_mV
    """
    num_segs= len(segs)
    time_axis= np.arange(steps)

    wave_mV= np.zeros((steps, num_segs), dtype=float)
    velocity= np.zeros((steps, num_segs), dtype=float)
    gap_in   = np.zeros((steps, num_segs), dtype=float)
    reflect  = np.zeros((steps, num_segs), dtype=float)
    ca_bits  = np.zeros((steps, num_segs), dtype=float)
    k_bits   = np.zeros((steps, num_segs), dtype=float)

    for step in range(steps):
        for i in range(num_segs):
            wv_final = dbg.fetch(step, i, "Wave_final", 0)
            wave_limb= int_to_limbs(wv_final)
            wave_mV[step, i] = HPCWave_to_mV(wave_limb, segs[i].wave_max)

            gap_in[step, i] = dbg.fetch(step, i, "GapJ_In", 0)
            reflect[step, i]= dbg.fetch(step, i, "BoundaryReflect", 0)
            ca_bits[step, i]= dbg.fetch(step, i, "Ca_bits", 0)
            k_bits[step, i] = dbg.fetch(step, i, "K_bits", 0)
            velocity[step, i]= dbg.fetch(step, i, "Velocity", 0)

    fig, axes= plt.subplots(3,2, figsize=(16,14))
    ax1, ax2, ax3, ax4, ax5, ax6= axes.flatten()

    # 1) HPC-limb wave => mV
    for i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:, i], label=f"S{i}")
    ax1.set_title("C. elegans HPC-limb wave => [-70..+40] mV")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.grid(True)
    ax1.legend()

    # 2) Gap-Junction In
    for i in range(num_segs):
        ax2.plot(time_axis, gap_in[:, i], alpha=0.7)
    ax2.set_title("Gap-Junction Inflow (wave//8 from neighbors)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude (int)")
    ax2.grid(True)

    # 3) Boundary Reflection
    for i in range(num_segs):
        ax3.plot(time_axis, reflect[:, i], alpha=0.7)
    ax3.set_title("Boundary Reflection (wave//16) at ends")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude (int)")
    ax3.grid(True)

    # 4) Ca & K bits => HPC-limb lower bits
    for i in range(num_segs):
        ax4.plot(time_axis, ca_bits[:, i], alpha=0.7)
        ax4.plot(time_axis, k_bits[:, i], '--', alpha=0.7)
    ax4.set_title("Ca & K Bits from HPC-limb State (EGL-19 vs. SLO-2 etc.)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Bits (0..4095)")
    ax4.grid(True)

    # 5) velocity => sum
    sum_vel= np.sum(velocity, axis=1)
    for i in range(num_segs):
        ax5.plot(time_axis, velocity[:, i], alpha=0.4)
    ax5.plot(time_axis, sum_vel, 'k-', linewidth=2, label="Sum velocity")
    ax5.set_title("Velocity => wave(t)-wave(t-1) (dep/repol changes)")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Velocity (int HPC-limb)")
    ax5.grid(True)
    ax5.legend()

    # 6) final distribution => wave_mV last step
    final_mV= wave_mV[-1,:]
    ax6.hist(final_mV, bins=30, alpha=0.7, color='orange')
    ax6.set_title("Final HPC-limb wave distribution (mV) after 50 steps")
    ax6.set_xlabel("Voltage (mV)")
    ax6.set_ylabel("Count")
    ax6.grid(True)

    plt.tight_layout()
    return fig

###############################################################################
# main => run small + stress test
###############################################################################

def main():
    print("1) Running small HPC-limb c.elegans model with debug + plots (8 seg, 50 steps)")
    segs_small, dbg_small= run_c_elegans_small(num_segments=8, steps=50)
    fig= plot_debug_data_c_elegans(segs_small, dbg_small, steps=50)
    plt.show()

    print("\n2) HPC-limb wave stress test: 3000 segments, 20 steps\n")
    t0= time.perf_counter()
    dur, final_sum= run_c_elegans_stress_test(num_segments=3000, steps=20)
    t1= time.perf_counter()
    print(f"Stress test loop => {t1-t0:.4f}s, wave updates => {dur:.4f}s")
    print(f"Final sum of wave states => {final_sum}")

if __name__=="__main__":
    main()

In [ ]:
#!/usr/bin/env python3
import math, random
import mpmath  # for high-precision sine
from array import array
from typing import Tuple, Dict

mpmath.mp.prec = 200  # e.g. 200 bits of precision

###############################################################################
# A) HPC-limb integer code (64-bit chunk)
###############################################################################

CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    """Nonnegative Python int -> HPC-limb array (64-bit)."""
    if value<0:
        raise ValueError("Negative ints not supported.")
    limbs = array('Q')
    while value>0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val=0
    shift=0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*(out_len+1))
    carry=0
    for i in range(out_len):
        av= A[i] if i<len(A) else 0
        bv= B[i] if i<len(B) else 0
        s_val= av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry= s_val >> CHUNK_BITS
    if carry:
        out[out_len]= carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len= max(len(A), len(B))
    out= array('Q',[0]*out_len)
    carry=0
    for i in range(out_len):
        av= A[i] if i<len(A) else 0
        bv= B[i] if i<len(B) else 0
        diff= av - bv - carry
        if diff<0:
            diff += CHUNK_BASE
            carry=1
        else:
            carry=0
        out[i] = diff & CHUNK_MASK
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb= len(A), len(B)
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top= max(la, lb)
    for i in range(top-1, -1, -1):
        av= A[i] if i<la else 0
        bv= B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts=  shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

###############################################################################
# B) HPC-limb fraction => HPCFraction
###############################################################################

class HPCFraction:
    """
    HPC-limb fraction => numerator, denominator each HPC-limb array (nonnegative).
    We store sign separately if needed, but let's keep it positive for angles.
    """
    __slots__ = ('num','den')

    def __init__(self, numerator: array, denominator: array):
        # assume gcd? or we can do it if we want
        if len(denominator)==1 and denominator[0]==0:
            raise ZeroDivisionError("Fraction denominator=0")
        self.num= numerator
        self.den= denominator

    @classmethod
    def from_int(cls, val: int) -> "HPCFraction":
        if val<0:
            raise ValueError("Negative HPCFraction not done in snippet.")
        return cls(int_to_limbs(val), int_to_limbs(1))

    def copy(self)->"HPCFraction":
        return HPCFraction(array('Q',self.num), array('Q',self.den))

def hpcf_add(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    """(A.num/A.den) + (B.num/B.den) => HPC-limb fraction."""
    # cross => A.num*B.den + B.num*A.den
    an= hpc_mul(A.num, B.den)
    bn= hpc_mul(B.num, A.den)
    ssum= hpc_add(an, bn)
    dden= hpc_mul(A.den, B.den)
    return HPCFraction(ssum, dden)

def hpcf_sub(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    an= hpc_mul(A.num, B.den)
    bn= hpc_mul(B.num, A.den)
    diff= hpc_sub(an, bn)
    dden= hpc_mul(A.den, B.den)
    # skip sign check for snippet
    return HPCFraction(diff, dden)

def hpcf_mul(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    nump= hpc_mul(A.num, B.num)
    denp= hpc_mul(A.den, B.den)
    return HPCFraction(nump, denp)

def hpcf_div(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    if len(B.num)==1 and B.num[0]==0:
        raise ZeroDivisionError("Divide fraction by 0 HPC-limb")
    nump= hpc_mul(A.num, B.den)
    denp= hpc_mul(A.den, B.num)
    return HPCFraction(nump, denp)

def hpc_mul(A: array, B: array)-> array:
    """Schoolbook HPC-limb multiply => array of size <= len(A)+len(B)."""
    la, lb= len(A), len(B)
    out= array('Q',[0]*(la+lb))
    for i in range(la):
        carry=0
        av= A[i]
        for j in range(lb):
            mul_val= av*B[j] + out[i+j] + carry
            out[i+j]= mul_val & CHUNK_MASK
            carry= mul_val >> CHUNK_BITS
        if carry:
            out[i+lb]+= carry
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

def hpcf_to_mpmath(A: HPCFraction)-> mpmath.mpf:
    """
    Convert HPCFraction => mpmath.mpf for trig, etc.
    """
    an= limbs_to_int(A.num)
    ad= limbs_to_int(A.den)
    return mpmath.mpf(an)/mpmath.mpf(ad)

###############################################################################
# C) HPC-limb-based angle => caching for sin
###############################################################################

_sin_cache: Dict[Tuple[int,int], int] = {}  
# key = (num_int, den_int), value= HPC-limb int amplitude => e.g. scaled sin

def hpcf_sin(angle: HPCFraction, amplitude: int=10000)-> array:
    """
    Return HPC-limb integer for sin(angle * pi?), depending on how you define angle.
    We'll do sin(angle) in radians if angle is fraction of rad.
    amplitude => scale final => HPC-limb integer.

    We do caching:
      key = (num_of_angle, den_of_angle) => HPC-limb int
    """
    an= limbs_to_int(angle.num)
    ad= limbs_to_int(angle.den)
    key= (an, ad, amplitude)
    if key in _sin_cache:
        # Already computed
        ival= _sin_cache[key]
        return int_to_limbs(ival)

    # convert fraction => mpf
    angle_mpf= hpcf_to_mpmath(angle)
    s_val= mpmath.sin(angle_mpf)
    scaled= s_val* amplitude
    # convert => HPC-limb int
    # nearest integer
    i_val= int(mpmath.nint(scaled))
    _sin_cache[key]= i_val
    return int_to_limbs(i_val)

###############################################################################
# D) Example usage => HPC-limb wave conduction
###############################################################################

def example_run(steps=20):
    """
    We'll define freq= HPCFraction(1,20) => 0.05 rad/step, or something
    We'll define angle(t)= freq * HPCFraction(t,1)
    Then sin => HPC-limb int => add to wave state
    """
    # HPC-limb wave (like your HPCWaveSegment)
    wave_state= int_to_limbs(0)
    wave_max_val= int_to_limbs(1_000_000)  # e.g.
    # freq => HPCFraction(1,20)
    freq= HPCFraction(int_to_limbs(1), int_to_limbs(20))

    for t in range(steps):
        # angle = freq * t
        Tfrac= HPCFraction(int_to_limbs(t), int_to_limbs(1))
        angle= hpcf_mul(freq, Tfrac)
        # compute sin => HPC-limb integer
        sin_val= hpcf_sin(angle, amplitude=5000)  
        # HPC-limb add
        wave_state= hpc_add(wave_state, sin_val)

        # optional conduction, thresholds, etc. 
        # ...

        # Print wave
        wave_int= limbs_to_int(wave_state)
        print(f"Step {t}: angle={t*0.05} rad (approx), sin_val= {limbs_to_int(sin_val)}  wave= {wave_int}")

def main():
    example_run(steps=10)

if __name__=="__main__":
    import mpmath
    main()

In [ ]:
#!/usr/bin/env python3

"""
Final HPC-limb Integration for C. elegans–Inspired Model
========================================================

Features:
1) HPC-limb wave states for each segment (membrane potential).
2) HPCFraction for storing frequencies/angles in exact rational form.
3) High-precision sine (via mpmath) with caching -> HPC-limb integer injection.
4) Biological naming: EGL-19 Ca threshold, SHK-1/SLO-2 K threshold, TWK-7 leak,
   partial reflection at boundaries, conduction akin to gap-junction coupling.
5) Voltage range mapped to [-70..+40] mV (110 mV total).
6) Strong conduction => wave//4, threshold at ~ -35 mV for Ca, -20 mV for K.
7) Two runs:
   - Small (8 seg, 50 steps) with debug & plots.
   - Stress (3000 seg, 20 steps) no debug.
"""

import math
import random
import time
import mpmath
from array import array
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Tuple

mpmath.mp.prec = 200  # 200 bits of precision for sine

###############################################################################
# A) HPC-limb integer code (64-bit chunks)
###############################################################################
CHUNK_BITS = 64
CHUNK_BASE = 1 << CHUNK_BITS
CHUNK_MASK = CHUNK_BASE - 1

def int_to_limbs(value: int) -> array:
    if value < 0:
        raise ValueError("Negative ints not supported.")
    limbs = array('Q')
    while value > 0:
        limbs.append(value & CHUNK_MASK)
        value >>= CHUNK_BITS
    if not limbs:
        limbs.append(0)
    return limbs

def limbs_to_int(limbs: array) -> int:
    val = 0
    shift = 0
    for limb in limbs:
        val += (limb << shift)
        shift += CHUNK_BITS
    return val

def hpc_add(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q',[0]*(out_len+1))
    carry = 0
    for i in range(out_len):
        av = A[i] if i < len(A) else 0
        bv = B[i] if i < len(B) else 0
        s_val = av + bv + carry
        out[i] = s_val & CHUNK_MASK
        carry = s_val >> CHUNK_BITS
    if carry:
        out[out_len] = carry
    else:
        out.pop()
    return out

def hpc_sub(A: array, B: array) -> array:
    out_len = max(len(A), len(B))
    out = array('Q',[0]*out_len)
    carry = 0
    for i in range(out_len):
        av = A[i] if i < len(A) else 0
        bv = B[i] if i < len(B) else 0
        diff = av - bv - carry
        if diff < 0:
            diff += CHUNK_BASE
            carry = 1
        else:
            carry = 0
        out[i] = diff & CHUNK_MASK
    while len(out) > 1 and out[-1] == 0:
        out.pop()
    return out

def hpc_compare(A: array, B: array)->int:
    la, lb = len(A), len(B)
    if la>lb and A[-1]!=0:
        return 1
    elif lb>la and B[-1]!=0:
        return -1
    top = max(la, lb)
    for i in range(top-1, -1, -1):
        av = A[i] if i<la else 0
        bv = B[i] if i<lb else 0
        if av>bv: return 1
        if av<bv: return -1
    return 0

def hpc_shr(A: array, shift_bits:int) -> array:
    if shift_bits<=0:
        return array('Q', A)
    out= array('Q', A)
    limb_shifts= shift_bits // CHUNK_BITS
    bit_shifts= shift_bits % CHUNK_BITS
    if limb_shifts>= len(out):
        return array('Q',[0])
    out= out[limb_shifts:]
    if bit_shifts==0:
        if not out:
            out.append(0)
        return out
    carry=0
    for i in reversed(range(len(out))):
        cur = out[i] | (carry<<CHUNK_BITS)
        out[i] = (cur>> bit_shifts) & CHUNK_MASK
        carry  = cur & ((1<<bit_shifts)-1)
    while len(out)>1 and out[-1]==0:
        out.pop()
    if not out:
        out.append(0)
    return out

def hpc_mul(A: array, B: array)-> array:
    la, lb= len(A), len(B)
    out= array('Q',[0]*(la+lb))
    for i in range(la):
        carry=0
        av= A[i]
        for j in range(lb):
            mul_val= av*B[j] + out[i+j] + carry
            out[i+j]= mul_val & CHUNK_MASK
            carry= mul_val >> CHUNK_BITS
        if carry:
            out[i+lb]+= carry
    while len(out)>1 and out[-1]==0:
        out.pop()
    return out

###############################################################################
# B) HPC-limb fraction => HPCFraction
###############################################################################

class HPCFraction:
    __slots__ = ('num','den')
    def __init__(self, numerator: array, denominator: array):
        # no gcd for snippet
        if len(denominator)==1 and denominator[0]==0:
            raise ZeroDivisionError("Denominator=0 HPCFraction")
        self.num= numerator
        self.den= denominator

    @classmethod
    def from_int(cls, val: int)->"HPCFraction":
        if val<0:
            raise ValueError("Negative HPCFraction not in snippet.")
        return cls(int_to_limbs(val), int_to_limbs(1))

    def copy(self)->"HPCFraction":
        return HPCFraction(array('Q',self.num), array('Q',self.den))

def hpcf_mul(A: HPCFraction, B: HPCFraction)-> HPCFraction:
    nup= hpc_mul(A.num, B.num)
    dep= hpc_mul(A.den, B.den)
    return HPCFraction(nup, dep)

def hpcf_to_mpmath(A: HPCFraction)-> mpmath.mpf:
    an= limbs_to_int(A.num)
    ad= limbs_to_int(A.den)
    return mpmath.mpf(an)/mpmath.mpf(ad)

###############################################################################
# C) HPC-limb wave => [-70..+40] mV mapping
###############################################################################

def HPCWave_to_mV(wave_limb: array, wave_max: array) -> float:
    wv= limbs_to_int(wave_limb)
    mx= limbs_to_int(wave_max)
    if mx<=0:
        return -70.0
    fraction= wv/mx
    return -70.0 + 110.0*fraction

###############################################################################
# D) HPC-limb Sine => caching
###############################################################################

_sin_cache: Dict[Tuple[int,int,int], int] = {}

def hpcf_sin(angle: HPCFraction, amplitude=5000)-> array:
    """
    Convert HPCFraction angle => mpmath sine => HPC-limb int scaled by amplitude.
    Cache the result for repeated angles.
    """
    an= limbs_to_int(angle.num)
    ad= limbs_to_int(angle.den)
    key= (an, ad, amplitude)
    if key in _sin_cache:
        i_val= _sin_cache[key]
        return int_to_limbs(i_val)

    # compute via mpmath
    angle_mp= hpcf_to_mpmath(angle)
    s_val= mpmath.sin(angle_mp)
    scaled= s_val* amplitude
    i_val= int(mpmath.nint(scaled))
    _sin_cache[key]= i_val
    return int_to_limbs(i_val)

###############################################################################
# E) HPCWaveSegment => c.elegans logic
###############################################################################

class HPCWaveSegment:
    """
    HPC-limb wave segment with more aggressive K threshold and higher Ca threshold.
      - K channel ~ -30 mV => fraction ~ 0.40
      - Ca channel ~ -35 mV => fraction ~ 0.318 or 0.35, your preference
    """
    def __init__(self, wave_max_val=1_000_000):
        self.wave_max = int_to_limbs(wave_max_val)
        # start ~ 5% of wave_max
        start_val = int(wave_max_val * 0.05)
        self.wave_state = int_to_limbs(start_val)

        # if you want exactly -30 => 0.364, -35 => 0.318. Let's pick:
        k_f = 0.40    # ~ -30 mV
        ca_f = 0.35   # ~ -35 mV
        k_thr = int(wave_max_val * k_f)
        ca_thr = int(wave_max_val * ca_f)
        self.k_threshold = int_to_limbs(k_thr)
        self.ca_threshold = int_to_limbs(ca_thr)

        self.toggle_history = []
        self.prev_int = start_val

###############################################################################
# HPCDebugData for logs
###############################################################################

class HPCDebugData:
    def __init__(self):
        self.data= {}

    def record_val(self, step, segi, key, val):
        if step not in self.data:
            self.data[step]= {}
        if segi not in self.data[step]:
            self.data[step][segi]= {}
        self.data[step][segi][key]= val

    def fetch(self, step, segi, key, default=0):
        return self.data.get(step,{}).get(segi,{}).get(key, default)

###############################################################################
# Conduction => wave//4 => neighbors with 1-step delay
###############################################################################

def conduction_step(segments, conduction_delay: int, dbg: HPCDebugData, step: int):
    """
    Instead of wave//4, use wave//6 for more conservative conduction.
    """
    n = len(segments)
    zeroA = int_to_limbs(0)
    toggles = [zeroA]*n

    for i, seg in enumerate(segments):
        c_out = hpc_shr(seg.wave_state, 6)  # wave//6 conduction
        if dbg:
            dbg.record_val(step, i, "GapJ_Out", limbs_to_int(c_out))
        # pass to neighbors
        if i > 0:
            toggles[i-1] = c_out
        if i < n-1:
            toggles[i+1] = c_out

    # apply conduction after backlog
    for i, seg in enumerate(segments):
        if len(seg.toggle_history) >= conduction_delay:
            inflow = seg.toggle_history.pop(0)
            seg.wave_state = hpc_add(seg.wave_state, inflow)
            if dbg:
                dbg.record_val(step, i, "GapJ_In", limbs_to_int(inflow))
        else:
            if dbg:
                dbg.record_val(step, i, "GapJ_In", 0)
        seg.toggle_history.append(toggles[i])

###############################################################################
# Ion channels => Ca => wave += wave//16 if wave>ca_threshold,
#                    K => wave -= wave//8  if wave>k_threshold
###############################################################################

def ion_channels(seg: HPCWaveSegment, dbg:HPCDebugData, step:int, segi:int):
    wave_pre= limbs_to_int(seg.wave_state)
    if dbg:
        # store bits
        ca_b= wave_pre & 0xFFF
        k_b= (wave_pre>>12)&0xFFF
        dbg.record_val(step, segi, "Ca_bits", ca_b)
        dbg.record_val(step, segi, "K_bits", k_b)

    # Ca
    if hpc_compare(seg.wave_state, seg.ca_threshold)>0:
        plus_val= hpc_shr(seg.wave_state,4) # wave//16
        seg.wave_state= hpc_add(seg.wave_state, plus_val)

    # K
    if hpc_compare(seg.wave_state, seg.k_threshold)>0:
        minus_val= hpc_shr(seg.wave_state,3) # wave//8
        seg.wave_state= hpc_sub(seg.wave_state, minus_val)

###############################################################################
# Leak => wave -= wave//32 => ~3%
###############################################################################

def leak_current(seg: HPCWaveSegment, dbg: HPCDebugData, step: int, segi: int):
    """
    Make leak wave//16 instead of wave//32 => ~6% each step if wave is large.
    """
    wave_pre = limbs_to_int(seg.wave_state)
    if wave_pre > 0:
        leak_val = hpc_shr(seg.wave_state, 4)  # wave//16
        seg.wave_state = hpc_sub(seg.wave_state, leak_val)
        if dbg:
            dbg.record_val(step, segi, "Leak_sub", limbs_to_int(leak_val))

###############################################################################
# Reflection => wave -= wave//16 at left, right boundaries
###############################################################################

def partial_reflection(segments, dbg:HPCDebugData, step:int):
    if not segments:
        return
    left= segments[0]
    right= segments[-1]
    subL= hpc_shr(left.wave_state,4)
    left.wave_state= hpc_sub(left.wave_state, subL)
    if dbg:
        dbg.record_val(step, 0, "BoundaryRef", limbs_to_int(subL))
    subR= hpc_shr(right.wave_state,4)
    right.wave_state= hpc_sub(right.wave_state, subR)
    if dbg:
        dbg.record_val(step, len(segments)-1, "BoundaryRef", limbs_to_int(subR))

###############################################################################
# velocity => wave(t) - wave(t-1)
###############################################################################

def compute_velocity(seg: HPCWaveSegment):
    curr= limbs_to_int(seg.wave_state)
    vel= curr- seg.prev_int
    seg.prev_int= curr
    return vel

###############################################################################
# HPCFraction-based sine injection => e.g. freq= HPCFraction(1,20)
###############################################################################

def fraction_sine_injection(t:int, freq: "HPCFraction", amplitude=5000)-> int:
    """
    angle = freq * HPCFraction(t,1), call hpcf_sin => HPC-limb int.
    returns python int for easy wave addition => then HPC-limb again
    or you can directly do HPC-limb add.
    """
    from_int= HPCFraction(int_to_limbs(t), int_to_limbs(1))
    angle= hpcf_mul(freq, from_int)  # HPC-limb fraction
    sin_limb= hpcf_sin(angle, amplitude)
    return limbs_to_int(sin_limb)

###############################################################################
# Single step update => conduction->ion->leak->reflection + optional sine injection
###############################################################################

def update_hpc_wave(segments, dbg:HPCDebugData, step:int, freq:HPCFraction=None, amplitude=5000):
    # 1) optional HPCFraction-based sine injection => seg0 each step
    if freq is not None:
        injection_val= fraction_sine_injection(step, freq, amplitude)
        # HPC-limb add => seg[0]
        segs0 = segments[0]
        segs0.wave_state= hpc_add(segs0.wave_state, int_to_limbs(injection_val))
        if dbg:
            dbg.record_val(step, 0, "SineInjection", injection_val)

    # 2) conduction => wave//4
    conduction_step(segments, conduction_delay=1, dbg=dbg, step=step)

    # 3) ion channels => Ca & K
    for i, seg in enumerate(segments):
        ion_channels(seg, dbg, step, i)

    # 4) leak => wave//32
    for i, seg in enumerate(segments):
        leak_current(seg, dbg, step, i)

    # record wave_postIon or wave_preRef
    if dbg:
        for i, seg in enumerate(segments):
            dbg.record_val(step, i, "Wave_preRef", limbs_to_int(seg.wave_state))

    # 5) reflection => wave//16 at edges
    partial_reflection(segments, dbg, step)

    # final wave + velocity
    for i, seg in enumerate(segments):
        wv_fin= limbs_to_int(seg.wave_state)
        if dbg:
            dbg.record_val(step, i, "Wave_final", wv_fin)
            vel= compute_velocity(seg)
            dbg.record_val(step, i, "Velocity", vel)
        else:
            compute_velocity(seg)  # not storing if no debug

###############################################################################
# Small run => 8 seg, 50 steps => HPCFraction freq= 1/20 => 0.05 rad
###############################################################################

class HPCDebugData:
    def __init__(self):
        self.data= {}
    def record_val(self, step, segi, key, val):
        if step not in self.data:
            self.data[step]= {}
        if segi not in self.data[step]:
            self.data[step][segi]= {}
        self.data[step][segi][key]= val
    def fetch(self, step, segi, key, default=0):
        return self.data.get(step,{}).get(segi,{}).get(key, default)

def run_small_hpc_celegans(num_segments=8, steps=50):
    segs=[]
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000))
    dbg= HPCDebugData()
    # define freq => HPCFraction(1,20) => angle=0.05 per step
    freq= HPCFraction(int_to_limbs(1), int_to_limbs(20))
    for step in range(steps):
        update_hpc_wave(segs, dbg, step, freq=freq, amplitude=5000)
    return segs, dbg

def plot_small_run(segs, dbg, steps=50):
    """Similar multi-subplot approach as before"""
    # gather wave_mV, velocity, conduction_in, reflection_sub, Ca bits, K bits...
    import numpy as np
    time_axis= np.arange(steps)
    num_segs= len(segs)
    wave_mV= np.zeros((steps,num_segs))
    velocity= np.zeros((steps,num_segs))

    for st in range(steps):
        for i, seg in enumerate(segs):
            wv_fin= dbg.fetch(st, i, "Wave_final", 0)
            wave_mV[st,i]= HPCWave_to_mV(int_to_limbs(wv_fin), seg.wave_max)
            velocity[st,i]= dbg.fetch(st, i, "Velocity", 0)

    fig, axes= plt.subplots(2,1, figsize=(10,8))
    ax1, ax2= axes.flatten()

    # HPC-limb wave => mV
    for i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:, i], label=f"S{i}")
    ax1.set_title("HPC-limb wave => [-70..+40] mV, Sine Inject at seg0")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Voltage (mV)")
    ax1.grid(True)
    ax1.legend()

    # velocity => sum in black
    sum_vel= np.sum(velocity, axis=1)
    for i in range(num_segs):
        ax2.plot(time_axis, velocity[:, i], alpha=0.4)
    ax2.plot(time_axis, sum_vel, 'k-', linewidth=2, label="Sum velocity")
    ax2.set_title("Velocity => wave(t)-wave(t-1)")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Velocity (HPC-limb int)")
    ax2.grid(True)
    ax2.legend()

    plt.tight_layout()
    return fig

###############################################################################
# Stress test => no debug
###############################################################################

def run_stress_hpc_celegans(num_segments=3000, steps=20):
    segs=[]
    for i in range(num_segments):
        segs.append(HPCWaveSegment(wave_max_val=1_000_000))
    # define freq => HPCFraction(1,20)
    freq= HPCFraction(int_to_limbs(1), int_to_limbs(20))

    t0= time.perf_counter()
    for step in range(steps):
        # no debug
        update_hpc_wave(segs, None, step, freq=freq, amplitude=5000)
    t1= time.perf_counter()
    dur= (t1 - t0)

    # final sum
    final_sum=0
    for s in segs:
        final_sum+= limbs_to_int(s.wave_state)

    return dur, final_sum

###############################################################################
# main
###############################################################################

def main():
    print("=== HPC-limb C. elegans final integration (small run) ===")
    segs_small, dbg_small= run_small_hpc_celegans(num_segments=8, steps=50)
    fig= plot_small_run(segs_small, dbg_small, steps=50)
    plt.show()

    print("\n=== HPC-limb wave stress test => 3000 seg, 20 steps ===")
    dur, final_sum= run_stress_hpc_celegans(num_segments=3000, steps=20)
    print(f"Stress test => {dur:.4f}s total. Final wave sum => {final_sum}")

if __name__=="__main__":
    main()

In [ ]:
def plot_hpc_debug_full(segs, dbg, steps=50):
    import numpy as np
    import matplotlib.pyplot as plt

    num_segs = len(segs)
    time_axis = np.arange(steps)

    # We'll extract: wave_mV, conduction_in/out, reflection, Ca bits, K bits, velocity
    wave_mV = np.zeros((steps, num_segs))
    gap_in   = np.zeros((steps, num_segs))
    gap_out  = np.zeros((steps, num_segs))
    reflect  = np.zeros((steps, num_segs))
    ca_bits  = np.zeros((steps, num_segs))
    k_bits   = np.zeros((steps, num_segs))
    velocity = np.zeros((steps, num_segs))

    for step in range(steps):
        for seg_i, seg in enumerate(segs):
            # 1) Final HPC-limb wave in mV
            wv_final = dbg.fetch(step, seg_i, "Wave_final", 0)
            wave_mV[step, seg_i] = HPCWave_to_mV(int_to_limbs(wv_final), seg.wave_max)

            # 2) Gap-junction in/out
            gap_in[step, seg_i]  = dbg.fetch(step, seg_i, "GapJ_In", 0)
            gap_out[step, seg_i] = dbg.fetch(step, seg_i, "GapJ_Out", 0)

            # 3) Reflection
            reflect[step, seg_i] = dbg.fetch(step, seg_i, "BoundaryRef", 0)

            # 4) Ion bits
            ca_bits[step, seg_i] = dbg.fetch(step, seg_i, "Ca_bits", 0)
            k_bits[step, seg_i]  = dbg.fetch(step, seg_i, "K_bits", 0)

            # 5) Velocity
            velocity[step, seg_i] = dbg.fetch(step, seg_i, "Velocity", 0)

    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    ax1, ax2, ax3, ax4, ax5, ax6 = axes.flatten()

    # Panel 1) HPC-limb wave => mV
    for i in range(num_segs):
        ax1.plot(time_axis, wave_mV[:, i], label=f"Seg{i}")
    ax1.set_title("HPC-limb wave => [-70..+40] mV")
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Membrane Potential (mV)")
    ax1.grid(True)
    ax1.legend()

    # Panel 2) Gap-junction conduction in/out
    # Let's just show 'in' for each seg
    for i in range(num_segs):
        ax2.plot(time_axis, gap_in[:, i], label=f"In S{i}", alpha=0.6)
        # optionally also plot gap_out
    ax2.set_title("GapJ Inflow (wave//4) from neighbors")
    ax2.set_xlabel("Time Step")
    ax2.set_ylabel("Amplitude (int HPC-limb)")
    ax2.grid(True)
    ax2.legend()

    # Panel 3) Boundary reflection
    for i in range(num_segs):
        ax3.plot(time_axis, reflect[:, i], alpha=0.6)
    ax3.set_title("Boundary Reflection (wave//16) at edges")
    ax3.set_xlabel("Time Step")
    ax3.set_ylabel("Amplitude (int HPC-limb)")
    ax3.grid(True)

    # Panel 4) Ion channel bits => Ca/K
    for i in range(num_segs):
        ax4.plot(time_axis, ca_bits[:, i], alpha=0.7)
        ax4.plot(time_axis, k_bits[:, i], '--', alpha=0.7)
    ax4.set_title("Ca & K HPC-limb Bits (EGL-19 vs. SLO-2 etc.)")
    ax4.set_xlabel("Time Step")
    ax4.set_ylabel("Bits (0..4095 maybe)")
    ax4.grid(True)

    # Panel 5) Velocity
    sum_vel = np.sum(velocity, axis=1)
    for i in range(num_segs):
        ax5.plot(time_axis, velocity[:, i], alpha=0.4)
    ax5.plot(time_axis, sum_vel, 'k-', linewidth=2, label="Sum Velocity")
    ax5.set_title("Velocity => wave(t) - wave(t-1)")
    ax5.set_xlabel("Time Step")
    ax5.set_ylabel("Velocity (HPC-limb int)")
    ax5.grid(True)
    ax5.legend()

    # Panel 6) final HPC-limb wave distribution => wave_mV last step
    final_mV = wave_mV[-1, :]
    ax6.hist(final_mV, bins=30, alpha=0.7, color='orange')
    ax6.set_title("Final HPC-limb wave distribution (mV)")
    ax6.set_xlabel("Voltage (mV)")
    ax6.set_ylabel("Count")
    ax6.grid(True)

    plt.tight_layout()
    return fig

def main():
    print("=== HPC-limb C. elegans final integration (small run) ===")
    segs_small, dbg_small= run_small_hpc_celegans(num_segments=8, steps=50)
    fig= plot_hpc_debug_full(segs_small, dbg_small, steps=50)
    plt.show()

    print("\n=== HPC-limb wave stress test => 3000 seg, 20 steps ===")
    dur, final_sum= run_stress_hpc_celegans(num_segments=3000, steps=20)
    print(f"Stress test => {dur:.4f}s total. Final wave sum => {final_sum}")

if __name__=="__main__":
    main()
